A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
kernelDensityEstimationExample.m

% Data
randn('seed',1)                            % Used for reproducibility
data = [randn(100,1)-10; randn(100,1)+10]; % Two Normals mixed
% True
phi = @(x) exp(-.5*x.^2)/sqrt(2*pi);       % Normal Density
tpdf = @(x) phi(x+10)/2+phi(x-10)/2;       % True Density
% Kernel
h = std(data)*(4/3/numel(data))^(1/5);     % Bandwidth estimated by Silverman's Rule of Thumb
kernel = @(x) mean(phi((x-data)/h)/h);     % Kernel Density
kpdf = @(x) arrayfun(kernel,x);            % Elementwise application
% Plot
figure(2), clf, hold on
x = linspace(-25,+25,1000);                % Linear Space
plot(x,tpdf(x))                            % Plot True Density
plot(x,kpdf(x))                            % Plot Kernel Density with Silverman's Rule of Thumb
kde(data)                                  % Plot Kernel Density with Solve-the-Equation Bandwidth

In [ ]:
function [bandwidth,density,xmesh,cdf]=kde(data,n,MIN,MAX)
% Reliable and extremely fast kernel density estimator for one-dimensional data;
%        Gaussian kernel is assumed and the bandwidth is chosen automatically;
%        Unlike many other implementations, this one is immune to problems
%        caused by multimodal densities with widely separated modes (see example). The
%        estimation does not deteriorate for multimodal densities, because we never assume
%        a parametric model for the data.
% INPUTS:
%     data    - a vector of data from which the density estimate is constructed;
%          n  - the number of mesh points used in the uniform discretization of the
%               interval [MIN, MAX]; n has to be a power of two; if n is not a power of two, then
%               n is rounded up to the next power of two, i.e., n is set to n=2^ceil(log2(n));
%               the default value of n is n=2^12;
%   MIN, MAX  - defines the interval [MIN,MAX] on which the density estimate is constructed;
%               the default values of MIN and MAX are:
%               MIN=min(data)-Range/10 and MAX=max(data)+Range/10, where Range=max(data)-min(data);
% OUTPUTS:
%   bandwidth - the optimal bandwidth (Gaussian kernel assumed);
%     density - column vector of length 'n' with the values of the density
%               estimate at the grid points;
%     xmesh   - the grid over which the density estimate is computed;
%             - If no output is requested, then the code automatically plots a graph of
%               the density estimate.
%        cdf  - column vector of length 'n' with the values of the cdf
%  Reference:
% Kernel density estimation via diffusion
% Z. I. Botev, J. F. Grotowski, and D. P. Kroese (2010)
% Annals of Statistics, Volume 38, Number 5, pages 2916-2957.

%
%  Example:
%           data=[randn(100,1);randn(100,1)*2+35 ;randn(100,1)+55];
%              kde(data,2^14,min(data)-5,max(data)+5);

data=data(:); %make data a column vector
if nargin<2 % if n is not supplied switch to the default
    n=2^14;
end
n=2^ceil(log2(n)); % round up n to the next power of 2;
if nargin<4 %define the default  interval [MIN,MAX]
    minimum=min(data); maximum=max(data);
    Range=maximum-minimum;
    MIN=minimum-Range/2; MAX=maximum+Range/2;
end
% set up the grid over which the density estimate is computed;
R=MAX-MIN; dx=R/(n-1); xmesh=MIN+[0:dx:R]; N=length(unique(data));
%bin the data uniformly using the grid defined above;
initial_data=histc(data,xmesh)/N;  initial_data=initial_data/sum(initial_data);
a=dct1d(initial_data); % discrete cosine transform of initial data
% now compute the optimal bandwidth^2 using the referenced method
I=[1:n-1]'.^2; a2=(a(2:end)/2).^2;
% use  fzero to solve the equation t=zeta*gamma^[5](t)
t_star=root(@(t)fixed_point(t,N,I,a2),N);
% smooth the discrete cosine transform of initial data using t_star
a_t=a.*exp(-[0:n-1]'.^2*pi^2*t_star/2);
% now apply the inverse discrete cosine transform
if (nargout>1)|(nargout==0)
    density=idct1d(a_t)/R;
end
% take the rescaling of the data into account
bandwidth=sqrt(t_star)*R;
density(density<0)=eps; % remove negatives due to round-off error
if nargout==0
    figure(1), plot(xmesh,density)
end
% for cdf estimation
if nargout>3
    f=2*pi^2*sum(I.*a2.*exp(-I*pi^2*t_star));
    t_cdf=(sqrt(pi)*f*N)^(-2/3);
    % now get values of cdf on grid points using IDCT and cumsum function
    a_cdf=a.*exp(-[0:n-1]'.^2*pi^2*t_cdf/2);
    cdf=cumsum(idct1d(a_cdf))*(dx/R);
    % take the rescaling into account if the bandwidth value is required
    bandwidth_cdf=sqrt(t_cdf)*R;
end

end
%################################################################
function  out=fixed_point(t,N,I,a2)
% this implements the function t-zeta*gamma^[l](t)
l=7;
f=2*pi^(2*l)*sum(I.^l.*a2.*exp(-I*pi^2*t));
for s=l-1:-1:2
    K0=prod([1:2:2*s-1])/sqrt(2*pi);  const=(1+(1/2)^(s+1/2))/3;
    time=(2*const*K0/N/f)^(2/(3+2*s));
    f=2*pi^(2*s)*sum(I.^s.*a2.*exp(-I*pi^2*time));
end
out=t-(2*N*sqrt(pi)*f)^(-2/5);
end



%##############################################################
function out = idct1d(data)

% computes the inverse discrete cosine transform
[nrows,ncols]=size(data);
% Compute weights
weights = nrows*exp(i*(0:nrows-1)*pi/(2*nrows)).';
% Compute x tilde using equation (5.93) in Jain
data = real(ifft(weights.*data));
% Re-order elements of each column according to equations (5.93) and
% (5.94) in Jain
out = zeros(nrows,1);
out(1:2:nrows) = data(1:nrows/2);
out(2:2:nrows) = data(nrows:-1:nrows/2+1);
%   Reference:
%      A. K. Jain, "Fundamentals of Digital Image
%      Processing", pp. 150-153.
end
%##############################################################

function data=dct1d(data)
% computes the discrete cosine transform of the column vector data
[nrows,ncols]= size(data);
% Compute weights to multiply DFT coefficients
weight = [1;2*(exp(-i*(1:nrows-1)*pi/(2*nrows))).'];
% Re-order the elements of the columns of x
data = [ data(1:2:end,:); data(end:-2:2,:) ];
% Multiply FFT by weights:
data= real(weight.* fft(data));
end

function t=root(f,N)
% try to find smallest root whenever there is more than one
N=50*(N<=50)+1050*(N>=1050)+N*((N<1050)&(N>50));
tol=10^-12+0.01*(N-50)/1000;
flag=0;
while flag==0
    try
        t=fzero(f,[0,tol]);
        flag=1;
    catch
        tol=min(tol*2,.1); % double search interval
    end
    if tol==.1 % if all else fails
        t=fminbnd(@(x)abs(f(x)),0,.1); flag=1;
    end
end
end







In [ ]:
classdef JointCalendar < Calendar

    

    properties(SetAccess = public)

        calendarList

    end

    

    methods

        function jointCalendar = JointCalendar(calendarList)
            
            jointCalendar.calendarList = calendarList;

        end
        

        function out = IsHoliday(jointCalendar,h_Date)
            
            if strcmp(class(h_Date),'char') || strcmp(class(h_Date),'string')

                h_Date = H_Date(h_Date);

            end
            
            for i=1:length(jointCalendar.calendarList)
                if jointCalendar.calendarList{i}.IsHoliday(h_Date)
                    out =true;
                    return;
                end
            end
            
            out = false;

        end

    end

    

end





In [ ]:
classdef IRModelRisky < Model
    %IRModelRisky is the SuperClass of all IRModelRisky(IRModel+ separate
    % discountFunctor
    % properties: zeroCurve , discountCurve
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        zeroCurve %forwardCurve
        discountCurve % discountCurve
    end
    
    methods
        function ir = IRModelRisky(Model)
            if nargin > 0
                ir.zeroCurve = Model.mktData('zeroCurve');
                ir.discountCurve = Model.mktData('discountCurve');
                ir.mktData =  Model.mktData;
                ir.modelParams = Model.modelParams;
            end
        end
        
        % risky  discount factor fuctor
        function df = DFRisky(ir,t)
            df = ir.discountCurve.DF(t);
        end
        %determinstic discount factor fuctor
        function df = DF(ir,t)
            df = ir.zeroCurve.DF(t);
        end
        %determinstic forward rate functor
        function fwd = Fwd(ir,t,tnr)
            fwd = (ir.DF(t)/ir.DF(t+tnr)-1)/tnr;
        end
        
        function liborDate = LiborDate(ir,valueDate,fwdStartTime,tenor)
            fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
            fwdEndDate = fwdStartDate.AddDate(tenor,'year');
            fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
            deltaT = (fwdEndTime - fwdStartTime)/365.0;
            
            df_Tn = ir.DF(fwdStartTime/365.0);
            df_TN = ir.DF(fwdEndTime/365.0);
            
            liborDate = (df_Tn/df_TN - 1.0)/deltaT;
        end
        
        %determinstic annuity functor
        function pv01 = PV01(ir,Ta,Tb,tau)
            pv01 = sum(arrayfun(@(i) tau*ir.DF(Ta+tau*i),1:round((Tb-Ta)/tau))); 
        end
        
        function pv01Date = PV01Date(ir,valueDate,toDate,tNDate,tau)
            to = toDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            
            cnt = round((tN-to)/tau);
            pv01 = 0.0;
            prevTime = to;
            deltaT = 0.0;
            for i=1:cnt
                iDate = toDate.AddDate(i*addMonthUnit,'month');
                ti = iDate.DateDiff(valueDate)/365.0;
                deltaT = ti - prevTime;
                df = ir.DF(ti);
%                 pv01 =  pv01 + tau * df;   % simple 0.25 or 0.5
                pv01 =  pv01 + deltaT * df;   % simple 0.25 or 0.5
                prevTime = ti;
            end
            pv01Date = pv01;
        end
        
        
        
        %determinstic forward swap rate functor
        function fsr = Fsr(ir,Ta,Tb,tau)
            fsr = (ir.DF(Ta)-ir.DF(Tb))/ir.PV01(Ta,Tb,tau); 
        end
        
    end
    
end



In [ ]:
classdef IRModel < Model
    %IRModel is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        zeroCurve
        
    end
    
    methods
        function ir = IRModel(Model)
            if nargin > 0
                ir.zeroCurve = Model.mktData('zeroCurve');
                ir.mktData =  Model.mktData;
                ir.modelParams = Model.modelParams;
                
            end
        end
       
        %determinstic discount factor fuctor
        function df = DF(ir,t)
            df = ir.zeroCurve.DF(t);
        end
        %determinstic forward rate functor
        function fwd = Fwd(ir,t,tnr)
            fwd = (ir.DF(t)/ir.DF(t+tnr)-1)/tnr;
        end
        %determinstic annuity functor
        function pv01 = PV01(ir,Ta,Tb,tau)
            pv01 = sum(arrayfun(@(i) tau*ir.DF(Ta+tau*i),1:round((Tb-Ta)/tau))); 
        end
        
        function pv01Date = PV01Date(ir,valueDate,toDate,tNDate,tau)
            to = toDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            
            cnt = round((tN-to)/tau);
            pv01 = 0.0;
            prevTime = to;
            deltaT = 0.0;
            for i=1:cnt
                iDate = toDate.AddDate(i*addMonthUnit,'month');
                ti = iDate.DateDiff(valueDate)/365.0;
                deltaT = ti - prevTime;
                df = ir.DF(ti);
%                 pv01 =  pv01 + tau * df;   % simple 0.25 or 0.5
                pv01 =  pv01 + deltaT * df;   % simple 0.25 or 0.5
                prevTime = ti;
            end
            pv01Date = pv01;
        end
        
        %determinstic forward swap rate functor
        function fsr = Fsr(ir,Ta,Tb,tau)
            fsr = (ir.DF(Ta)-ir.DF(Tb))/ir.PV01(Ta,Tb,tau); 
        end
        
    end
    
end



In [ ]:
classdef IRDetModel < IRModel
    %IRModel is the SuperClass of all IRModel
    % properties: zeroCurve 
    % methods : DF, FWD,PV01,FSR
    properties(SetAccess = public)
        numOfFactors
    end
    
    methods
        function ir = IRDetModel(IRModel)
            if nargin > 0
                ir.zeroCurve = IRModel.zeroCurve;
                ir.mktData =  IRModel.mktData;
                ir.modelParams = IRModel.modelParams;
                ir.numOfFactors = 0;
            end
        end
       
    end
    
end



In [ ]:
classdef IRBlackLiborTerm < H_RootObject
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        modelParams
        
        swaptionVol
        underlyingSwap
    end
    
    methods
        function irBlack = IRBlackLiborTerm(mktData,modelParams)
            if nargin > 0
                irBlack.modelParams = modelParams;
                
                if isKey(mktData,'swaptionVol')
                    swaptionVolData = mktData('swaptionVol');
                    swaptionVolrawData =  swaptionVolData.params('rawData');
                    expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1);
                    tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2))';
                    swptnVol= swaptionVolrawData(2:size(swaptionVolrawData,1),2:size(swaptionVolrawData,2));
                    irBlack.swaptionVol = @(t,T) interp2(expiry,tenor,swptnVol',t,T);
                else
                    error('swaptionVol is needed');
                end
                
                if isKey(mktData,'underlyingSwap')
                    irBlack.underlyingSwap = mktData('underlyingSwap');
                    
                else
                    error('swaptionVol is needed');
                end
            end
        end
        
%         function atmSwaption = BlackATMSwaption(irblack,tx,tnr)
%             sig = irblack.swaptionVol(tx,tnr);
%             p = @irblack.DF;
%             atmSwaption = (p(tx)-p(tx+tnr))*(2*H_ncdf(0.5*sig*sqrt(tx))-1);
%         end
%         
%         function atmSwaption = NormalATMSwaption(irblack,tx,tnr)
%             tau = irblack.tau;
%             pv01 = irblack.PV01(tx,tx+tnr,tau);
%             sig = irblack.swaptionVol(tx,tnr);
%             p = @irblack.DF;
%             atmSwaption = pv01 * (sig*sqrt(tx)*H_ndf(0.0));
%             
%         end
%         
%         function swaption = BlackSwaptionWithVol(irblack,tx,tnr,K,sig)
% %             sig = irblack.swaptionVol(tx,tnr);
%             p = @irblack.DF;
%             
% %             tau = 0.25;
%             tau = irblack.tau;
%             
%             pv01 = irblack.PV01(tx,tx+tnr,tau);
%             fsr  = irblack.Fsr(tx,tx+tnr,tau);
%             d1 = log(fsr/K)/(sig*sqrt(tx))+0.5*sig*sqrt(tx);
%             d2 = d1 - sig*sqrt(tx);
%             swaption = pv01 * (fsr*H_ncdf(d1) - K*H_ncdf(d2));
%             
%         end
%         
%         function swaption = BlackSwaptionWithVolTau(irblack,tx,tnr,K,sig,tau)
% %             sig = irblack.swaptionVol(tx,tnr);
%             p = @irblack.DF;
% %             tau = 0.25;
%             pv01 = irblack.PV01(tx,tx+tnr,tau);
%             fsr  = irblack.Fsr(tx,tx+tnr,tau);
%             d1 = log(fsr/K)/(sig*sqrt(tx))+0.5*sig*sqrt(tx);
%             d2 = d1 - sig*sqrt(tx);
%             swaption = pv01 * (fsr*H_ncdf(d1) - K*H_ncdf(d2));
%             
%         end
%         
%         function swaptionVega = BlackSwaptionVegaWithVol(irblack,tx,tnr,K,sig)
% %             sig = irblack.swaptionVol(tx,tnr);
%             p = @irblack.DF;
%             
%             %             tau = 0.25;
%             tau = irblack.tau;
%             
%             pv01 = irblack.PV01(tx,tx+tnr,tau);
%             fsr  = irblack.Fsr(tx,tx+tnr,tau);
%             d1 = log(fsr/K)/(sig*sqrt(tx))+0.5*sig*sqrt(tx);
%             d2 = d1 - sig*sqrt(tx);
% %             swaptionVega = pv01 * fsr * H_ncdf(d1)*sqrt(tx);
%             swaptionVega = pv01 * fsr * H_ndf(d1)*sqrt(tx);
%             
%         end
        
        function out = NormalSwaptionWithStrike(irBlack,discountCurve,exerciseDate,K)
            swapStartDate = irBlack.underlyingSwap.StartDate();
            if DateDiff(exerciseDate,swapStartDate) ~= 0
                error('exerciseDate should be the swapStartDate');
            end
            
            dummy = irBlack.underlyingSwap.ForwardParSwapRate(discountCurve);
            fsr = dummy.forwardParSwapRate;
            pv01 = dummy.pv01;
            expiry = DateDiff(swapStartDate,discountCurve.params('asOfDate'))/365.0;
            tx = expiry;
            tenor  =  irBlack.underlyingSwap.swapTenorYearly;
            sig = irBlack.swaptionVol(expiry,tenor);
            
            d1 = (fsr-K)/(sig*sqrt(tx));
            swaption = pv01 * ((fsr-K)*H_ncdf(d1) + sig*sqrt(tx)*H_ndf(d1));
            
            out.fsr = fsr;
            out.pv01 = pv01;
            out.tx = tx;
            out.sig = sig;
            out.swaptionValue = swaption;
            
            
        end
        
        function out = NormalSwaptionWithStrikeR(irBlack,discountCurve,exerciseDate,K)
            swapStartDate = irBlack.underlyingSwap.StartDate();
            if DateDiff(exerciseDate,swapStartDate) ~= 0
                error('exerciseDate should be the swapStartDate');
            end
            
            dummy = irBlack.underlyingSwap.ForwardParSwapRate(discountCurve);
            fsr = dummy.forwardParSwapRate;
            pv01 = dummy.pv01;
            expiry = DateDiff(swapStartDate,discountCurve.params('asOfDate'))/365.0;
            tx = expiry;
            tenor  =  irBlack.underlyingSwap.swapTenorYearly;
            sig = irBlack.swaptionVol(expiry,tenor);
            
            d1 = -1.0*(fsr-K)/(sig*sqrt(tx));
            swaption = pv01 * ((-fsr+K)*H_ncdf(d1) + sig*sqrt(tx)*H_ndf(d1));
            
            out.fsr = fsr;
            out.pv01 = pv01;
            out.tx = tx;
            out.sig = sig;
            out.swaptionValue = swaption;
            
            
        end
%             sig = irblack.swaptionVol(tx,tnr);
%             p = @irblack.DF;
%             
% %             tau = 0.25;
%             tau = irblack.tau;
            
            
            
        
        
    end
    
end



In [ ]:
classdef IRBlack < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        capVol
        swaptionVol
        tau
    end
    
    methods
        function irBlack = IRBlack(IRModel)
            if nargin > 0
                irBlack.zeroCurve = IRModel.zeroCurve;
                irBlack.mktData =  IRModel.mktData;
                irBlack.modelParams = IRModel.modelParams;
                % frequency added for KTB 20180619
                if isKey(irBlack.modelParams,'freq')
                    irBlack.tau = 1.0/irBlack.modelParams('freq');
                else
                    irBlack.tau = 0.25; % default
                end
                
                if isKey(irBlack.mktData,'capVol')
                    capVolData = irBlack.mktData('capVol');
                    capVolrawData =  capVolData.params('rawData');
                    capVolrawData =  [capVolrawData];
                    irBlack.capVol = @(t) H_interpolation(capVolrawData(:,1),capVolrawData(:,2),t,1);
                end
                
                if isKey(irBlack.mktData,'swaptionVol')
                    swaptionVolData = irBlack.mktData('swaptionVol');
                    swaptionVolrawData =  swaptionVolData.params('rawData');
                    expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1);
                    tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2))';
                    swptnVol= swaptionVolrawData(2:size(swaptionVolrawData,1),2:size(swaptionVolrawData,2));
                    irBlack.swaptionVol = @(t,T) interp2(expiry,tenor,swptnVol',t,T);
                end
                
            end
        end
        
        function caplet = BlackCaplet(irblack,tx,x,tnr,sig)
        %blackCaplet:a function that computes a blackCaplet price
        %arguments
        %tx =(annualized)time to reset
        %sig= caplet volatility
        %x  = caplet strike
        %tnr=(annualized)time to payment
        %NA =Notional Amount
            f = @irblack.Fwd;
            p = @irblack.DF;
            d1= (log(f(tx,tnr)/x)+(sig^2/2.0)*tx)/(sig*sqrt(tx));
            d2 = d1 -sig*sqrt(tx);
            caplet = tnr*p(tx+tnr)*(f(tx,tnr)*H_ncdf(d1)-x*H_ncdf(d2));
        end
        
        function floorlet = BlackFloorlet(irblack,tx,x,tnr,sig)
            f = @irblack.Fwd;
            p = @irblack.DF;
            d1= (log(f(tx,tnr)/x)+(sig^2/2.0)*tx)/(sig*sqrt(tx));
            d2 = d1 -sig*sqrt(tx);
            floorlet = tnr*p(tx+tnr)*(x*H_ncdf(-d2) - f(tx,tnr)*H_ncdf(-d1));
        end
        
        function cap = BlackCapWithVol(irblack,tx,x,tnr,sig)
            blackCaplet = @irblack.BlackCaplet;
            cap = sum(arrayfun(@(i) blackCaplet(tnr*i,x,tnr,sig),1:round(tx/tnr)-1));
        end
        
        function cap = BlackCap(irblack,tx,x,tnr)
            blackCaplet = @irblack.BlackCaplet;
            sig = irblack.capVol(tx);
            cap = sum(arrayfun(@(i) blackCaplet(tnr*i,x,tnr,sig),1:round(tx/tnr)-1));
        end
        
        function floor = BlackFloor(irblack,tx,x,tnr)
            blackFloorlet = @irblack.BlackFloorlet;
            sig = irblack.capVol(tx);
            floor = sum(arrayfun(@(i) blackFloorlet(tnr*i,x,tnr,sig),1:round(tx/tnr)-1));
        end
        
        function atmSwaption = BlackATMSwaption(irblack,tx,tnr)
            sig = irblack.swaptionVol(tx,tnr);
            p = @irblack.DF;
            atmSwaption = (p(tx)-p(tx+tnr))*(2*H_ncdf(0.5*sig*sqrt(tx))-1);
        end
        
        function atmSwaption = NormalATMSwaption(irblack,tx,tnr)
            tau = irblack.tau;
            pv01 = irblack.PV01(tx,tx+tnr,tau);
            sig = irblack.swaptionVol(tx,tnr);
            p = @irblack.DF;
            atmSwaption = pv01 * (sig*sqrt(tx)*H_ndf(0.0));
            
        end
        
        function swaption = BlackSwaptionWithVol(irblack,tx,tnr,K,sig)
%             sig = irblack.swaptionVol(tx,tnr);
            p = @irblack.DF;
            
%             tau = 0.25;
            tau = irblack.tau;
            
            pv01 = irblack.PV01(tx,tx+tnr,tau);
            fsr  = irblack.Fsr(tx,tx+tnr,tau);
            d1 = log(fsr/K)/(sig*sqrt(tx))+0.5*sig*sqrt(tx);
            d2 = d1 - sig*sqrt(tx);
            swaption = pv01 * (fsr*H_ncdf(d1) - K*H_ncdf(d2));
            
        end
        
        function swaption = BlackSwaptionWithVolTau(irblack,tx,tnr,K,sig,tau)
%             sig = irblack.swaptionVol(tx,tnr);
            p = @irblack.DF;
%             tau = 0.25;
            pv01 = irblack.PV01(tx,tx+tnr,tau);
            fsr  = irblack.Fsr(tx,tx+tnr,tau);
            d1 = log(fsr/K)/(sig*sqrt(tx))+0.5*sig*sqrt(tx);
            d2 = d1 - sig*sqrt(tx);
            swaption = pv01 * (fsr*H_ncdf(d1) - K*H_ncdf(d2));
            
        end
        
        function swaptionVega = BlackSwaptionVegaWithVol(irblack,tx,tnr,K,sig)
%             sig = irblack.swaptionVol(tx,tnr);
            p = @irblack.DF;
            
            %             tau = 0.25;
            tau = irblack.tau;
            
            pv01 = irblack.PV01(tx,tx+tnr,tau);
            fsr  = irblack.Fsr(tx,tx+tnr,tau);
            d1 = log(fsr/K)/(sig*sqrt(tx))+0.5*sig*sqrt(tx);
            d2 = d1 - sig*sqrt(tx);
%             swaptionVega = pv01 * fsr * H_ncdf(d1)*sqrt(tx);
            swaptionVega = pv01 * fsr * H_ndf(d1)*sqrt(tx);
            
        end
        
        function swaption = NormalSwaptionWithVol(irblack,tx,tnr,K,sig)
%             sig = irblack.swaptionVol(tx,tnr);
            p = @irblack.DF;
            
%             tau = 0.25;
            tau = irblack.tau;
            
            pv01 = irblack.PV01(tx,tx+tnr,tau);
            fsr  = irblack.Fsr(tx,tx+tnr,tau);
            d1 = (fsr-K)/(sig*sqrt(tx));
            swaption = pv01 * ((fsr-K)*H_ncdf(d1) + sig*sqrt(tx)*H_ndf(d1));
            
        end
        
    end
    
end



In [ ]:
function vq = interp2FlatExtrap(x,y,v,xq,yq)

    vq = interp2(x,y,v,xq,yq);

    [XMax, idxVMax] = max(x);
    [XMin, idxVMin] = min(x);

    idxMax = xq > XMax;
    idxMin = xq < XMin;
   if ~isempty(yq(idxMax))
    vq(idxMax) = interp1FlatExtrap(y,v(:,idxVMax),yq(idxMax));
   end
   if ~ isempty(yq(idxMin))
    vq(idxMin) = interp1FlatExtrap(y,v(:,idxVMin),yq(idxMin));
   end

   [YMax, idyVMax] = max(y);
   [YMin, idyVMin] = min(y);

    idyMax = yq > YMax;
    idyMin = yq < YMin;
   if ~isempty(xq(idyMax))
    vq(idyMax) = interp1FlatExtrap(x,v(idyVMax,:),xq(idyMax));
   end
   if ~ isempty(xq(idyMin))
    vq(idyMin) = interp1FlatExtrap(x,v(idyVMin,:),xq(idyMin));
   end
   
end



In [ ]:
function vq = interp1LeftFlatExtrapNew(x,v,xq,method)

    vq = interp1(x,v,xq,method);
    
    [XMax, idxVMax] = max(x);
    [XMin, idxVMin] = min(x);

    idxMax = xq > XMax;
    idxMin = xq < XMin;

%     vq(idxMax) = v(idxVMax);
    vq(idxMax) = v(idxVMax).*xq(idxMax)./XMax;
    vq(idxMin) = v(idxVMin);

end



In [ ]:
function vq = interp1FlatExtrapNew(x,v,xq,method)

    try
        vq = interp1(x,v,xq,method);
    catch exception
        throw(exception)
    end
    [XMax, idxVMax] = max(x);
    [XMin, idxVMin] = min(x);

    idxMax = xq > XMax;
    idxMin = xq < XMin;

    vq(idxMax) = v(idxVMax);
    vq(idxMin) = v(idxVMin);

end



In [ ]:
function vq = interp1FlatExtrap(x,v,xq)

    vq = interp1(x,v,xq);

    [XMax, idxVMax] = max(x);
    [XMin, idxVMin] = min(x);

    idxMax = xq > XMax;
    idxMin = xq < XMin;

    vq(idxMax) = v(idxVMax);
    vq(idxMin) = v(idxVMin);

end



In [ ]:
function vq = interp1ExtrapLinearHong(x,v,xq)

    if xq < x(1)
        delta = (v(2)-v(1))/(x(2)-x(1));
        vq = delta*(xq-x(1))+v(1);
    elseif xq > x(end)
        delta = (v(end)-v(end-1))/(x(end)-x(end-1));
        vq = delta*(xq-x(end))+v(end);
    else
        vq = interp1(x,v,xq,'linear');
    end
    
    
end



In [ ]:
function vq = interp1ExtrapLinear(x,v,xq,method)

    vq = interp1(x,v,xq,method);
    
    [XMax, idxVMax] = max(x);
    [XMin, idxVMin] = min(x);

    idxMax = xq > XMax;
    idxMin = xq < XMin;

%     vq(idxMax) = v(idxVMax);
%     vq(idxMin) = v(idxVMin);
    vq(idxMax) = v(idxVMax).*xq(idxMax)./XMax;
    vq(idxMin) = v(idxVMin).*xq(idxMin)./XMin;
end



In [ ]:
classdef IborIndex < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        tenor
        currency
        familyName
        fixingDays
        fixingCalendar
        convention
        endOfMonth
        dayCounter
        forwardCurve
        forecastTodaysFixing
        useRounding
        useSimpleTeleScoping
        useLastEndDateAsPillarDate
        
        historicalFixings
        
    end
    
    methods
        % constructor
        function iborIndex = IborIndex(params)
                if nargin > 0
                    iborIndex.tenor = params('tenor');
                    
                    if ~isKey(params,'currency')
                        iborIndex.currency = 'KRW';
                    else
                        iborIndex.currency = params('currency');
                    end
                    
                    iborIndex.familyName = strcat(iborIndex.currency,'_','Libor','_',iborIndex.tenor);
                    
                    if ~isKey(params,'fixingDays')
                        iborIndex.fixingDays = 1;
                    else
                        iborIndex.fixingDays = params('fixingDays');
                    end
                    
                    if ~isKey(params,'fixingCalendar')
                        iborIndex.fixingCalendar = Calendar();
                    else
                        iborIndex.fixingCalendar = params('fixingCalendar');
                    end
                    
                    if ~isKey(params,'convention')
                        iborIndex.convention = 'ModifiedFollowing';
                    else
                        iborIndex.convention = params('convention');
                    end
                    
                    if ~isKey(params,'dayCounter')
                        iborIndex.dayCounter = 'Act365';
                    else
                        iborIndex.dayCounter = params('dayCounter');
                    end
                    
                    if ~isKey(params,'endOfMonth')
                        iborIndex.endOfMonth = false;
                    else
                        iborIndex.endOfMonth = params('endOfMonth');
                    end
                    
                    if ~isKey(params,'forwardCurve')
                        iborIndex.forwardCurve = ZeroCurve();
                    else
                        iborIndex.forwardCurve = params('forwardCurve');
                    end
                    
                    if ~isKey(params,'forecastTodaysFixing')
                        iborIndex.forecastTodaysFixing = false;
                    else
                        iborIndex.forecastTodaysFixing = params('forecastTodaysFixing');
                    end
                    
                    if ~isKey(params,'useRounding')
                        iborIndex.useRounding = false;
                    else
                        iborIndex.useRounding = params('useRounding');
                    end
                    
                    
                    if ~isKey(params,'useSimpleTeleScoping')
                        iborIndex.useSimpleTeleScoping = false;
                    else
                        iborIndex.useSimpleTeleScoping = params('useSimpleTeleScoping');
                    end
                    
                    
                    if ~isKey(params,'useLastEndDateAsPillarDate')
                        iborIndex.useLastEndDateAsPillarDate = false;
                    else
                        iborIndex.useLastEndDateAsPillarDate = params('useLastEndDateAsPillarDate');
                    end
                    
                    if ~isKey(params,'historicalFixings')
                        iborIndex.historicalFixings = containers.Map();
                    else
                        iborIndex.historicalFixings = params('historicalFixings');
                    end
                    
                else
                    iborIndex.tenor = '3M';
                    iborIndex.currency = 'KRW';
                    iborIndex.familyName = strcat(iborIndex.currency,'_','Libor','_',iborIndex.tenor);
                    iborIndex.fixingDays = 1;
                    iborIndex.fixingCalendar = Calendar();
                    iborIndex.convention = 'ModifiedFollowing';
                    iborIndex.dayCounter = 'Act365';
                    iborIndex.endOfMonth = false;
                    iborIndex.forwardCurve = ZeroCurve();
                    iborIndex.forecastTodaysFixing = false;
                end
        end
        
        function out = Fixing(iborIndex,fixingDate)
%             if DateDiff(fixingDate,iborIndex.forwardCurve.asOfDate) > 0
%             need to be updated with pastFixing

            if DateDiff(fixingDate,iborIndex.forwardCurve.asOfDate) > 0 || ...
               (DateDiff(fixingDate,iborIndex.forwardCurve.asOfDate) == 0 && iborIndex.forecastTodaysFixing)
               
                out = ForecastFixing(iborIndex,fixingDate);
            else
                % find from fixing
                out = iborIndex.historicalFixings(StrDate(fixingDate,'str'));
                
%                 out = 0;
%                 error('unimplemented');
            end
                
        end
        
        function out = ForecastFixing(iborIndex,date)
            if strcmp(class(date),'H_Date')
               fixingValueDate = iborIndex.fixingCalendar.Advance(date,iborIndex.convention,...
                                  iborIndex.fixingDays,'day', iborIndex.endOfMonth);
              if strcmp(iborIndex.tenor(end),'D')
                  fixingEndDate = iborIndex.fixingCalendar.Advance(fixingValueDate,iborIndex.convention,...
                                  str2double(iborIndex.tenor(1:end-1)),'day', iborIndex.endOfMonth);
                              
              elseif strcmp(iborIndex.tenor(end),'M')
                   fixingEndDate = iborIndex.fixingCalendar.Advance(fixingValueDate,iborIndex.convention,...
                                  str2double(iborIndex.tenor(1:end-1)),'month', iborIndex.endOfMonth);
                              
               else
                   error('unimplemented');
              end
               
%                dcf = DayCount(fixingValueDate,fixingEndDate,iborIndex.dayCounter);
               dcf = DayCountFraction(fixingValueDate,fixingEndDate,iborIndex.dayCounter);
               
               out = ForecastFixingSimple(iborIndex,fixingValueDate,fixingEndDate,dcf);
%                t1 = DateDiff(fixingValueDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
%                t2 = DateDiff(fixingEndDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
%                disc1 = iborIndex.forwardCurve.DF(t1);
%                disc2 = iborIndex.forwardCurve.DF(t2);
%                out = (disc1/disc2 - 1.0)/dcf;
            end    
       
        end
        
        function out = ForecastFixingSimple(iborIndex,d1,d2,dcf)
               
               t1 = DateDiff(d1,iborIndex.forwardCurve.params('asOfDate'))/365.0;
               t2 = DateDiff(d2,iborIndex.forwardCurve.params('asOfDate'))/365.0;
               disc1 = iborIndex.forwardCurve.DF(t1);
               disc2 = iborIndex.forwardCurve.DF(t2);
               out = (disc1/disc2 - 1.0)/dcf;
        end
        
    end
    
end



In [ ]:
function [x,fixedPointIndicies] = HyperbolicMeshing( xMin, xMax,xCent,N,beta,...
    nodePoints,antiNodePoints)
%HYPERBOLICMESHING(xMin, xMax,xCent,N,betam,nodePoints,antiNodePoints)
%Form a grid of N points (indexed from 1 to N) in the range xMin to xMax,
%such that the density of point in higher around xCent.  beta -> 0, gives a
%high density of points around xCent, while beta >> 0 gives uniform distribution subject
%to the fixedPoints (if any) being on the grid.
%see also UniformGrid, ExpMeshing
%
%Copyright (C) 2011 - present by OpenGamma Inc. and the OpenGamma group of companies
%Please see distribution for license.

if(nargin < 7); antiNodePoints=[]; end
if(nargin < 6); nodePoints=[]; end
if(any(nodePoints<=xMin | nodePoints>=xMax))
    err= MException('OpenGamma:ExpMeshing','setPoints outside range');
    throw (err);
end
if(beta == 0.0)
    err= MException('OpenGamma:ExpMeshing','Need beta != 0.0');
    throw (err);
end

xCent = xCent/(xMax-xMin);
delta = -asinh(xCent/beta);
gamma = asinh((1.0-xCent)/beta)-delta;
nodePoints = (nodePoints-xMin)/(xMax-xMin);
antiNodePoints = (antiNodePoints-xMin)/(xMax-xMin);
if(beta>0)
    np = (asinh((nodePoints-xCent)/beta)-delta)/gamma;
    anp = (asinh((antiNodePoints-xCent)/beta)-delta)/gamma;
    [z, fixedPointIndicies] = UniformGrid(N, np,anp);
    x = xMin + (xMax-xMin)*(xCent + sinh(gamma*z+delta)*beta);
else
    np = xCent + sinh(gamma*nodePoints+delta)*beta;
    anp = xCent + sinh(gamma*antiNodePoints+delta)*beta;
    [z, fixedPointIndicies] = UniformGrid(N, np,anp);
    x = xMin + (xMax-xMin)*((asinh((z-xCent)/beta)-delta)/gamma);
end
end


In [ ]:

hssQuantLibGetCalendarTest.m

clc
clear

% valueDateInt = [20180222;20180223;20180223;20180226;20180227;20180228;20180302];

% http://45.249.2.10:50100/data/common/calendars/krex?fromDt=20180101&toDt=20480101

fromDateStr = '20180101';
endDateStr = '20480101';
calendarQueryStr1 = 'http://45.249.2.10:50100/common/calendars/krex?fromDt=';
calendarQueryStr2 = '&toDt=';
queryStr = strcat(calendarQueryStr1,fromDateStr,calendarQueryStr2,endDateStr);

%get Holidays List from GhostDB
% queryStr2 = 'http://45.249.2.10:50100/data/common/calendars/krex?fromDt=20180101&toDt=20221230';
% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJson = webread(queryStr);


krexHolidaysList= [];
sizeHolidays = length(dataJson.HoliDts);
for i=1:sizeHolidays
    hDate = H_Date(str2num(dataJson.HoliDts{i}));
    krexHolidaysList = [krexHolidaysList;hDate];
end

% addHolidayLIst to the existing calendar

py.importlib.import_module('QuantLib')
qlSouthKorea = py.QuantLib.SouthKorea;

for i=1:sizeHolidays
    hDate = krexHolidaysList(i);
    qDate = py.QuantLib.Date(int32(hDate.day),int32(hDate.month),int32(hDate.year));
    qlSouthKorea.addHoliday(qDate)
end


valueDateNum = 20180101;
TargetDateNum = 20230101;

valueDateH = H_Date(valueDateNum);
TargetDateH = H_Date(TargetDateNum);

valueDateQ = py.QuantLib.Date(int32(valueDateH.day),int32(valueDateH.month),int32(valueDateH.year));
isHolidayYN = [];
valueDateList = [];
while DateDiff(valueDateH,TargetDateH) < 0
    isHolidayFlag = qlSouthKorea.isHoliday(valueDateQ);
    isHolidayYN=[isHolidayYN;isHolidayFlag];
    valueDateList = [valueDateList;FngDate(valueDateH)];
    valueDateH = valueDateH.AddDate(1,'day');
    valueDateQ = py.QuantLib.Date(int32(valueDateH.day),int32(valueDateH.month),int32(valueDateH.year));
end
    
% idx = [1;2;3;4;5;9;13;17;21;29;41;49;61;81;101;121];
% diffRatesArr = zeros(length(idx),length(valueDateInt));
% for i=1:length(valueDateInt)
%     valueDateStr = num2str(valueDateInt(i));
%     zeroCurveFromBootStrapping = GetZeroCurveFromBootStrappingGhost(valueDateStr);
%     zeroCurveFromGhost = GetZeroCurveFromGhost(valueDateStr);
% 
%     targetTenors = zeroCurveFromBootStrapping.tenors;
% 
%     % idx = ismember(zeroCurveFromGhost.tenors(:,1),targetTenors);
%     
% 
%     zeroRatesForTargetTenors = zeroCurveFromGhost.zeroRates(idx,1);
% 
%     diffRates = zeros(length(targetTenors),1);
%     diffRates = zeroCurveFromBootStrapping.zeroRates - zeroRatesForTargetTenors;
%     
%     diffRatesArr(:,i) = diffRates(:);
% end
% 
% function out = GetZeroCurveFromBootStrappingGhost(valueDateStr)
% %     valueDateStr = '20180302';
% 
%     %% import KRW Yield Curve Data From DB
%     yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
%     queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');
% 
%     % dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
%     dataJson = webread(queryStr);
% 
%     sizeCurve = length(dataJson.Yields);
%     tenorYears = zeros(sizeCurve,1);
%     yieldsRates = zeros(sizeCurve,1);
% 
%     for i=1:sizeCurve
%         tenorYears(i) = dataJson.Yields(i).Tenor.Year;
%         yieldsRates(i) = dataJson.Yields(i).Rate;
%     end
% 
%     %% BootStrapping YieldCurve using QuantLibPython
%     py.importlib.import_module('QuantLib');
% 
%     inputDay = str2num(valueDateStr(7:8));
%     inputMonth = str2num(valueDateStr(5:6));
%     inputYear = str2num(valueDateStr(1:4));
% 
%     valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
%     qlSettings_Instance = py.QuantLib.Settings_instance;
%     qlSettings_Instance.setEvaluationDate(valueDate);
% 
%     % Construct Rate helpers
% 
%     %% Deposit Helpers
%     depoHelpers = py.list;
% 
%     % 1Days
%     qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
%     qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
%     qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
%     qlDepoFixingdays = int32(1);
%     qlWeekOnlyCalendar = py.QuantLib.WeekendsOnly; 
%     qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
%     qlDepoEndOfMonth = logical(false);
%     qlActual365Fixed = py.QuantLib.Actual365Fixed;
% 
%     depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
%                                            qlPeriod,qlDepoFixingdays, ...
%                                            qlWeekOnlyCalendar,qlModifiedFollowing,...
%                                            qlDepoEndOfMonth,qlActual365Fixed);
%     depoHelpers.append(depoHelper);                                       
% 
%     % 3Months
%     qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
%     qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
%     qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);
% 
%     depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
%                                            qlPeriod,qlDepoFixingdays, ...
%                                            qlWeekOnlyCalendar,qlModifiedFollowing,...
%                                            qlDepoEndOfMonth,qlActual365Fixed);
%     depoHelpers.append(depoHelper);                                       
% 
%     %% Swap Helpers
% 
%     % construct floating KRWCDIdx
%     tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
%     KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
%                                     py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
%                                     int32(1),py.QuantLib.KRWCurrency, ...
%                                     qlWeekOnlyCalendar, qlModifiedFollowing, ...
%                                     logical(false), qlActual365Fixed,tempCurve);
%     swapHelpers = py.list;
% 
%     %Swap Spec
%     swapFixedLegFrequency  = py.QuantLib.Quarterly;
%     swapFixedLegDayCounter = qlActual365Fixed;
%     swapFixedLengConv = qlModifiedFollowing;
%     fixedLegCalendar = qlWeekOnlyCalendar;
% 
%     for i=3:sizeCurve
%         qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
%         qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
%         if tenorYears(i) < 1
%             qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
%         else
%             qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
%         end
% 
%         swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
%                                            qlPeriod,fixedLegCalendar, ...
%                                            swapFixedLegFrequency,swapFixedLengConv,...
%                                            swapFixedLegDayCounter,KRWCDIdx);
%         swapHelpers.append(swapHelper);    
%     end
% 
%     rateHelpers = depoHelpers + swapHelpers;
% 
%     qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
%     enableExtrapolation(qlYieldCurve);
% 
%     % zeroRates
%     targetTenors = tenorYears;
%     zeroRates = zeros(length(targetTenors),1);
% 
%     qlContinuous = py.QuantLib.Continuous;
%     qlAnnual = py.QuantLib.Annual;
% 
%     for i=1:length(targetTenors)
%         zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
%         zeroRates(i) = zeroRate.rate;
%     end
%     
%     out.tenors = targetTenors;
%     out.zeroRates = zeroRates;
% end
% 
% %% Check with zeroRates Curve
% function out = GetZeroCurveFromGhost(valueDateStr)
% 
%     yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
%     queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);
% 
%     % dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
%     dataJsonZero = webread(queryStrZero);
% 
%     sizeZeoRatesFromGhost = length(dataJsonZero.Yields);
% 
%     zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
%     zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);
% 
%     for i=1:sizeZeoRatesFromGhost
%         zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
%         zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
%     end
%     out.tenors = zeroRatesFromGhost_tenorYears;
%     out.zeroRates = zeroRatesFromGhost_zeroRates;
% end



In [ ]:
hssQuantLibBootStrappingTestWithHoliday.m


clc
clear

valueDateStr = '20180302';

%% import KRW Yield Curve Data From DB
yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJson = webread(queryStr);

sizeCurve = length(dataJson.Yields);
tenorYears = zeros(sizeCurve,1);
yieldsRates = zeros(sizeCurve,1);

for i=1:sizeCurve
    tenorYears(i) = dataJson.Yields(i).Tenor.Year;
    yieldsRates(i) = dataJson.Yields(i).Rate;
end

%% BootStrapping YieldCurve using QuantLibPython
py.importlib.import_module('QuantLib')

inputDay = str2num(valueDateStr(7:8));
inputMonth = str2num(valueDateStr(5:6));
inputYear = str2num(valueDateStr(1:4));

valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
qlSettings_Instance = py.QuantLib.Settings_instance;
qlSettings_Instance.setEvaluationDate(valueDate);

% Construct Rate helpers

%% Deposit Helpers
depoHelpers = py.list;

% 1Days
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
qlDepoFixingdays = int32(1);
qlWeekOnlyCalendar = py.QuantLib.WeekendsOnly; 
qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
qlDepoEndOfMonth = logical(false);
qlActual365Fixed = py.QuantLib.Actual365Fixed;

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

% 3Months
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

%% Swap Helpers

% construct floating KRWCDIdx
tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
                                py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
                                int32(1),py.QuantLib.KRWCurrency, ...
                                qlWeekOnlyCalendar, qlModifiedFollowing, ...
                                logical(false), qlActual365Fixed,tempCurve);
swapHelpers = py.list;

%Swap Spec
swapFixedLegFrequency  = py.QuantLib.Quarterly;
swapFixedLegDayCounter = qlActual365Fixed;
swapFixedLengConv = qlModifiedFollowing;
fixedLegCalendar = qlWeekOnlyCalendar;

for i=3:sizeCurve
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    if tenorYears(i) < 1
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
    else
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
    end
    
    swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
                                       qlPeriod,fixedLegCalendar, ...
                                       swapFixedLegFrequency,swapFixedLengConv,...
                                       swapFixedLegDayCounter,KRWCDIdx);
    swapHelpers.append(swapHelper);    
end

rateHelpers = depoHelpers + swapHelpers;

qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
enableExtrapolation(qlYieldCurve);

% zeroRates
targetTenors = tenorYears;
zeroRates = zeros(length(targetTenors),1);

qlContinuous = py.QuantLib.Continuous;
qlAnnual = py.QuantLib.Annual;

for i=1:length(targetTenors)
    zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
    zeroRates(i) = zeroRate.rate;
end


%% Check with zeroRates Curve
queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJsonZero = webread(queryStrZero);

sizeZeoRatesFromGhost = length(dataJsonZero.Yields);

zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);

for i=1:sizeZeoRatesFromGhost
    zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
    zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
end



In [ ]:
hssQuantLibBootStrappingTestPerDate.m

clc
clear

valueDateInt = [20180222;20180223;20180223;20180226;20180227;20180228;20180302];

idx = [1;2;3;4;5;9;13;17;21;29;41;49;61;81;101;121];
diffRatesArr = zeros(length(idx),length(valueDateInt));
for i=1:length(valueDateInt)
    valueDateStr = num2str(valueDateInt(i));
    zeroCurveFromBootStrapping = GetZeroCurveFromBootStrappingGhost(valueDateStr);
    zeroCurveFromGhost = GetZeroCurveFromGhost(valueDateStr);

    targetTenors = zeroCurveFromBootStrapping.tenors;

    % idx = ismember(zeroCurveFromGhost.tenors(:,1),targetTenors);
    

    zeroRatesForTargetTenors = zeroCurveFromGhost.zeroRates(idx,1);

    diffRates = zeros(length(targetTenors),1);
    diffRates = zeroCurveFromBootStrapping.zeroRates - zeroRatesForTargetTenors;
    
    diffRatesArr(:,i) = diffRates(:);
end

function out = GetZeroCurveFromBootStrappingGhost(valueDateStr)
%     valueDateStr = '20180302';
    %get Holidays List from GhostDB
    queryStr2 = 'http://45.249.2.10:50100/data/common/calendars/krex?fromDt=20180101&toDt=20221230';
    % dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
    dataJson = webread(queryStr2);


    krexHolidaysList= [];
    sizeHolidays = length(dataJson.HoliDts);
    for i=1:sizeHolidays
        hDate = H_Date(str2num(dataJson.HoliDts{i}));
        krexHolidaysList = [krexHolidaysList;hDate];
    end

    % addHolidayLIst to the existing calendar

    py.importlib.import_module('QuantLib')
    qlSouthKorea = py.QuantLib.SouthKorea;

    for i=1:sizeHolidays
        hDate = krexHolidaysList(i);
        qDate = py.QuantLib.Date(int32(hDate.day),int32(hDate.month),int32(hDate.year));
        qlSouthKorea.addHoliday(qDate)
    end


    %% import KRW Yield Curve Data From DB
    yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
    queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');

    % dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
    dataJson = webread(queryStr);

    sizeCurve = length(dataJson.Yields);
    tenorYears = zeros(sizeCurve,1);
    yieldsRates = zeros(sizeCurve,1);

    for i=1:sizeCurve
        tenorYears(i) = dataJson.Yields(i).Tenor.Year;
        yieldsRates(i) = dataJson.Yields(i).Rate;
    end

    %% BootStrapping YieldCurve using QuantLibPython
%     py.importlib.import_module('QuantLib');

    inputDay = str2num(valueDateStr(7:8));
    inputMonth = str2num(valueDateStr(5:6));
    inputYear = str2num(valueDateStr(1:4));

    valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
    qlSettings_Instance = py.QuantLib.Settings_instance;
    qlSettings_Instance.setEvaluationDate(valueDate);

    % Construct Rate helpers

    %% Deposit Helpers
    depoHelpers = py.list;

    % 1Days
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
    qlDepoFixingdays = int32(1);
%     qlSouthKorea = py.QuantLib.WeekendsOnly; 
    qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
    qlDepoEndOfMonth = logical(false);
    qlActual365Fixed = py.QuantLib.Actual365Fixed;

    depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                           qlPeriod,qlDepoFixingdays, ...
                                           qlSouthKorea,qlModifiedFollowing,...
                                           qlDepoEndOfMonth,qlActual365Fixed);
    depoHelpers.append(depoHelper);                                       

    % 3Months
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);

    depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                           qlPeriod,qlDepoFixingdays, ...
                                           qlSouthKorea,qlModifiedFollowing,...
                                           qlDepoEndOfMonth,qlActual365Fixed);
    depoHelpers.append(depoHelper);                                       

    %% Swap Helpers

    % construct floating KRWCDIdx
    tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
    KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
                                    py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
                                    int32(1),py.QuantLib.KRWCurrency, ...
                                    qlSouthKorea, qlModifiedFollowing, ...
                                    logical(false), qlActual365Fixed,tempCurve);
    swapHelpers = py.list;

    %Swap Spec
    swapFixedLegFrequency  = py.QuantLib.Quarterly;
    swapFixedLegDayCounter = qlActual365Fixed;
    swapFixedLengConv = qlModifiedFollowing;
    fixedLegCalendar = qlSouthKorea;

    for i=3:sizeCurve
        qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
        qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
        if tenorYears(i) < 1
            qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
        else
            qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
        end

        swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
                                           qlPeriod,fixedLegCalendar, ...
                                           swapFixedLegFrequency,swapFixedLengConv,...
                                           swapFixedLegDayCounter,KRWCDIdx);
        swapHelpers.append(swapHelper);    
    end

    rateHelpers = depoHelpers + swapHelpers;

    qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
    enableExtrapolation(qlYieldCurve);

    % zeroRates
    targetTenors = tenorYears;
    zeroRates = zeros(length(targetTenors),1);

    qlContinuous = py.QuantLib.Continuous;
    qlAnnual = py.QuantLib.Annual;

    for i=1:length(targetTenors)
        zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
        zeroRates(i) = zeroRate.rate;
    end
    
    out.tenors = targetTenors;
    out.zeroRates = zeroRates;
end

%% Check with zeroRates Curve
function out = GetZeroCurveFromGhost(valueDateStr)

    yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
    queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);

    dataJsonZero = webread(queryStrZero);

    sizeZeoRatesFromGhost = length(dataJsonZero.Yields);

    zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
    zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);

    for i=1:sizeZeoRatesFromGhost
        zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
        zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
    end
    out.tenors = zeroRatesFromGhost_tenorYears;
    out.zeroRates = zeroRatesFromGhost_zeroRates;
end



In [ ]:
hssQuantLibBootStrappingTest_20201218_KTB.m

clc
clear

valueDateStr = '20201218';

%% import KRW Yield Curve Data From DB
% yieldCurvQueryStr = 'http://45.12.71.111:50100/data/market/yieldcurves/KRW_IRS/';
yieldCurvQueryStr = 'http://45.12.71.111:50100/data/market/yieldcurves/KRW_KTB/';

queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJson = webread(queryStr);

sizeCurve = length(dataJson.Yields);
tenorYears = zeros(sizeCurve,1);
yieldsRates = zeros(sizeCurve,1);

for i=1:sizeCurve
    tenorYears(i) = dataJson.Yields(i).Tenor.Year;
    yieldsRates(i) = dataJson.Yields(i).Rate;
end

%% BootStrapping YieldCurve using QuantLibPython
py.importlib.import_module('QuantLib')

inputDay = str2num(valueDateStr(7:8));
inputMonth = str2num(valueDateStr(5:6));
inputYear = str2num(valueDateStr(1:4));

valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
qlSettings_Instance = py.QuantLib.Settings_instance;
qlSettings_Instance.setEvaluationDate(valueDate);

% Construct Rate helpers

%% Deposit Helpers
depoHelpers = py.list;

% 1Days
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
qlDepoFixingdays = int32(1);
qlWeekOnlyCalendar = py.QuantLib.WeekendsOnly; 
qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
qlDepoEndOfMonth = logical(false);
qlActual365Fixed = py.QuantLib.Actual365Fixed;

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

% 3Months
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

%% Swap Helpers

% construct floating KRWCDIdx
tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
                                py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
                                int32(1),py.QuantLib.KRWCurrency, ...
                                qlWeekOnlyCalendar, qlModifiedFollowing, ...
                                logical(false), qlActual365Fixed,tempCurve);
swapHelpers = py.list;

%Swap Spec
swapFixedLegFrequency  = py.QuantLib.Quarterly;
swapFixedLegDayCounter = qlActual365Fixed;
swapFixedLengConv = qlModifiedFollowing;
fixedLegCalendar = qlWeekOnlyCalendar;

for i=3:sizeCurve
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    roundedYears = round(tenorYears(i));
    diffYear = abs(tenorYears(i) - roundedYears);
    
    if diffYear > 0.01
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
    else
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
    end
    
    swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
                                       qlPeriod,fixedLegCalendar, ...
                                       swapFixedLegFrequency,swapFixedLengConv,...
                                       swapFixedLegDayCounter,KRWCDIdx);
    swapHelpers.append(swapHelper);    
end

rateHelpers = depoHelpers + swapHelpers;

qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
enableExtrapolation(qlYieldCurve);

% zeroRates
targetTenors = tenorYears;
zeroRates = zeros(length(targetTenors),1);

qlContinuous = py.QuantLib.Continuous;
qlAnnual = py.QuantLib.Annual;

for i=1:length(targetTenors)
    zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
    zeroRates(i) = zeroRate.rate;
end


%% Check with zeroRates Curve
queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJsonZero = webread(queryStrZero);

sizeZeoRatesFromGhost = length(dataJsonZero.Yields);

zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);

for i=1:sizeZeoRatesFromGhost
    zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
    zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
end



In [ ]:
hssQuantLibBootStrappingTest_20201218.m

clc
clear

valueDateStr = '20201218';

%% import KRW Yield Curve Data From DB
% yieldCurvQueryStr = 'http://45.12.71.111:50100/data/market/yieldcurves/KRW_IRS/';
yieldCurvQueryStr = 'http://45.12.71.111:50100/data/market/yieldcurves/KRW_IRS/';

queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJson = webread(queryStr);

sizeCurve = length(dataJson.Yields);
tenorYears = zeros(sizeCurve,1);
yieldsRates = zeros(sizeCurve,1);

for i=1:sizeCurve
    tenorYears(i) = dataJson.Yields(i).Tenor.Year;
    yieldsRates(i) = dataJson.Yields(i).Rate;
end

%% BootStrapping YieldCurve using QuantLibPython
py.importlib.import_module('QuantLib')

inputDay = str2num(valueDateStr(7:8));
inputMonth = str2num(valueDateStr(5:6));
inputYear = str2num(valueDateStr(1:4));

valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
qlSettings_Instance = py.QuantLib.Settings_instance;
qlSettings_Instance.setEvaluationDate(valueDate);

% Construct Rate helpers

%% Deposit Helpers
depoHelpers = py.list;

% 1Days
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
qlDepoFixingdays = int32(1);
qlWeekOnlyCalendar = py.QuantLib.WeekendsOnly; 
qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
qlDepoEndOfMonth = logical(false);
qlActual365Fixed = py.QuantLib.Actual365Fixed;

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

% 3Months
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

%% Swap Helpers

% construct floating KRWCDIdx
tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
                                py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
                                int32(1),py.QuantLib.KRWCurrency, ...
                                qlWeekOnlyCalendar, qlModifiedFollowing, ...
                                logical(false), qlActual365Fixed,tempCurve);
swapHelpers = py.list;

%Swap Spec
swapFixedLegFrequency  = py.QuantLib.Quarterly;
swapFixedLegDayCounter = qlActual365Fixed;
swapFixedLengConv = qlModifiedFollowing;
fixedLegCalendar = qlWeekOnlyCalendar;

for i=3:sizeCurve
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    roundedYears = round(tenorYears(i));
    diffYear = abs(tenorYears(i) - roundedYears);
    
    if diffYear > 0.01
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
    else
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
    end
    
    swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
                                       qlPeriod,fixedLegCalendar, ...
                                       swapFixedLegFrequency,swapFixedLengConv,...
                                       swapFixedLegDayCounter,KRWCDIdx);
    swapHelpers.append(swapHelper);    
end

rateHelpers = depoHelpers + swapHelpers;

qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
enableExtrapolation(qlYieldCurve);

% zeroRates
targetTenors = tenorYears;
zeroRates = zeros(length(targetTenors),1);

qlContinuous = py.QuantLib.Continuous;
qlAnnual = py.QuantLib.Annual;

for i=1:length(targetTenors)
    zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
    zeroRates(i) = zeroRate.rate;
end


%% Check with zeroRates Curve
queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJsonZero = webread(queryStrZero);

sizeZeoRatesFromGhost = length(dataJsonZero.Yields);

zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);

for i=1:sizeZeoRatesFromGhost
    zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
    zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
end



In [ ]:
hssQuantLibBootStrappingTest.m

clc
clear

valueDateStr = '20180910';

%% import KRW Yield Curve Data From DB
yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJson = webread(queryStr);

sizeCurve = length(dataJson.Yields);
tenorYears = zeros(sizeCurve,1);
yieldsRates = zeros(sizeCurve,1);

for i=1:sizeCurve
    tenorYears(i) = dataJson.Yields(i).Tenor.Year;
    yieldsRates(i) = dataJson.Yields(i).Rate;
end

%% BootStrapping YieldCurve using QuantLibPython
py.importlib.import_module('QuantLib')

inputDay = str2num(valueDateStr(7:8));
inputMonth = str2num(valueDateStr(5:6));
inputYear = str2num(valueDateStr(1:4));

valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
qlSettings_Instance = py.QuantLib.Settings_instance;
qlSettings_Instance.setEvaluationDate(valueDate);

% Construct Rate helpers

%% Deposit Helpers
depoHelpers = py.list;

% 1Days
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
qlDepoFixingdays = int32(1);
qlWeekOnlyCalendar = py.QuantLib.WeekendsOnly; 
qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
qlDepoEndOfMonth = logical(false);
qlActual365Fixed = py.QuantLib.Actual365Fixed;

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

% 3Months
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

%% Swap Helpers

% construct floating KRWCDIdx
tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
                                py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
                                int32(1),py.QuantLib.KRWCurrency, ...
                                qlWeekOnlyCalendar, qlModifiedFollowing, ...
                                logical(false), qlActual365Fixed,tempCurve);
swapHelpers = py.list;

%Swap Spec
swapFixedLegFrequency  = py.QuantLib.Quarterly;
swapFixedLegDayCounter = qlActual365Fixed;
swapFixedLengConv = qlModifiedFollowing;
fixedLegCalendar = qlWeekOnlyCalendar;

for i=3:sizeCurve
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    if tenorYears(i) < 1
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
    else
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
    end
    
    swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
                                       qlPeriod,fixedLegCalendar, ...
                                       swapFixedLegFrequency,swapFixedLengConv,...
                                       swapFixedLegDayCounter,KRWCDIdx);
    swapHelpers.append(swapHelper);    
end

rateHelpers = depoHelpers + swapHelpers;

qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
enableExtrapolation(qlYieldCurve);

% zeroRates
targetTenors = tenorYears;
zeroRates = zeros(length(targetTenors),1);

qlContinuous = py.QuantLib.Continuous;
qlAnnual = py.QuantLib.Annual;

for i=1:length(targetTenors)
    zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
    zeroRates(i) = zeroRate.rate;
end


%% Check with zeroRates Curve
queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJsonZero = webread(queryStrZero);

sizeZeoRatesFromGhost = length(dataJsonZero.Yields);

zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);

for i=1:sizeZeoRatesFromGhost
    zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
    zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
end



In [ ]:
hssJsonSchemaTest.m

clc
clear

% code_qry = 'http://45.249.2.10:50500/Schema/Json/Common.json';
% codejson = webread(code_qry);

% schema_qry = 'http://45.249.2.10:50500/Schema/Json/YieldCurve.json';
% schema_qry = 'http://45.249.2.10:50500/Schema/Json/AddInfo.json';
schema_qry = 'http://45.249.2.10:50500/Schema/Json/Code.json';

dataJson = webread(schema_qry);

volSurfaceMaster_qry = 'http://45.249.2.10:50100/data/market/vols/0321L2_COMD/20171228';
volSurfaceMasterJson = webread(volSurfaceMaster_qry);

yieldCurveMaster_qry = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180102/?type=market';
dataJsonMaster = webread(yieldCurveMaster_qry);

yieldCurveMaster_qry1 = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180102';
dataJsonMaster1 = webread(yieldCurveMaster_qry1);

yieldCurveMaster_qry2 = 'http://45.249.2.10:50500/Schema/Json/YieldCurve.json#Yield';
dataJsonMaster2 = webread(yieldCurveMaster_qry2);

valueDateStr = '20180302';

%% import KRW Yield Curve Data From DB
yieldCurvQueryStr = 'http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/';
queryStr = strcat(yieldCurvQueryStr,valueDateStr,'?type=market');

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJson = webread(queryStr);

sizeCurve = length(dataJson.Yields);
tenorYears = zeros(sizeCurve,1);
yieldsRates = zeros(sizeCurve,1);

for i=1:sizeCurve
    tenorYears(i) = dataJson.Yields(i).Tenor.Year;
    yieldsRates(i) = dataJson.Yields(i).Rate;
end

%% BootStrapping YieldCurve using QuantLibPython
py.importlib.import_module('QuantLib')

inputDay = str2num(valueDateStr(7:8));
inputMonth = str2num(valueDateStr(5:6));
inputYear = str2num(valueDateStr(1:4));

valueDate = py.QuantLib.Date(int32(inputDay),int32(inputMonth),int32(inputYear));
qlSettings_Instance = py.QuantLib.Settings_instance;
qlSettings_Instance.setEvaluationDate(valueDate);

% Construct Rate helpers

%% Deposit Helpers
depoHelpers = py.list;

% 1Days
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(1)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(1),py.QuantLib.Days);
qlDepoFixingdays = int32(1);
qlWeekOnlyCalendar = py.QuantLib.WeekendsOnly; 
qlModifiedFollowing = py.QuantLib.ModifiedFollowing;
qlDepoEndOfMonth = logical(false);
qlActual365Fixed = py.QuantLib.Actual365Fixed;

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

% 3Months
qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(2)));
qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
qlPeriod      = py.QuantLib.Period(int32(3),py.QuantLib.Months);

depoHelper = py.QuantLib.DepositRateHelper(qlQuoteHandle, ...
                                       qlPeriod,qlDepoFixingdays, ...
                                       qlWeekOnlyCalendar,qlModifiedFollowing,...
                                       qlDepoEndOfMonth,qlActual365Fixed);
depoHelpers.append(depoHelper);                                       

%% Swap Helpers

% construct floating KRWCDIdx
tempCurve = py.QuantLib.YieldTermStructureHandle(py.QuantLib.FlatForward(valueDate,double(0.02),qlActual365Fixed));
KRWCDIdx = py.QuantLib.IborIndex("temp_krwirs",...
                                py.QuantLib.Period(int32(3),py.QuantLib.Months), ...
                                int32(1),py.QuantLib.KRWCurrency, ...
                                qlWeekOnlyCalendar, qlModifiedFollowing, ...
                                logical(false), qlActual365Fixed,tempCurve);
swapHelpers = py.list;

%Swap Spec
swapFixedLegFrequency  = py.QuantLib.Quarterly;
swapFixedLegDayCounter = qlActual365Fixed;
swapFixedLengConv = qlModifiedFollowing;
fixedLegCalendar = qlWeekOnlyCalendar;

for i=3:sizeCurve
    qlSimpleQuote = py.QuantLib.SimpleQuote(double(yieldsRates(i)));
    qlQuoteHandle = py.QuantLib.QuoteHandle(qlSimpleQuote);
    if tenorYears(i) < 1
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)*12),py.QuantLib.Months);
    else
        qlPeriod      = py.QuantLib.Period(int32(tenorYears(i)),py.QuantLib.Years);
    end
    
    swapHelper = py.QuantLib.SwapRateHelper(qlQuoteHandle, ...
                                       qlPeriod,fixedLegCalendar, ...
                                       swapFixedLegFrequency,swapFixedLengConv,...
                                       swapFixedLegDayCounter,KRWCDIdx);
    swapHelpers.append(swapHelper);    
end

rateHelpers = depoHelpers + swapHelpers;

qlYieldCurve = py.QuantLib.PiecewiseLinearZero(valueDate,rateHelpers,qlActual365Fixed);
enableExtrapolation(qlYieldCurve);

% zeroRates
targetTenors = tenorYears;
zeroRates = zeros(length(targetTenors),1);

qlContinuous = py.QuantLib.Continuous;
qlAnnual = py.QuantLib.Annual;

for i=1:length(targetTenors)
    zeroRate = qlYieldCurve.zeroRate(double(targetTenors(i)),qlContinuous,qlAnnual);
    zeroRates(i) = zeroRate.rate;
end


%% Check with zeroRates Curve
queryStrZero = strcat(yieldCurvQueryStr,valueDateStr);

% dataJson = webread('http://45.249.2.10:50100/data/market/yieldcurves/KRW_IRS/20180302?type=market');
dataJsonZero = webread(queryStrZero);

sizeZeoRatesFromGhost = length(dataJsonZero.Yields);

zeroRatesFromGhost_tenorYears = zeros(sizeZeoRatesFromGhost,1);
zeroRatesFromGhost_zeroRates = zeros(sizeZeoRatesFromGhost,1);

for i=1:sizeZeoRatesFromGhost
    zeroRatesFromGhost_tenorYears(i) = dataJsonZero.Yields(i).Tenor.Year;
    zeroRatesFromGhost_zeroRates(i) = dataJsonZero.Yields(i).ZeroRate;
end



In [ ]:
hss_Octagon_USDCMSKRWCMSSpreadDualRA_test_20181022.m

clc
clear

%% open to user: to price a deal user need to supply the following information
%% product id,
%% context,
%% valueDate

%USDCMSKRWCMSSpreadRA instrument
product = containers.Map({'insId','context',},{'F181029-00006','MtM'});
valueDate = H_Date(20181022);

% global dictionary(map) init
globalDict = containers.Map();

%% assetNameModelNameMap : for each assetName which modelName is used
%% assetName : usedModelName
assetNameModelNameMap =  containers.Map();
assetNameModelNameMap('KRW') = 'LGM2F';
assetNameModelNameMap('KRWUSD') = 'FXQuantoDummy';
assetNameModelNameMap('USD') = 'CK1F';

globalDict('assetNameModelNameMap') = assetNameModelNameMap;
%% static data   

%% calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;

%% MarketData YLD
%yldCurveSetting
yiledCurveSettings = containers.Map();

%KRW_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%% Models CK1F, LGM2F
%modelSettings
modelSettings = containers.Map();

%IRCK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;
ck1FModelParams('assetType') = 'IR';

modelSettings('CK1F') = ck1FModelParams;

%IRLGM2F
lgm2FModelParams = containers.Map();
lgm2FModelParams('dH_initial') = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421];

lgm2FModelParams('Alpha_initial') = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438];

lgm2FModelParams('smoothdH') = 1.0;
lgm2FModelParams('smoothAlpha') = 1.0;
lgm2FModelParams('smoothTermCorr') = 0.1;
lgm2FModelParams('targetTermCorr') = 0.92;
lgm2FModelParams('freq') = 4;
lgm2FModelParams('maxIter') = 200;
lgm2FModelParams('modelName') = 'LGM2F';
lgm2FModelParams('assetType') = 'IR';

modelSettings('LGM2F') = lgm2FModelParams;

%FXBS
fxBSModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.1];
vol_r.tenor = [1.0];
fxBSModelParams('vol_r') = vol_r;

fxBSModelParams('modelName') = 'FXBS';

fxBSModelParams('assetType') = 'FX';

modelSettings('FXBS') = fxBSModelParams;

%% FXQuantoDummy
fxQuantoDummyModelParams = containers.Map();

fxQuantoDummyModelParams('modelName') = 'FXQuantoDummy';

fxQuantoDummyModelParams('assetType') = 'FX';
fxQuantoDummyModelParams('stochasticYN') = 'NO';

modelSettings('FXQuantoDummy') = fxQuantoDummyModelParams;

%% multiAssetModel Settings
multiAssetModelParams = containers.Map();
multiAssetModelParams('refModelName') = 'KRW';
multiAssetModelParams('useQuasiRandomYN') = 'false';
multiAssetModelParams('useBrigoCorrYN') = 'simple_adjust';

modelSettings('multiAssetModel') = multiAssetModelParams;

globalDict('modelSettings') = modelSettings;

%% Correlation data to be upgraded

% correlation mktData is defined by key & value where key is a pair
% (KRW,USD), (KRW,KRWUSD), (KRWUSD,USD)
correlMap = containers.Map();
correlMap('KRW_USD') = 0.3;
correlMap('KRW_KRWUSD') = -0.17;
correlMap('KRWUSD_USD') = 0.1;

globalDict('correlMap') = correlMap;

%% numMethod Settings
numMethodSettings = containers.Map();
%% AMC Settings
amcNumMethodParams = containers.Map();
amcNumMethodParams('numMethodType') = 'AMC';
amcNumMethodParams('mcOneTimeStep') = 7;
amcNumMethodParams('pathSize') = 40000;

numMethodSettings('amc') = amcNumMethodParams;

globalDict('numMethodSettings') = numMethodSettings;
%% Static data setting end


%% Instrument API > Security > Underlyings > Mapped MarketData > Model Building
% get Instrument info
%% instrument : development server API  45.249.2.10:50100

instrumentQueryStr = 'http://45.249.2.36:50100/data/instruments/';
queryStr = strcat(instrumentQueryStr,product('insId'));
instrumentJson = webread(queryStr);

%% instrument layer info
instrumentPayCcy = instrumentJson.CcyCd.Cd;

globalDict('payCcy') = instrumentPayCcy;

components = instrumentJson.Components;

valGroup = strcat(components.ProdTpNm,'_',components.TmpltNm);

globalDict('valGroup') = valGroup;

%
if strcmp(valGroup,'InterestRateDerivatives_DualSpreadRA')
   %% DualSpreadRA Info 
    %security Info : startDate,endDate,nominal,payCcy from instrument.Components
    % construct rangeAccrualParams
    
    rangeAccrualParams.payCcy = globalDict('payCcy');
    
    % reference ModelName
    globalDict('refModelName') = rangeAccrualParams.payCcy;
    
    rangeAccrualParams.startDate = H_Date(components.EffDt);
    rangeAccrualParams.endDate = H_Date(components.EndDt);
    
    rangeAccrualInfo = containers.Map();

    rangeAccrualInfo('nominal') = components.UnitAmt;
    
    %% payoff info start from instrument.Components.Payoffs
    payoffs = components.Payoffs;
    
    % underlyings ordering
    % Underlying1Id  : index name for range accrual observation
    % Underlying2Id  : index name for spread ragne accrual observation
    % (Long)
    % Underlying3Id  : index name for spread ragne accrual observation
    % (Short)
    
    rangeAccrualInfo('RAIndexName') = payoffs.Underlying1Id;
    rangeAccrualInfo('SpreadRAIndexNameLong') = payoffs.Underlying2Id;
    rangeAccrualInfo('SpreadRAIndexNameShort') = payoffs.Underlying3Id;
    
    % Coupon
    rangeAccrualInfo('coupon') = payoffs.FixedIr;  % 
    
    % Single RA
    rangeAccrualInfo('upper') = 1.0;
    if isfield(payoffs,'Cap1')
        rangeAccrualInfo('upper') = payoffs.Cap1;
    end

    rangeAccrualInfo('lower') = -1.0;
    if isfield(payoffs,'Floor1')
        rangeAccrualInfo('lower') = payoffs.Floor1;
    end

    % Spread RA
    
    rangeAccrualInfo('lowerS') = -1.0;
    if isfield(payoffs,'Floor2')
        rangeAccrualInfo('lowerS') = payoffs.Floor2;
    end

    
    rangeAccrualInfo('upperS') = 1.0;
    if isfield(payoffs,'Cap2')
        rangeAccrualInfo('upperS') = payoffs.Cap2;
    end
    
    %% payoff info end from instrument.Components.Payoffs

    %% Schedule info start from instrument.Components.Items
    %% structuredLeg, callableLeg Schedule
    
    items = instrumentJson.Components.Items;
    % find structured leg
    structuredLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemSubTpNm,'structured')
            structuredLeg = items{idx};
            break;
        end
    end
    
    rangeAccrualInfo('couponFreq') = structuredLeg.ExerFreqCd.Cd;
    
    % generated Schedule
    structuredSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
                                    rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Forward',false);
    % manual Schedule
    structuredScheduleRaw = structuredLeg.Cashflows;
    %reset start end pay FngDate format array
    couponScheduleInt = zeros(length(structuredLeg.Cashflows),4);
    for idx =1 :length(structuredScheduleRaw)
        couponScheduleInt(idx,1) = FngDate(H_Date(structuredScheduleRaw(idx).ResetDt));
        couponScheduleInt(idx,2) = FngDate(H_Date(structuredScheduleRaw(idx).BegDt));
        couponScheduleInt(idx,3) = FngDate(H_Date(structuredScheduleRaw(idx).EndDt));
        couponScheduleInt(idx,4) = FngDate(H_Date(structuredScheduleRaw(idx).PayDt));
        
    end
    
    rangeAccrualInfo('couponScheduleInt') = couponScheduleInt;
    
    % find callable exerciseEvents
    callalbleLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemTpNm,'callable')
            callalbleLeg = items{idx};
            break;
        end
    end
    
    % generated Schedule
    nonCallTermYear = callalbleLeg.NonCallTermYear;
    %% CallNotice OffSet reflected Call Schedule Generation
    %% to be added
    
    callStartDate = globalDict('jointCalendar').AdvanceTenor(rangeAccrualParams.startDate,'ModifiedFollowing',strcat(num2str(nonCallTermYear),'Y'),false);
                    
    callFreq = callalbleLeg.ExerFreqCd.Cd;
    
    callableSchedule0 = Schedule(callStartDate,rangeAccrualParams.endDate,...
                                    callFreq ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Forward',false);

    % should be fixed to use callable schedule directly !
    callYN = ones(size(couponScheduleInt,1),1);
    %only yearly schedule
    for i=1:length(callYN)
        if i <= nonCallTermYear
            callYN(i) = 0;
        end
    end
    
    % not used in the pricer ended with simpleCall ! dummyVariable
    rangeAccrualInfo('callYN') = callYN;
    
    % manual Schedule
    callalbleScheduleRaw = callalbleLeg.ExerciseEvents;
    %reset start end pay FngDate format array
    callScheduleInt  = zeros(length(callalbleLeg.ExerciseEvents),2);
    for idx = 1:length(callalbleScheduleRaw)
        callScheduleInt(idx,1) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,2) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
    end
    
callScheduleInt = [20181010	20181010	20191010	20191010
20191007	20191010	20201012	20191010
20201007	20201012	20211011	20201012
20211006	20211011	20221010	20211011
20221005	20221010	20231010	20221010
20231005	20231010	20241010	20231010
20241007	20241010	20251010	20241010
20251006	20251010	20261012	20251010
20261007	20261012	20271011	20261012
20271006	20271011	20281010	20271011
20281005	20281010	20291010	20281010
20291005	20291010	20301010	20291010
20301007	20301010	20311010	20301010
20311006	20311010	20321011	20311010
20321006	20321011	20331010	20321011
];
    
    rangeAccrualInfo('callScheduleInt') = callScheduleInt;
    
   %% Underlying Info from instrumentJson.Components.Underlyings
    % getUnderlyings
    underlyings = components.Underlyings;
    underlyingsInfo = struct;
    for i=1:length(underlyings)
        underlyingsInfo(i).name = underlyings(i).UnderlyingId;
        underlyingsInfo(i).type = underlyings(i).Instrument.InsTpCd.Cd;
        underlyingsInfo(i).ccy  = underlyings(i).Instrument.CcyCd.Cd;
        if strcmp(underlyingsInfo(i).name(4:6),'IRS')
            underlyingsInfo(i).assetType = 'IR';
            underlyingsInfo(i).rateType = 'CMS';
            if strcmp(underlyingsInfo(i).name(end),'Y') 
                underlyingsInfo(i).rateTenor = uint16(str2num(underlyingsInfo(i).name(end-2:end-1)));
            else
                error('CMS Rate tenor should be given in years!')
            end
        elseif strcmp(underlyingsInfo(i).name(4:6),'LBR') || strcmp(underlyingsInfo(i).name,'CD91D') || strcmp(underlyingsInfo(i).name,'KTB3M')
            underlyingsInfo(i).assetType = 'IR';
            underlyingsInfo(i).rateType = 'Libor';
            underlyingsInfo(i).rateTenor = 0.25;
        else
            error('unknown index type!')
        end
            
    end
    
    % underlyings info in the global dictionary
    globalDict('underlyingsInfo') = underlyingsInfo;
    
      %% need to be generalized to include general underlyings in the payoff description using functor
    
    %find RAIndex in the underlyings
    for idx=1:length(underlyingsInfo)
        if strcmp(underlyingsInfo(idx).name,rangeAccrualInfo('RAIndexName'))
            rangeAccrualInfo('tenor') = underlyingsInfo(idx).rateTenor;
            break;
        end
    end
    
     %find SpreadRAIndexLong in the underlyings
    for idx=1:length(underlyingsInfo)
        if strcmp(underlyingsInfo(idx).name,rangeAccrualInfo('SpreadRAIndexNameLong'))
            rangeAccrualInfo('tenorL') = underlyingsInfo(idx).rateTenor;
            break;
        end
    end
    
     %find SpreadRAIndexShort in the underlyings
    for idx=1:length(underlyingsInfo)
        if strcmp(underlyingsInfo(idx).name,rangeAccrualInfo('SpreadRAIndexNameShort'))
            rangeAccrualInfo('tenorS') = underlyingsInfo(idx).rateTenor;
            break;
        end
    end
    
    %     %% fixings to be updated later
    %% fixing needs underlings info 
%     rangeAccrualInfo('RAIndexName') = payoffs.Underlying1Id;
%     rangeAccrualInfo('SpreadRAIndexNameLong') = payoffs.Underlying2Id;
%     rangeAccrualInfo('SpreadRAIndexNameShort') = payoffs.Underlying3Id;
    
    fixingQueryStr1 = 'http://45.249.2.36:50100/data/market/prices/';
    fixingFromDate = FngDate(rangeAccrualParams.startDate);
    fixingToDate = FngDate(rangeAccrualParams.endDate);
    fixingInfo = containers.Map();
    
    for idx=1:length(underlyingsInfo)
        fixingQueryStr = strcat(fixingQueryStr1,underlyingsInfo(idx).name,'/',int2str(fixingFromDate),'-',int2str(fixingToDate));
        fixingJson = webread(fixingQueryStr);
        
        tempFixingInfo = containers.Map();
        
        for idx2=1:length(fixingJson)
            tempFixingInfo(fixingJson(idx2).BaseDt) = fixingJson.SettlePrice;
        end
        fixingInfo(underlyingsInfo(idx).name) = tempFixingInfo;
    end
    
    fixingInfo('RAIndexName') = rangeAccrualInfo('RAIndexName');
    fixingInfo('SpreadRAIndexNameLong') = rangeAccrualInfo('SpreadRAIndexNameLong');
    fixingInfo('SpreadRAIndexNameShort') = rangeAccrualInfo('SpreadRAIndexNameShort');
    
    %%  fixingInfo('USDIRS10Y') > USDIRS10Y fixings
    %%  aa = fixingInfo('USDIRS10Y')
    %%  aa('20181022') > 0.0326
    
    %% fill rangeDays using FixingInfo
    
    rangeDays = zeros(size(couponScheduleInt,1),1);
    %% full range
    for i=1:length(rangeDays)
        fixingDateEnd = valueDate;
        if DateDiff(valueDate,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDate,H_Date(couponScheduleInt(i,4))) < 0
            
            if DateDiff(valueDate,H_Date(couponScheduleInt(i,3))) >= 0
                fixingDateEnd = H_Date(couponScheduleInt(i,3));
                fixingDateEnd = fixingDateEnd.AddDate(-1,'day');
            else
                fixingDateEnd = valueDate;
            end
           %%  fixing function
            rangeDays(i) = GetDualSpreadRAFixingRangeDays(fixingInfo,rangeAccrualInfo('lower'),rangeAccrualInfo('upper'),...
                            rangeAccrualInfo('lowerS'),rangeAccrualInfo('upperS'),H_Date(couponScheduleInt(i,2)),fixingDateEnd);
        end
    end
    
%     for i=1:length(rangeDays)
%         if DateDiff(valueDate,H_Date(couponScheduleInt(i,2))) >= 0 ...
%             && DateDiff(valueDate,H_Date(couponScheduleInt(i,3))) < 0
%             rangeDays(i) = DateDiff(valueDate,H_Date(couponScheduleInt(i,2))) + 1;
%         elseif DateDiff(valueDate,H_Date(couponScheduleInt(i,3))) >= 0
%             rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
%         end
%     end
    
    rangeAccrualInfo('rangeDays') = rangeDays;
    rangeAccrualInfo('manualScheduleYN') = 1;
    rangeAccrualInfo('callStrike') = 1.0;
    globalDict('securityInfo') = rangeAccrualInfo;
    

   
  
    
    rangeAccrualParams.params = rangeAccrualInfo;              
    rangeAccrual = CMSCMSSpreadDualRangeAccrual(rangeAccrualParams);

    globalDict('security') = rangeAccrual;

    %% fixings to be added here
    
    
    %% From underlyings info , we fill rangeAccrualInfo: tenor, tenorL, tenorS
    
    
    %% Build Models based on underlyings 
    %% Get the mapped market data using context
    %% Build market data
    %% Build model
    %% Link built model to asset
    %% containers to have assetName : usedModel Object
    modelNameMap =  containers.Map();
    
    for i=1:length(underlyings)
        
        if strcmp(underlyingsInfo(i).type,'RATE_INDEX')
                % for RATE INDEX, underlyings currency is the asset name!
                % asset is not in the assetModel Yet then we add new asset!
                % otherwise we skip
                
                if ~isKey(modelNameMap,underlyingsInfo(i).ccy)
                    %get Underlying Mapped MarketData
                    asset = struct;
                    asset.name = underlyingsInfo(i).ccy;
                    asset.ccy = asset.name;
                    asset.type = 'IR';

                    %% marketData init

                    asset.allMarketData = containers.Map();

                    % if asset is discountingModel(referenceModel) then add mappedDiscountYLDCurve
                    if strcmp(asset.ccy,globalDict('payCcy'))
                        %% discount YLD
                        %% marketData : Real server API  45.249.2.36:50100
                        discountYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                            asset.ccy,'&mapTrgtTpCd=DISCOUNT');
                        mappedYldJson = webread(discountYldQueryStr);
                        yldCurveName = mappedYldJson.Items.MapTrgtNm;
                        bootStrapFlag = true;
                       %bootStrapFlag = false;
                        % if bootStrapFlag is true then we strip zeroCurve
                        ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);
                        %YLD Curve
                        asset.allMarketData('discountYLD') = ycData;
                        %Zero Curve
                        asset.allMarketData('discountCurve') = ycData.zcCurve.zeroCurve;

%                         globalDict('refModelName') = asset.ccy;
                    % if asset's currency is different then we need FX
                    % Model
                    else
%                         asset.ccy,globalDict('payCcy')
                        fxAsset = struct;
                        fxAsset.ccy = asset.ccy;
                        
                        fxAsset.name = strcat(globalDict('payCcy'),fxAsset.ccy);
                        
                        fxAsset.type = 'FX';
                        fxAsset.allMarketData = containers.Map();
                        
                        % FX MarketData
                     %% FX ATM TermStructure Vol
                        fxVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                        'FX',fxAsset.name,'&mapTrgtTpCd=VOL');
                        fxVolJson = webread(fxVolQueryStr);

                        fxVolName = fxVolJson.Items.MapTrgtNm;
                        
                        fxVolData = GetFXVolTermStructureFromOctagon(valueDate,fxVolName);
                        rawDataFXVol = fxVolData.rawData;
                        mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataFXVol,valueDate,fxAsset.ccy,fxVolName});
                    
                        impliedTermVolMktData = MktData(mktData);
                        
                        fxAsset.allMarketData('impliedTermVol') = impliedTermVolMktData;
                        
                     %% fxQuantoDummy Model Init
                        assetNameModelNameMap = globalDict('assetNameModelNameMap');
                        fxAsset.modelName = assetNameModelNameMap(fxAsset.name);

                        if strcmp(fxAsset.modelName,'FXQuantoDummy')
                            % construct FXQuantoDummy model
                            modelSettings = globalDict('modelSettings');
                            fxQuantoDummyModelParams = modelSettings('FXQuantoDummy');
                            fxQuantoDummy = Model(fxAsset.allMarketData,fxQuantoDummyModelParams);
                            modelNameMap(fxAsset.name) = fxQuantoDummy;
                        end
                    end

                    %% fwd YLD
                    fwdYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                        underlyingsInfo(1).name,'&mapTrgtTpCd=RISKFREE');
                    mappedYldJson = webread(fwdYldQueryStr);
                    yldCurveName = mappedYldJson.Items.MapTrgtNm;
                    bootStrapFlag = true;
                   %bootStrapFlag = false;
                    ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);

                    asset.allMarketData('fwdYLD') = ycData;
                    asset.allMarketData('zeroCurve') = ycData.zcCurve.zeroCurve;
                    
                    %% Swaption Vol
                    
                    %get mapped swaption volatility
                    swaptionVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                        underlyingsInfo(1).ccy,'&mapTrgtTpCd=SWAPTVOL');
                    mappedVolJson = webread(swaptionVolQueryStr);

                    swaptionVolName = mappedVolJson.Items.MapTrgtNm;

                    swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,swaptionVolName);
                    rawDataSwaption = swaptionVolData.rawData;

                    asset.allMarketData('swaptionVolRaw') = rawDataSwaption;

                    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                                {rawDataSwaption,valueDate, underlyingsInfo(1).ccy,strcat(underlyingsInfo(1).ccy,'SwaptionVol')});
                    swaptionVolMktData = MktData(mktData);
                    asset.allMarketData('swaptionVol') = swaptionVolMktData;

                     %% marketData end
                     
                    %% model Init
                    assetNameModelNameMap = globalDict('assetNameModelNameMap');
                    asset.modelName = assetNameModelNameMap(asset.name);

                    if strcmp(asset.modelName,'CK1F')
                        % construct ck1F model
                        modelSettings = globalDict('modelSettings');
                        ck1FModelParams = modelSettings('CK1F');

                        % added modelParams

                        ck1FModelParams('instrumentStartDate') = rangeAccrualParams.startDate;
                        ck1FModelParams('instrumentEndDate') = rangeAccrualParams.endDate;
                        ck1FModelParams('calibrationTargetTenor') =  1:1:14;

                        ck1F = CK1F(IRModel(Model(asset.allMarketData,ck1FModelParams)));

                        % calibration
                        % construct black model 
                        blackModelParams= containers.Map({'modelName'},{'Black'});
                        black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 


                        calibOut = ck1F.CalibrateToATMDiagSwaptionForward(black,'global');
                        
                        % if reference Model then construnct
                        % RiskyModel(MultiCurve Discounting)
                        if  strcmp(asset.ccy,globalDict('refModelName')) 
                            vol_r_Calib = ck1F.vol_r;

                            modelParamsCalib= ck1FModelParams;
                            modelParamsCalib('vol_r') = vol_r_Calib;

                            ck1FRisky = CK1FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));

                            modelNameMap(asset.name) = ck1FRisky;
                        else
                            modelNameMap(asset.name) = ck1F;
                        end
                        % map is a handle class so you don't need the foillowing step
        %                 globalDict('assetModelMap') = assetModelMap;
                    
                    elseif strcmp(asset.modelName,'LGM2F')
                        % calibration
                        % construct black model 
                        blackModelParams= containers.Map({'modelName'},{'Black'});
                        black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 
                        
                        % construct lgm2F model
                        modelSettings = globalDict('modelSettings');
                        lgm2FModelParams = modelSettings('LGM2F');
                        
                        % added modelParams
                        rawDataSwaption = asset.allMarketData('swaptionVolRaw');
                        expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
                        tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
                        
                        tempdHTenor = zeros(size(expiry,1),size(tenor,1));
                        for idx1=1:size(expiry,1)
                            for idx2=1:size(tenor,1)
                                tempdHTenor(idx1,idx2) = expiry(idx1) + tenor(idx2);
                            end
                        end
                        tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
                        
                        dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
                        dH2.tenor = dH1.tenor;
                        
                        dH_initial = lgm2FModelParams('dH_initial');
                        
                        % consistency check !
                        if length(dH1.tenor) ~= size(dH_initial,1)
                            error('dH_initial tenor is inconsistent with market swaption!')
                        end
                        
                        dH1.quote = dH_initial(:,1);
                        dH2.quote = dH_initial(:,2);
                        
                        lgm2FModelParams('dH1') = dH1;
                        lgm2FModelParams('dH2') = dH2;
                        
                        dH1Orig = dH1;
                        dH2Orig = dH2;
                        
                        lgm2FModelParams('dH1Orig') = dH1Orig;
                        lgm2FModelParams('dH2Orig') = dH2Orig;
                        
                        Alpha1.tenor =  expiry;
                        Alpha2.tenor =  Alpha1.tenor;
                        correl.tenor = Alpha1.tenor;
                        correl.quote = zeros(length(expiry),1);
                        
                        Alpha_initial = lgm2FModelParams('Alpha_initial');
                        
                        % consistency check !
                        if length(Alpha1.tenor) ~= size(Alpha_initial,1)
                            error('Alpha_initial tenor is inconsistent with market swaption!')
                        end
                        
                        Alpha1.quote = Alpha_initial(:,1);
                        Alpha2.quote = Alpha_initial(:,2);
                        
                        lgm2FModelParams('Alpha1') = Alpha1;
                        lgm2FModelParams('Alpha2') = Alpha2;
                        lgm2FModelParams('correl') = correl;
                        
                        lgm2F = LGM2F(IRModel(Model(asset.allMarketData,lgm2FModelParams)));

                        % calibration
                        calibOut = lgm2F.CalibrateToATMSwaptionSurface(black,'global',lgm2FModelParams('maxIter'));
                        
                        % if reference Model then construnct
                        % RiskyModel(MultiCurve Discounting)
                        if  strcmp(asset.ccy,globalDict('refModelName')) 
                            modelParamsCalib= lgm2FModelParams;

                            dH1Calib = lgm2F.dH1;
                            dH2Calib = lgm2F.dH2;
                            Alpha1Calib = lgm2F.Alpha1;
                            Alpha2Calib = lgm2F.Alpha2;

                            modelParamsCalib('dH1') = dH1Calib;
                            modelParamsCalib('dH2') = dH2Calib;
                            modelParamsCalib('Alpha1') = Alpha1Calib;
                            modelParamsCalib('Alpha2') = Alpha2Calib;

                            lgm2FRisky = LGM2FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));

                            modelNameMap(asset.name) = lgm2FRisky;
                        else
                            modelNameMap(asset.name) = lgm2F;
                        end
                    else
                        error('unknown modelName!')
                    end
                end
        end
    end
    
    %% multiAssetModel Build Start
    multiAssetMktData = containers.Map();
    keys = modelNameMap.keys;
    values = modelNameMap.values;
    for idx1=1:length(keys)
        multiAssetMktData(keys{idx1}) = values{idx1}.mktData;
    end

    %% correlation between asset 
    %% need to be updated later, no octagon correlation
    correlMap = globalDict('correlMap');
    sortedKeys = sort(keys);
    correlationMatrix = ones(length(sortedKeys),length(sortedKeys));
    
    for idx1=1:length(sortedKeys)-1
        firstAssetName = string(sortedKeys{idx1});
        for idx2=idx1+1:length(sortedKeys)
            secondAssetName = string(sortedKeys{idx2});
            correlMapKey = convertStringsToChars(strcat(firstAssetName,'_',secondAssetName));
            if isKey(correlMap,correlMapKey)
                correlationMatrix(idx1,idx2) = correlMap(correlMapKey);
                correlationMatrix(idx2,idx1) = correlMap(correlMapKey);
            else
                error('cannot find correlation between %s and %s',firstAssetName,secondAssetName)
            end  
        end
    end
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelSettings = globalDict('modelSettings');
    multiAssetModelParams = modelSettings('multiAssetModel');
                        
%     multiAssetModelParams = containers.Map();
    multiAssetModelParams('correlationMatrix') = correlationMatrix;
    multiAssetModelParams('modelNameMap') = modelNameMap;
    
    keys = modelNameMap.keys;
    
    stochasticModelNames = {};
    % 1st element matlab cell bad point
    if ~isKey(modelNameMap(keys{1}).modelParams,'stochasticYN') 
            stochasticModelNames{1} = keys{1};
    else
        if strcmp(modelNameMap(keys{i}).modelParams('stochasticYN') ,'YES')
            stochasticModelNames{1} = keys{1};
        end
    end
        
    for idx1=2:length(keys)
        if ~isKey(modelNameMap(keys{idx1}).modelParams,'stochasticYN') 
            stochasticModelNames{end+1} = keys{idx1};
        else
            if strcmp(modelNameMap(keys{idx1}).modelParams('stochasticYN') ,'YES')
                stochasticModelNames{end+1} = keys{idx1};
            end
        end
    end
    
    multiAssetModelParams('stochasticModelNames') = stochasticModelNames;
    
    multiAssetModelParams('refModelName') = globalDict('refModelName');
    multiAssetModelParams('useQuasiRandomYN') = 'false';

    multiAssetModelParams('useBrigoCorrYN') = 'simple_adjust';
    % multiAssetModelParams('useBrigoCorrYN') = 'NO';
%     multiAssetModelParams('useBrigoCorrYN') = 'YES';
    assetNameModelNameMap = globalDict('assetNameModelNameMap');
    
    multiAssetModelName = '';
    for idx=1:length(stochasticModelNames)
        multiAssetModelName = strcat(multiAssetModelName,assetNameModelNameMap(stochasticModelNames{idx}));
    end
    if strcmp(multiAssetModelName,'LGM2FCK1F')
        multiAssetModel = MultiAssetModelLGM2FCK1F(multiAssetMktData,multiAssetModelParams);
    else
        error('no matched model')
    end
    
    globalDict('pricingModel') = multiAssetModel;
    %% multiAssetModel building End
    
    
    %% numMethodInfo
    numMethodSettings = globalDict('numMethodSettings');
    numMethodAMC = NumMethod(numMethodSettings('amc'));
    
    pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRASimpleCall(globalDict('security'),globalDict('pricingModel'),numMethodAMC,false);
%     pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRA(globalDict('security'),globalDict('pricingModel'),numMethodAMC,false);
    
    pricingOutMC = pricer.computePriceMCCallableGeneric(valueDate,'PSA');
    pricingOutClosed = pricer.computePriceClosedSimple(valueDate);
    
    disp(['nonCallNPV;  ',num2str(pricingOutMC.nonCall.npv)])
    disp(['callableNPV;  ',num2str(pricingOutMC.callable.npv)])
    disp(['nonCallClosedNPV;  ',num2str(pricingOutClosed.pricerInfo.nonCall.npv)])
end

aaa = 1.0;




In [ ]:
% hss_Octagon_USDCMSKRWCMSSpreadDualRA_test.m

clc
clear

%% open to user: to price a deal user need to supply the following information
%% product id,
%% context,
%% valueDate

%USDCMSKRWCMSSpreadRA instrument
product = containers.Map({'insId','context',},{'F181025-00022','MtM'});
valueDate = H_Date(20181022);
valueDateRaw = H_Date(20181022);

% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F180802-78418','MtM'});
% 
% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F170216-45342','MtM'});
% 
SimulInfo = struct;
SimulInfo.valueDates = {};
SimulInfo.nonCallNPVs = {};
SimulInfo.nonCallClosedNPVs = {};
SimulInfo.callableNPVs = {};

% global dictionary(map) init
globalDict = containers.Map();

%% assetNameModelNameMap : for each assetName which modelName is used
%% assetName : usedModelName
assetNameModelNameMap =  containers.Map();
assetNameModelNameMap('KRW') = 'LGM2F';
assetNameModelNameMap('KRWUSD') = 'FXQuantoDummy';
assetNameModelNameMap('USD') = 'CK1F';

globalDict('assetNameModelNameMap') = assetNameModelNameMap;
%% static data   

%% calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;

%% MarketData YLD
%yldCurveSetting
yiledCurveSettings = containers.Map();

%KRW_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%% Models CK1F, LGM2F
%modelSettings

modelSettings = containers.Map();

%IRCK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;
ck1FModelParams('assetType') = 'IR';

modelSettings('CK1F') = ck1FModelParams;

%IRLGM2F
lgm2FModelParams = containers.Map();
lgm2FModelParams('dH_initial') = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421];

lgm2FModelParams('Alpha_initial') = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438];

lgm2FModelParams('smoothdH') = 1.0;
lgm2FModelParams('smoothAlpha') = 1.0;
lgm2FModelParams('smoothTermCorr') = 0.1;
lgm2FModelParams('targetTermCorr') = 0.92;
lgm2FModelParams('freq') = 4;
lgm2FModelParams('maxIter') = 200;
lgm2FModelParams('modelName') = 'LGM2F';
lgm2FModelParams('assetType') = 'IR';

modelSettings('LGM2F') = lgm2FModelParams;

%FXBS
fxBSModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.1];
vol_r.tenor = [1.0];
fxBSModelParams('vol_r') = vol_r;

fxBSModelParams('modelName') = 'FXBS';

fxBSModelParams('assetType') = 'FX';

modelSettings('FXBS') = fxBSModelParams;

%% FXQuantoDummy
fxQuantoDummyModelParams = containers.Map();

fxQuantoDummyModelParams('modelName') = 'FXQuantoDummy';

fxQuantoDummyModelParams('assetType') = 'FX';
fxQuantoDummyModelParams('stochasticYN') = 'NO';

modelSettings('FXQuantoDummy') = fxQuantoDummyModelParams;

%% multiAssetModel Settings
multiAssetModelParams = containers.Map();
multiAssetModelParams('refModelName') = 'KRW';
multiAssetModelParams('useQuasiRandomYN') = 'false';
multiAssetModelParams('useBrigoCorrYN') = 'simple_adjust';

modelSettings('multiAssetModel') = multiAssetModelParams;

globalDict('modelSettings') = modelSettings;

%% Correlation data to be upgraded

% correlation mktData is defined by key & value where key is a pair
% (KRW,USD), (KRW,KRWUSD), (KRWUSD,USD)
correlMap = containers.Map();
correlMap('KRW_USD') = 0.3;
correlMap('KRW_KRWUSD') = -0.17;
correlMap('KRWUSD_USD') = 0.1;

globalDict('correlMap') = correlMap;

%% numMethod Settings
numMethodSettings = containers.Map();
%% AMC Settings
amcNumMethodParams = containers.Map();
amcNumMethodParams('numMethodType') = 'AMC';
amcNumMethodParams('mcOneTimeStep') = 7;
amcNumMethodParams('pathSize') = 40000;

numMethodSettings('amc') = amcNumMethodParams;

globalDict('numMethodSettings') = numMethodSettings;
%% Static data setting end

for idxSimul=1:10
    
valueDate = jointCalendar.Advance(valueDateRaw,'Following',idxSimul-1,'day',false);
% valueDate = valueDateRaw.AddDate(idxSimul-1,'day');

if idxSimul==1
    SimulInfo.valueDates{1,1} = FngDate(valueDate);
else
    SimulInfo.valueDates{end+1,1} = FngDate(valueDate);
end


%% Instrument API > Security > Underlyings > Mapped MarketData > Model Building
% get Instrument info

instrumentQueryStr = 'http://45.249.2.10:50100/data/instruments/';
queryStr = strcat(instrumentQueryStr,product('insId'));
instrumentJson = webread(queryStr);

%% instrument layer info
instrumentPayCcy = instrumentJson.CcyCd.Cd;

globalDict('payCcy') = instrumentPayCcy;

components = instrumentJson.Components;

valGroup = strcat(components.ProdTpNm,'_',components.TmpltNm);

globalDict('valGroup') = valGroup;

%
if strcmp(valGroup,'InterestRateDerivatives_DualSpreadRA')
   %% SimpleRA Info 
    %security Info : startDate,endDate,nominal,payCcy from instrument.Components
    % construct rangeAccrualParams
    
    rangeAccrualParams.payCcy = globalDict('payCcy');
    
    % reference ModelName
    globalDict('refModelName') = rangeAccrualParams.payCcy;
    
    rangeAccrualParams.startDate = H_Date(components.EffDt);
    rangeAccrualParams.endDate = H_Date(components.EndDt);
    
    rangeAccrualInfo = containers.Map();

    rangeAccrualInfo('nominal') = components.UnitAmt;
    
    %% payoff info start from instrument.Components.Payoffs
    payoffs = components.Payoffs;
    
    % underlyings ordering
    % Underlying1Id  : index name for range accrual observation
    % Underlying2Id  : index name for spread ragne accrual observation
    % (Long)
    % Underlying3Id  : index name for spread ragne accrual observation
    % (Short)
    
    rangeAccrualInfo('RAIndexName') = payoffs.Underlying1Id;
    rangeAccrualInfo('SpreadRAIndexNameLong') = payoffs.Underlying2Id;
    rangeAccrualInfo('SpreadRAIndexNameShort') = payoffs.Underlying3Id;
    
    % Coupon
    rangeAccrualInfo('coupon') = payoffs.FixedIr;  % 
    
    % Single RA
    rangeAccrualInfo('upper') = 1.0;
    if isfield(payoffs,'Cap1')
        rangeAccrualInfo('upper') = payoffs.Cap1;
    end

    rangeAccrualInfo('lower') = -1.0;
    if isfield(payoffs,'Floor1')
        rangeAccrualInfo('lower') = payoffs.Floor1;
    end

    % Spread RA
    
    rangeAccrualInfo('lowerS') = -1.0;
    if isfield(payoffs,'Floor2')
        rangeAccrualInfo('lowerS') = payoffs.Floor2;
    end

    
    rangeAccrualInfo('upperS') = 1.0;
    if isfield(payoffs,'Cap2')
        rangeAccrualInfo('upperS') = payoffs.Cap2;
    end
    
    %% payoff info end from instrument.Components.Payoffs

    %% Schedule info start from instrument.Components.Items
    %% structuredLeg, callableLeg Schedule
    
    items = instrumentJson.Components.Items;
    % find structured leg
    structuredLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemSubTpNm,'structured')
            structuredLeg = items{idx};
            break;
        end
    end
    
    rangeAccrualInfo('couponFreq') = structuredLeg.ExerFreqCd.Cd;
    
    % generated Schedule
    structuredSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
                                    rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Forward',false);
    % manual Schedule
    structuredScheduleRaw = structuredLeg.Cashflows;
    %reset start end pay FngDate format array
    couponScheduleInt = zeros(length(structuredLeg.Cashflows),4);
    for idx =1 :length(structuredScheduleRaw)
        couponScheduleInt(idx,1) = FngDate(H_Date(structuredScheduleRaw(idx).ResetDt));
        couponScheduleInt(idx,2) = FngDate(H_Date(structuredScheduleRaw(idx).BegDt));
        couponScheduleInt(idx,3) = FngDate(H_Date(structuredScheduleRaw(idx).EndDt));
        couponScheduleInt(idx,4) = FngDate(H_Date(structuredScheduleRaw(idx).PayDt));
        
    end
    
    rangeAccrualInfo('couponScheduleInt') = couponScheduleInt;
    
    % find callable exerciseEvents
    callalbleLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemTpNm,'callable')
            callalbleLeg = items{idx};
            break;
        end
    end
    
    % generated Schedule
    nonCallTermYear = callalbleLeg.NonCallTermYear;
    %% CallNotice OffSet reflected Call Schedule Generation
    %% to be added
    
    callStartDate = globalDict('jointCalendar').AdvanceTenor(rangeAccrualParams.startDate,'ModifiedFollowing',strcat(num2str(nonCallTermYear),'Y'),false);
                    
    callFreq = callalbleLeg.ExerFreqCd.Cd;
    
    callableSchedule0 = Schedule(callStartDate,rangeAccrualParams.endDate,...
                                    callFreq ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Forward',false);

    % should be fixed to use callable schedule directly !
    callYN = ones(size(couponScheduleInt,1),1);
    %only yearly schedule
    for i=1:length(callYN)
        if i <= nonCallTermYear
            callYN(i) = 0;
        end
    end
    
    % not used in the pricer ended with simpleCall ! dummyVariable
    rangeAccrualInfo('callYN') = callYN;
    
    % manual Schedule
    callalbleScheduleRaw = callalbleLeg.ExerciseEvents;
    %reset start end pay FngDate format array
    callScheduleInt  = zeros(length(callalbleLeg.ExerciseEvents),4);
    for idx = 1:length(callalbleScheduleRaw)
        callScheduleInt(idx,1) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,2) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
    end
    
    rangeAccrualInfo('callScheduleInt') = callScheduleInt;
    
    %% fixings to be updated later
    %% fixing needs underlings info 
    
     
    rangeDays = zeros(size(couponScheduleInt,1),1);
    %% full range
    for i=1:length(rangeDays)
        if DateDiff(valueDate,H_Date(couponScheduleInt(i,2))) >= 0 ...
            && DateDiff(valueDate,H_Date(couponScheduleInt(i,3))) < 0
            rangeDays(i) = DateDiff(valueDate,H_Date(couponScheduleInt(i,2))) + 1;
        elseif DateDiff(valueDate,H_Date(couponScheduleInt(i,3))) >= 0
            rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
        end
    end
    
    rangeAccrualInfo('rangeDays') = rangeDays;
    rangeAccrualInfo('manualScheduleYN') = 1;
    rangeAccrualInfo('callStrike') = 1.0;
    globalDict('securityInfo') = rangeAccrualInfo;

%     rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
%                                 'tenorS','lowerS','upperS','tenor','lower','upper',...
%                                 'rangeDays','callYN', ...
%                                 'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
%                         {nominal,coupon,couponFreq,tenorL,tenorS,lowerS, ...
%                         upperS,tenor,lower,upper,...
%                         rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});


%     rangeAccrualParams.params = rangeAccrualInfo;              
%     rangeAccrual = CMSCMSSpreadDualRangeAccrual(rangeAccrualParams);
% 
%     globalDict('security') = rangeAccrual;
%     
    
   %% Underlying Info from instrumentJson.Components.Underlyings
    % getUnderlyings
    underlyings = components.Underlyings;
    underlyingsInfo = struct;
    for i=1:length(underlyings)
        underlyingsInfo(i).name = underlyings(i).UnderlyingId;
        underlyingsInfo(i).type = underlyings(i).Instrument.InsTpCd.Cd;
        underlyingsInfo(i).ccy  = underlyings(i).Instrument.CcyCd.Cd;
        if strcmp(underlyingsInfo(i).name(4:6),'IRS')
            underlyingsInfo(i).assetType = 'IR';
            underlyingsInfo(i).rateType = 'CMS';
            if strcmp(underlyingsInfo(i).name(end),'Y') 
                underlyingsInfo(i).rateTenor = uint16(str2num(underlyingsInfo(i).name(end-2:end-1)));
            else
                error('CMS Rate tenor should be given in years!')
            end
        elseif strcmp(underlyingsInfo(i).name(4:6),'LBR') || strcmp(underlyingsInfo(i).name,'CD91D') || strcmp(underlyingsInfo(i).name,'KTB3M')
            underlyingsInfo(i).assetType = 'IR';
            underlyingsInfo(i).rateType = 'Libor';
            underlyingsInfo(i).rateTenor = 0.25;
        else
            error('unknown index type!')
        end
            
    end
    
    % underlyings info in the global dictionary
    globalDict('underlyingsInfo') = underlyingsInfo;
    
    rangeAccrualInfo = globalDict('securityInfo');
   
    %% need to be generalized to include general underlyings in the payoff description using functor
%     rangeAccrualInfo('RAIndexName') = payoffs.Underlying1Id;
%     rangeAccrualInfo('SpreadRAIndexNameLong') = payoffs.Underlying2Id;
%     rangeAccrualInfo('SpreadRAIndexNameShort') = payoffs.Underlying3Id;
    
    %find RAIndex in the underlyings
    for idx=1:length(underlyingsInfo)
        if strcmp(underlyingsInfo(idx).name,rangeAccrualInfo('RAIndexName'))
            rangeAccrualInfo('tenor') = underlyingsInfo(idx).rateTenor;
            break;
        end
    end
    
     %find SpreadRAIndexLong in the underlyings
    for idx=1:length(underlyingsInfo)
        if strcmp(underlyingsInfo(idx).name,rangeAccrualInfo('SpreadRAIndexNameLong'))
            rangeAccrualInfo('tenorL') = underlyingsInfo(idx).rateTenor;
            break;
        end
    end
    
     %find SpreadRAIndexShort in the underlyings
    for idx=1:length(underlyingsInfo)
        if strcmp(underlyingsInfo(idx).name,rangeAccrualInfo('SpreadRAIndexNameShort'))
            rangeAccrualInfo('tenorS') = underlyingsInfo(idx).rateTenor;
            break;
        end
    end
    
    rangeAccrualParams.params = rangeAccrualInfo;              
    rangeAccrual = CMSCMSSpreadDualRangeAccrual(rangeAccrualParams);

    globalDict('security') = rangeAccrual;

    %% fixings to be added here
    
    
    %% From underlyings info , we fill rangeAccrualInfo: tenor, tenorL, tenorS
    
    
    %% Build Models based on underlyings 
    %% Get the mapped market data using context
    %% Build market data
    %% Build model
    %% Link built model to asset
    %% containers to have assetName : usedModel Object
    modelNameMap =  containers.Map();
    
    for i=1:length(underlyings)
        
        if strcmp(underlyingsInfo(i).type,'RATE_INDEX')
                % for RATE INDEX, underlyings currency is the asset name!
                % asset is not in the assetModel Yet then we add new asset!
                % otherwise we skip
                
                if ~isKey(modelNameMap,underlyingsInfo(i).ccy)
                    %get Underlying Mapped MarketData
                    asset = struct;
                    asset.name = underlyingsInfo(i).ccy;
                    asset.ccy = asset.name;
                    asset.type = 'IR';

                    %% marketData init

                    asset.allMarketData = containers.Map();

                    % if asset is discountingModel(referenceModel) then add mappedDiscountYLDCurve
                    if strcmp(asset.ccy,globalDict('payCcy'))
                        %% discount YLD
                        discountYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                            asset.ccy,'&mapTrgtTpCd=DISCOUNT');
                        mappedYldJson = webread(discountYldQueryStr);
                        yldCurveName = mappedYldJson.Items.MapTrgtNm;
                        bootStrapFlag = true;
                       %bootStrapFlag = false;
                        % if bootStrapFlag is true then we strip zeroCurve
                        ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);
                        %YLD Curve
                        asset.allMarketData('discountYLD') = ycData;
                        %Zero Curve
                        asset.allMarketData('discountCurve') = ycData.zcCurve.zeroCurve;

%                         globalDict('refModelName') = asset.ccy;
                    % if asset's currency is different then we need FX
                    % Model
                    else
%                         asset.ccy,globalDict('payCcy')
                        fxAsset = struct;
                        fxAsset.ccy = asset.ccy;
                        
                        fxAsset.name = strcat(globalDict('payCcy'),fxAsset.ccy);
                        
                        fxAsset.type = 'FX';
                        fxAsset.allMarketData = containers.Map();
                        
                        % FX MarketData
                     %% FX ATM TermStructure Vol
                        fxVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                        'FX',fxAsset.name,'&mapTrgtTpCd=VOL');
                        fxVolJson = webread(fxVolQueryStr);

                        fxVolName = fxVolJson.Items.MapTrgtNm;
                        
                        fxVolData = GetFXVolTermStructureFromOctagon(valueDate,fxVolName);
                        rawDataFXVol = fxVolData.rawData;
                        mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataFXVol,valueDate,fxAsset.ccy,fxVolName});
                    
                        impliedTermVolMktData = MktData(mktData);
                        
                        fxAsset.allMarketData('impliedTermVol') = impliedTermVolMktData;
                        
                     %% fxQuantoDummy Model Init
                        assetNameModelNameMap = globalDict('assetNameModelNameMap');
                        fxAsset.modelName = assetNameModelNameMap(fxAsset.name);

                        if strcmp(fxAsset.modelName,'FXQuantoDummy')
                            % construct FXQuantoDummy model
                            modelSettings = globalDict('modelSettings');
                            fxQuantoDummyModelParams = modelSettings('FXQuantoDummy');
                            fxQuantoDummy = Model(fxAsset.allMarketData,fxQuantoDummyModelParams);
                            modelNameMap(fxAsset.name) = fxQuantoDummy;
                        end
                    end

                    %% fwd YLD
                    fwdYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                        underlyingsInfo(1).name,'&mapTrgtTpCd=RISKFREE');
                    mappedYldJson = webread(fwdYldQueryStr);
                    yldCurveName = mappedYldJson.Items.MapTrgtNm;
                    bootStrapFlag = true;
                   %bootStrapFlag = false;
                    ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);

                    asset.allMarketData('fwdYLD') = ycData;
                    asset.allMarketData('zeroCurve') = ycData.zcCurve.zeroCurve;
                    
                    %% Swaption Vol
                    
                    %get mapped swaption volatility
                    swaptionVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                        underlyingsInfo(1).ccy,'&mapTrgtTpCd=SWAPTVOL');
                    mappedVolJson = webread(swaptionVolQueryStr);

                    swaptionVolName = mappedVolJson.Items.MapTrgtNm;

                    swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,swaptionVolName);
                    rawDataSwaption = swaptionVolData.rawData;

                    asset.allMarketData('swaptionVolRaw') = rawDataSwaption;

                    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                                {rawDataSwaption,valueDate, underlyingsInfo(1).ccy,strcat(underlyingsInfo(1).ccy,'SwaptionVol')});
                    swaptionVolMktData = MktData(mktData);
                    asset.allMarketData('swaptionVol') = swaptionVolMktData;

                     %% marketData end
                     
                    %% model Init
                    assetNameModelNameMap = globalDict('assetNameModelNameMap');
                    asset.modelName = assetNameModelNameMap(asset.name);

                    if strcmp(asset.modelName,'CK1F')
                        % construct ck1F model
                        modelSettings = globalDict('modelSettings');
                        ck1FModelParams = modelSettings('CK1F');

                        % added modelParams

                        ck1FModelParams('instrumentStartDate') = rangeAccrualParams.startDate;
                        ck1FModelParams('instrumentEndDate') = rangeAccrualParams.endDate;
                        ck1FModelParams('calibrationTargetTenor') =  1:1:14;

                        ck1F = CK1F(IRModel(Model(asset.allMarketData,ck1FModelParams)));

                        % calibration
                        % construct black model 
                        blackModelParams= containers.Map({'modelName'},{'Black'});
                        black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 


                        calibOut = ck1F.CalibrateToATMDiagSwaptionForward(black,'global');
                        
                        % if reference Model then construnct
                        % RiskyModel(MultiCurve Discounting)
                        if  strcmp(asset.ccy,globalDict('refModelName')) 
                            vol_r_Calib = ck1F.vol_r;

                            modelParamsCalib= ck1FModelParams;
                            modelParamsCalib('vol_r') = vol_r_Calib;

                            ck1FRisky = CK1FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));

                            modelNameMap(asset.name) = ck1FRisky;
                        else
                            modelNameMap(asset.name) = ck1F;
                        end
                        % map is a handle class so you don't need the foillowing step
        %                 globalDict('assetModelMap') = assetModelMap;
                    
                    elseif strcmp(asset.modelName,'LGM2F')
                        % calibration
                        % construct black model 
                        blackModelParams= containers.Map({'modelName'},{'Black'});
                        black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 
                        
                        % construct lgm2F model
                        modelSettings = globalDict('modelSettings');
                        lgm2FModelParams = modelSettings('LGM2F');
                        
                        % added modelParams
                        rawDataSwaption = asset.allMarketData('swaptionVolRaw');
                        expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
                        tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
                        
                        tempdHTenor = zeros(size(expiry,1),size(tenor,1));
                        for idx1=1:size(expiry,1)
                            for idx2=1:size(tenor,1)
                                tempdHTenor(idx1,idx2) = expiry(idx1) + tenor(idx2);
                            end
                        end
                        tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
                        
                        dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
                        dH2.tenor = dH1.tenor;
                        
                        dH_initial = lgm2FModelParams('dH_initial');
                        
                        % consistency check !
                        if length(dH1.tenor) ~= size(dH_initial,1)
                            error('dH_initial tenor is inconsistent with market swaption!')
                        end
                        
                        dH1.quote = dH_initial(:,1);
                        dH2.quote = dH_initial(:,2);
                        
                        lgm2FModelParams('dH1') = dH1;
                        lgm2FModelParams('dH2') = dH2;
                        
                        dH1Orig = dH1;
                        dH2Orig = dH2;
                        
                        lgm2FModelParams('dH1Orig') = dH1Orig;
                        lgm2FModelParams('dH2Orig') = dH2Orig;
                        
                        Alpha1.tenor =  expiry;
                        Alpha2.tenor =  Alpha1.tenor;
                        correl.tenor = Alpha1.tenor;
                        correl.quote = zeros(length(expiry),1);
                        
                        Alpha_initial = lgm2FModelParams('Alpha_initial');
                        
                        % consistency check !
                        if length(Alpha1.tenor) ~= size(Alpha_initial,1)
                            error('Alpha_initial tenor is inconsistent with market swaption!')
                        end
                        
                        Alpha1.quote = Alpha_initial(:,1);
                        Alpha2.quote = Alpha_initial(:,2);
                        
                        lgm2FModelParams('Alpha1') = Alpha1;
                        lgm2FModelParams('Alpha2') = Alpha2;
                        lgm2FModelParams('correl') = correl;
                        
                        lgm2F = LGM2F(IRModel(Model(asset.allMarketData,lgm2FModelParams)));

                        % calibration
                        calibOut = lgm2F.CalibrateToATMSwaptionSurface(black,'global',lgm2FModelParams('maxIter'));
                        
                        % if reference Model then construnct
                        % RiskyModel(MultiCurve Discounting)
                        if  strcmp(asset.ccy,globalDict('refModelName')) 
                            modelParamsCalib= lgm2FModelParams;

                            dH1Calib = lgm2F.dH1;
                            dH2Calib = lgm2F.dH2;
                            Alpha1Calib = lgm2F.Alpha1;
                            Alpha2Calib = lgm2F.Alpha2;

                            modelParamsCalib('dH1') = dH1Calib;
                            modelParamsCalib('dH2') = dH2Calib;
                            modelParamsCalib('Alpha1') = Alpha1Calib;
                            modelParamsCalib('Alpha2') = Alpha2Calib;

                            lgm2FRisky = LGM2FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));

                            modelNameMap(asset.name) = lgm2FRisky;
                        else
                            modelNameMap(asset.name) = lgm2F;
                        end
                    else
                        error('unknown modelName!')
                    end
                end
        end
    end
    
    %% multiAssetModel Build Start
    multiAssetMktData = containers.Map();
    keys = modelNameMap.keys;
    values = modelNameMap.values;
    for idx1=1:length(keys)
        multiAssetMktData(keys{idx1}) = values{idx1}.mktData;
    end

    %% correlation between asset 
    %% need to be updated later, no octagon correlation
    correlMap = globalDict('correlMap');
    sortedKeys = sort(keys);
    correlationMatrix = ones(length(sortedKeys),length(sortedKeys));
    
    for idx1=1:length(sortedKeys)-1
        firstAssetName = string(sortedKeys{idx1});
        for idx2=idx1+1:length(sortedKeys)
            secondAssetName = string(sortedKeys{idx2});
            correlMapKey = convertStringsToChars(strcat(firstAssetName,'_',secondAssetName));
            if isKey(correlMap,correlMapKey)
                correlationMatrix(idx1,idx2) = correlMap(correlMapKey);
                correlationMatrix(idx2,idx1) = correlMap(correlMapKey);
            else
                error('cannot find correlation between %s and %s',firstAssetName,secondAssetName)
            end  
        end
    end
    
    multiAssetMktData('correlationMatrix') = correlationMatrix;
    
    modelSettings = globalDict('modelSettings');
    multiAssetModelParams = modelSettings('multiAssetModel');
                        
%     multiAssetModelParams = containers.Map();
    multiAssetModelParams('correlationMatrix') = correlationMatrix;
    multiAssetModelParams('modelNameMap') = modelNameMap;
    
    keys = modelNameMap.keys;
    
    stochasticModelNames = {};
    % 1st element matlab cell bad point
    if ~isKey(modelNameMap(keys{1}).modelParams,'stochasticYN') 
            stochasticModelNames{1} = keys{1};
    else
        if strcmp(modelNameMap(keys{i}).modelParams('stochasticYN') ,'YES')
            stochasticModelNames{1} = keys{1};
        end
    end
        
    for idx1=2:length(keys)
        if ~isKey(modelNameMap(keys{idx1}).modelParams,'stochasticYN') 
            stochasticModelNames{end+1} = keys{idx1};
        else
            if strcmp(modelNameMap(keys{idx1}).modelParams('stochasticYN') ,'YES')
                stochasticModelNames{end+1} = keys{idx1};
            end
        end
    end
    
    multiAssetModelParams('stochasticModelNames') = stochasticModelNames;
    
    multiAssetModelParams('refModelName') = globalDict('refModelName');
    multiAssetModelParams('useQuasiRandomYN') = 'false';

    multiAssetModelParams('useBrigoCorrYN') = 'simple_adjust';
    % multiAssetModelParams('useBrigoCorrYN') = 'NO';
%     multiAssetModelParams('useBrigoCorrYN') = 'YES';
    assetNameModelNameMap = globalDict('assetNameModelNameMap');
    
    multiAssetModelName = '';
    for idx=1:length(stochasticModelNames)
        multiAssetModelName = strcat(multiAssetModelName,assetNameModelNameMap(stochasticModelNames{idx}));
    end
    if strcmp(multiAssetModelName,'LGM2FCK1F')
        multiAssetModel = MultiAssetModelLGM2FCK1F(multiAssetMktData,multiAssetModelParams);
    else
        error('no matched model')
    end
    
    globalDict('pricingModel') = multiAssetModel;
    %% multiAssetModel building End
    
    
    %% numMethodInfo
    numMethodSettings = globalDict('numMethodSettings');
    numMethodAMC = NumMethod(numMethodSettings('amc'));
    
    pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRASimpleCall(globalDict('security'),globalDict('pricingModel'),numMethodAMC,true);
    
    pricingOutMC = pricer.computePriceMCCallableGeneric(valueDate,'PSA');
    pricingOutClosed = pricer.computePriceClosedSimple(valueDate);
    
    if idxSimul==1
        SimulInfo.nonCallNPVs{1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    else
        SimulInfo.nonCallNPVs{end+1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{end+1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{end+1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    end
    
    disp(['nonCallNPV;  ',num2str(pricingOutMC.nonCall.npv)])
    disp(['callableNPV;  ',num2str(pricingOutMC.callable.npv)])
    disp(['nonCallClosedNPV;  ',num2str(pricingOutClosed.pricerInfo.nonCall.npv)])
end

end
aaa = 1.0;




In [ ]:
% hss_Octagon_CallableDualSpreadRA_test.m


clc
clear

%CDRA instrument
% product = containers.Map({'insId','context',},{'F181015-00005','MtM'});

% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F160517-34710','MtM'});
product = containers.Map({'insId','context',},{'F180802-78418','MtM'});


% 
% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F170216-45342','MtM'});
% 

valueDate = H_Date(20181015);
valueDateRaw = H_Date(20181015);
SimulInfo = struct;
SimulInfo.valueDates = {};
SimulInfo.nonCallNPVs = {};
SimulInfo.nonCallClosedNPVs = {};
SimulInfo.callableNPVs = {};

% global dictionary init
globalDict = containers.Map();

%% static data
% default calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;

% static Setting for globalDict
%yldCurveSetting

yiledCurveSettings = containers.Map();

%KRW_IRS Setting

yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';


globalDict('modelNameMap') = modelNameMap;

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;



for idxSimul=1:10
    
valueDate = jointCalendar.Advance(valueDateRaw,'Following',idxSimul-1,'day',false);
% valueDate = valueDateRaw.AddDate(idxSimul-1,'day');

if idxSimul==1
    SimulInfo.valueDates{1,1} = FngDate(valueDate);
else
    SimulInfo.valueDates{end+1,1} = FngDate(valueDate);
end


% get Instrument info

% instrumentQueryStr = 'http://45.249.2.10:50100/data/instruments/';
instrumentQueryStr = 'http://45.12.71.111:50100/data/instruments/';


queryStr = strcat(instrumentQueryStr,product('insId'));
instrumentJson = webread(queryStr);

instrumentPayCcy = instrumentJson.CcyCd.Cd;

globalDict('payCcy') = instrumentPayCcy;

components = instrumentJson.Components;

valGroup = strcat(components.ProdTpNm,'_',components.TmpltNm);

globalDict('valGroup') = valGroup;

%
if strcmp(valGroup,'InterestRateDerivatives_DualSpreadRA')
   %% SimpleRA Info 
    %security Info : startDate,endDate,nominal,payCcy from instrument.Components
    % construct rangeAccrualParams
    
    rangeAccrualParams.payCcy = globalDict('payCcy');
    rangeAccrualParams.startDate = H_Date(components.EffDt);
    rangeAccrualParams.endDate = H_Date(components.EndDt);
    
    rangeAccrualInfo = containers.Map();

    rangeAccrualInfo('nominal') = components.UnitAmt;
    
    %% payoff info start from instrument.Components.Payoffs
    payoffs = components.Payoffs;
 
    rangeAccrualInfo('coupon') = payoffs.FixedIr * 100.0;  % input mistake 0.03%
    
    rangeAccrualInfo('upper') = 0.0;
    if isfield(payoffs,'Cap1')
        rangeAccrualInfo('upper') = payoffs.Cap1;
    end

    rangeAccrualInfo('lower') = 0.0;
    if isfield(payoffs,'Floor1')
        rangeAccrualInfo('lower') = payoffs.Floor1;
    end

    
%% payoff info end from instrument.Components.Payoffs
    
   %% Underlying Info from instrumentJson.Components.Underlyings
    % getUnderlyings
    underlyings = components.Underlyings;
    underlyingsInfo = struct;
    for i=1:length(underlyings)
        underlyingsInfo(i).name = underlyings(i).UnderlyingId;
        underlyingsInfo(i).type = underlyings(i).Instrument.InsTpCd.Cd;
        underlyingsInfo(i).ccy  = underlyings(i).Instrument.CcyCd.Cd; 
    end
    
    % underlyingsInfo : underlying tenor Info
    if strcmp(underlyingsInfo(1).name,'CD91D')
        rangeAccrualInfo('tenor') = 0.25;
    end
    
    % GetUnderlyingObject
    if strcmp(underlyingsInfo(1).type,'RATE_INDEX')
            %get mapped fwd yld\
            asset = struct;
            asset.name = underlyingsInfo(1).ccy;
            asset.ccy = asset.name;
            asset.type = 'IR';
            
            %marketData init
            
            asset.allMarketData = containers.Map();
            
            % if referenceModel (discounting) then add mappedDiscountYLDCurve
            if strcmp(asset.ccy,globalDict('payCcy'))
                %discountYLDCurve
%                 discountYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
%                                     asset.ccy,'&mapTrgtTpCd=DISCOUNT');
%                                 
                discountYldQueryStr = strcat('http://45.12.71.111:50100/data/contexts/MtM?paramId=', ...
                                    asset.ccy,'&mapTrgtTpCd=DISCOUNT');
                                
                mappedYldJson = webread(discountYldQueryStr);
                
                itemDummy = getfield(mappedYldJson.Items, strcat('INS_', asset.ccy, '___DISCOUNT'));
                
                yldCurveName = itemDummy.MapTrgtNm;
                bootStrapFlag = true;
%             bootStrapFlag = false;
                ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);

                asset.allMarketData('discountYLD') = ycData;
                asset.allMarketData('discountCurve') = ycData.zcCurve.zeroCurve;
                
                globalDict('refModelName') = asset.ccy;
            end
            
%             modelSettings = globalDict('modelSettings'); 
%             modelParams = modelSettings(pricingModelName);
%             construct = str2func(pricingModelName);
%             pricingModel = construct(modelParams);
%             KRWLGM2F = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));
%             if strcmp(pricingModel,'CK1F')
%                 aaa = 1.0;
%             end
            
            %fwdYLD
            fwdYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                underlyingsInfo(1).name,'&mapTrgtTpCd=RISKFREE');
            mappedYldJson = webread(fwdYldQueryStr);
            yldCurveName = mappedYldJson.Items.MapTrgtNm;
            bootStrapFlag = true;
%             bootStrapFlag = false;
            ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);
            
            asset.allMarketData('fwdYLD') = ycData;
            asset.allMarketData('zeroCurve') = ycData.zcCurve.zeroCurve;
            %discountYLD
            
            %get mapped swaption volatility
            swaptionVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                underlyingsInfo(1).ccy,'&mapTrgtTpCd=SWAPTVOL');
            mappedVolJson = webread(swaptionVolQueryStr);
            
%             swaptionVolName = mappedVolJson.Items.MapTrgtId;
            swaptionVolName = mappedVolJson.Items.MapTrgtNm;

            swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,swaptionVolName);
            rawDataSwaption = swaptionVolData.rawData;
            
            asset.allMarketData('swaptionVolRaw') = rawDataSwaption;
            
            mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate, underlyingsInfo(1).ccy,strcat(underlyingsInfo(1).ccy,'SwaptionVol')});
            swaptionVolMktData = MktData(mktData);
            asset.allMarketData('swaptionVol') = swaptionVolMktData;
            
            %modelInit
            modelNameMap = globalDict('modelNameMap');
            asset.modelName = modelNameMap(asset.name);
            
            if strcmp(asset.modelName,'CK1F')
                % construct ck1F model
                modelSettings = globalDict('modelSettings');
                ck1FModelParams = modelSettings('CK1F');
                
                % added modelParams
                
                ck1FModelParams('instrumentStartDate') = rangeAccrualParams.startDate;
                ck1FModelParams('instrumentEndDate') = rangeAccrualParams.endDate;
                ck1FModelParams('calibrationTargetTenor') =  1:1:14;
                
                ck1F = CK1F(IRModel(Model(asset.allMarketData,ck1FModelParams)));
                
                % calibration
                % construct black model 
                blackModelParams= containers.Map({'modelName'},{'Black'});
                black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 
                
                
                calibOut = ck1F.CalibrateToATMDiagSwaptionForward(black,'global');
                
                vol_r_Calib = ck1F.vol_r;
                
                modelParamsCalib= ck1FModelParams;
                modelParamsCalib('vol_r') = vol_r_Calib;

                ck1FRisky = CK1FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));
                
                assetModelMap = globalDict('assetModelMap');
                assetModelMap(asset.name) = ck1FRisky;
                
                % map is a handle class so you don't need this step
%                 globalDict('assetModelMap') = assetModelMap;
                
            end
            
    end
    
    % instrument info : rnageAccrualLeg, (structured leg , callable leg
    % schedule
    items = instrumentJson.Components.Items;
    % find structured leg
    structuredLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemSubTpNm,'structured')
            structuredLeg = items{idx};
            break;
        end
    end
    
    rangeAccrualInfo('couponFreq') = structuredLeg.ExerFreqCd.Cd;
    
    % generated Schedule
    structuredSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
                                    rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Backward',false);
    % manual Schedule
    structuredScheduleRaw = structuredLeg.Cashflows;
    %reset start end pay FngDate format array
    couponScheduleInt = zeros(length(structuredLeg.Cashflows),4);
    for idx =1 :length(structuredScheduleRaw)
        couponScheduleInt(idx,1) = FngDate(H_Date(structuredScheduleRaw(idx).ResetDt));
        couponScheduleInt(idx,2) = FngDate(H_Date(structuredScheduleRaw(idx).BegDt));
        couponScheduleInt(idx,3) = FngDate(H_Date(structuredScheduleRaw(idx).EndDt));
        couponScheduleInt(idx,4) = FngDate(H_Date(structuredScheduleRaw(idx).PayDt));
        
    end
    
    rangeAccrualInfo('couponScheduleInt') = couponScheduleInt;
    
    % find callable leg
    callalbleLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemTpNm,'callable')
            callalbleLeg = items{idx};
            break;
        end
    end
    
    % generated Schedule
%     callableSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
%                                     rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
%                                     'ModifiedFollowing', 'Backward',false);

    % should be fixed to use callable schedule directly !
    nonCallTermYear = callalbleLeg.NonCallTermYear;
    
    callYN = ones(size(couponScheduleInt,1),1);
    %only yearly schedule
    for i=1:length(callYN)
        if i <= nonCallTermYear
            callYN(i) = 0;
        end
    end
    
    rangeAccrualInfo('callYN') = callYN;
    
    % manual Schedule
    callalbleScheduleRaw = callalbleLeg.ExerciseEvents;
    %reset start end pay FngDate format array
    callScheduleInt  = zeros(length(callalbleLeg.ExerciseEvents),4);
    for idx = 1:length(callalbleScheduleRaw)
        callScheduleInt(idx,1) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,2) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,3) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
        callScheduleInt(idx,4) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
        
    end
    
    callScheduleInt = [20181012	20181012	20191014	20191014
20191009	20191014	20201012	20201012
20201007	20201012	20211012	20211012
20211007	20211012	20221012	20221012
20221007	20221012	20231012	20231012
20231009	20231012	20241014	20241014
20241009	20241014	20251013	20251013
20251008	20251013	20261012	20261012
20261007	20261012	20271012	20271012
20271007	20271012	20281012	20281012
20281009	20281012	20291012	20291012
20291008	20291012	20301014	20301014
20301009	20301014	20311013	20311013
20311008	20311013	20321012	20321012
20321007	20321012	20331012	20331012
];
    rangeAccrualInfo('callScheduleInt') = callScheduleInt;
    
    
    % fixings to do later
    
    rangeDays = zeros(size(couponScheduleInt,1),1);
    %rangeDays(1) = 1;
    rangeDays(1) = DateDiff(valueDate,rangeAccrualParams.startDate) + 1;
    
    rangeAccrualInfo('rangeDays') = rangeDays;
    
    rangeAccrualInfo('manualScheduleYN') = 1;
%     rangeAccrualInfo('callStrike') = rangeAccrualInfo('nominal');
    rangeAccrualInfo('callStrike') = 1.0;


%     rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenor','lower','upper','rangeDays','callYN', ...
%                                 'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
%                         {nominal,coupon,couponFreq,tenor,lower,upper,rangeDays,callYN,...
%                        couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;              

    rangeAccrual = CDRangeAccrual(rangeAccrualParams);

    globalDict('security') = rangeAccrual;
    

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % weekly step calculator
    % pricer = Pricer(rangeAccrual,USDLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,USDLGMRisky,numMethodAMC,true);
    assetModelMap = globalDict('assetModelMap');
    
    pricer = DailyWeeklyRiskyPricer_CDRA(globalDict('security'),assetModelMap(globalDict('refModelName')),numMethodAMC,false);
    pricingOutMC = pricer.computePriceMCCallable(valueDate);
    pricingOutClosed = pricer.computePriceClosedSimple(valueDate);
    
    if idxSimul==1
        SimulInfo.nonCallNPVs{1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    else
        SimulInfo.nonCallNPVs{end+1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{end+1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{end+1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    end
    
    disp(['nonCallNPV;  ',num2str(pricingOutMC.nonCall.npv)])
    disp(['callableNPV;  ',num2str(pricingOutMC.callable.npv)])
    disp(['nonCallClosedNPV;  ',num2str(pricingOutClosed.pricerInfo.nonCall.npv)])
end

end
aaa = 1.0;




In [ ]:
% hss_Octagon_CallableCDRA_test_20190522.m

clc
clear

%CDRA instrument
product = containers.Map({'insId','context',},{'F181015-00005','MtM'});

% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F180802-78418','MtM'});
% 
% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F170216-45342','MtM'});
% 

valueDate = H_Date(20190522);
valueDateRaw = H_Date(20190522);
SimulInfo = struct;
SimulInfo.valueDates = {};
SimulInfo.nonCallNPVs = {};
SimulInfo.nonCallClosedNPVs = {};
SimulInfo.callableNPVs = {};

% global dictionary init
globalDict = containers.Map();

%% static data
% default calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;

% static Setting for globalDict
%yldCurveSetting

yiledCurveSettings = containers.Map();

%KRW_IRS Setting

yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';


globalDict('modelNameMap') = modelNameMap;

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;



for idxSimul=1:1
    
valueDate = jointCalendar.Advance(valueDateRaw,'Following',idxSimul-1,'day',false);
% valueDate = valueDateRaw.AddDate(idxSimul-1,'day');

if idxSimul==1
    SimulInfo.valueDates{1,1} = FngDate(valueDate);
else
    SimulInfo.valueDates{end+1,1} = FngDate(valueDate);
end


% get Instrument info

instrumentQueryStr = 'http://45.249.2.10:50100/data/instruments/';
queryStr = strcat(instrumentQueryStr,product('insId'));
instrumentJson = webread(queryStr);

instrumentPayCcy = instrumentJson.CcyCd.Cd;

globalDict('payCcy') = instrumentPayCcy;

components = instrumentJson.Components;

valGroup = strcat(components.ProdTpNm,'_',components.TmpltNm);

globalDict('valGroup') = valGroup;

%
if strcmp(valGroup,'InterestRateDerivatives_SimpleRA')
   %% SimpleRA Info 
    %security Info : startDate,endDate,nominal,payCcy from instrument.Components
    % construct rangeAccrualParams
    
    rangeAccrualParams.payCcy = globalDict('payCcy');
    rangeAccrualParams.startDate = H_Date(components.EffDt);
    rangeAccrualParams.endDate = H_Date(components.EndDt);
    
    rangeAccrualInfo = containers.Map();

    rangeAccrualInfo('nominal') = components.UnitAmt;
    
    %% payoff info start from instrument.Components.Payoffs
    payoffs = components.Payoffs;
 
    rangeAccrualInfo('coupon') = payoffs.FixedIr * 100.0;  % input mistake 0.03%
    
    rangeAccrualInfo('upper') = 0.0;
    if isfield(payoffs,'Cap1')
        rangeAccrualInfo('upper') = payoffs.Cap1;
    end

    rangeAccrualInfo('lower') = 0.0;
    if isfield(payoffs,'Floor1')
        rangeAccrualInfo('lower') = payoffs.Floor1;
    end

    
%% payoff info end from instrument.Components.Payoffs
    
   %% Underlying Info from instrumentJson.Components.Underlyings
    % getUnderlyings
    underlyings = components.Underlyings;
    underlyingsInfo = struct;
    for i=1:length(underlyings)
        underlyingsInfo(i).name = underlyings(i).UnderlyingId;
        underlyingsInfo(i).type = underlyings(i).Instrument.InsTpCd.Cd;
        underlyingsInfo(i).ccy  = underlyings(i).Instrument.CcyCd.Cd; 
    end
    
    % underlyingsInfo : underlying tenor Info
    if strcmp(underlyingsInfo(1).name,'CD91D')
        rangeAccrualInfo('tenor') = 0.25;
    end
    
    % GetUnderlyingObject
    if strcmp(underlyingsInfo(1).type,'RATE_INDEX')
            %get mapped fwd yld\
            asset = struct;
            asset.name = underlyingsInfo(1).ccy;
            asset.ccy = asset.name;
            asset.type = 'IR';
            
            %marketData init
            
            asset.allMarketData = containers.Map();
            
            % if referenceModel (discounting) then add mappedDiscountYLDCurve
            if strcmp(asset.ccy,globalDict('payCcy'))
                %discountYLDCurve
                discountYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                    asset.ccy,'&mapTrgtTpCd=DISCOUNT');
                mappedYldJson = webread(discountYldQueryStr);
                yldCurveName = mappedYldJson.Items.MapTrgtNm;
                            bootStrapFlag = true;
%             bootStrapFlag = false;
                ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);

                asset.allMarketData('discountYLD') = ycData;
                asset.allMarketData('discountCurve') = ycData.zcCurve.zeroCurve;
                
                globalDict('refModelName') = asset.ccy;
            end
            
%             modelSettings = globalDict('modelSettings'); 
%             modelParams = modelSettings(pricingModelName);
%             construct = str2func(pricingModelName);
%             pricingModel = construct(modelParams);
%             KRWLGM2F = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));
%             if strcmp(pricingModel,'CK1F')
%                 aaa = 1.0;
%             end
            
            %fwdYLD
            fwdYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                underlyingsInfo(1).name,'&mapTrgtTpCd=RISKFREE');
            mappedYldJson = webread(fwdYldQueryStr);
            yldCurveName = mappedYldJson.Items.MapTrgtNm;
            bootStrapFlag = true;
%             bootStrapFlag = false;
            ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);
            
            asset.allMarketData('fwdYLD') = ycData;
            asset.allMarketData('zeroCurve') = ycData.zcCurve.zeroCurve;
            %discountYLD
            
            %get mapped swaption volatility
            swaptionVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                underlyingsInfo(1).ccy,'&mapTrgtTpCd=SWAPTVOL');
            mappedVolJson = webread(swaptionVolQueryStr);
            
%             swaptionVolName = mappedVolJson.Items.MapTrgtId;
            swaptionVolName = mappedVolJson.Items.MapTrgtNm;

            swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,swaptionVolName);
            rawDataSwaption = swaptionVolData.rawData;
            
            asset.allMarketData('swaptionVolRaw') = rawDataSwaption;
            
            mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate, underlyingsInfo(1).ccy,strcat(underlyingsInfo(1).ccy,'SwaptionVol')});
            swaptionVolMktData = MktData(mktData);
            asset.allMarketData('swaptionVol') = swaptionVolMktData;
            
            %modelInit
            modelNameMap = globalDict('modelNameMap');
            asset.modelName = modelNameMap(asset.name);
            
            if strcmp(asset.modelName,'CK1F')
                % construct ck1F model
                modelSettings = globalDict('modelSettings');
                ck1FModelParams = modelSettings('CK1F');
                
                % added modelParams
                
                ck1FModelParams('instrumentStartDate') = rangeAccrualParams.startDate;
                ck1FModelParams('instrumentEndDate') = rangeAccrualParams.endDate;
                ck1FModelParams('calibrationTargetTenor') =  1:1:14;
                
                ck1F = CK1F(IRModel(Model(asset.allMarketData,ck1FModelParams)));
                
                % calibration
                % construct black model 
                blackModelParams= containers.Map({'modelName'},{'Black'});
                black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 
                
                
                calibOut = ck1F.CalibrateToATMDiagSwaptionForward(black,'global');
                
                vol_r_Calib = ck1F.vol_r;
                
                modelParamsCalib= ck1FModelParams;
                modelParamsCalib('vol_r') = vol_r_Calib;

                ck1FRisky = CK1FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));
                
                assetModelMap = globalDict('assetModelMap');
                assetModelMap(asset.name) = ck1FRisky;
                
                % map is a handle class so you don't need this step
%                 globalDict('assetModelMap') = assetModelMap;
                
            end
            
    end
    
    % instrument info : rnageAccrualLeg, (structured leg , callable leg
    % schedule
    items = instrumentJson.Components.Items;
    % find structured leg
    structuredLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemSubTpNm,'structured')
            structuredLeg = items{idx};
            break;
        end
    end
    
    rangeAccrualInfo('couponFreq') = structuredLeg.ExerFreqCd.Cd;
    
    % generated Schedule
    structuredSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
                                    rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Backward',false);
    % manual Schedule
    structuredScheduleRaw = structuredLeg.Cashflows;
    %reset start end pay FngDate format array
    couponScheduleInt = zeros(length(structuredLeg.Cashflows),4);
    for idx =1 :length(structuredScheduleRaw)
        couponScheduleInt(idx,1) = FngDate(H_Date(structuredScheduleRaw(idx).ResetDt));
        couponScheduleInt(idx,2) = FngDate(H_Date(structuredScheduleRaw(idx).BegDt));
        couponScheduleInt(idx,3) = FngDate(H_Date(structuredScheduleRaw(idx).EndDt));
        couponScheduleInt(idx,4) = FngDate(H_Date(structuredScheduleRaw(idx).PayDt));
        
    end
    
    rangeAccrualInfo('couponScheduleInt') = couponScheduleInt;
    
    % find callable leg
    callalbleLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemTpNm,'callable')
            callalbleLeg = items{idx};
            break;
        end
    end
    
    % generated Schedule
%     callableSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
%                                     rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
%                                     'ModifiedFollowing', 'Backward',false);

    % should be fixed to use callable schedule directly !
    nonCallTermYear = callalbleLeg.NonCallTermYear;
    
    callYN = ones(size(couponScheduleInt,1),1);
    %only yearly schedule
    for i=1:length(callYN)
        if i <= nonCallTermYear
            callYN(i) = 0;
        end
    end
    
    rangeAccrualInfo('callYN') = callYN;
    
    % manual Schedule
    callalbleScheduleRaw = callalbleLeg.ExerciseEvents;
    %reset start end pay FngDate format array
    callScheduleInt  = zeros(length(callalbleLeg.ExerciseEvents),4);
    for idx = 1:length(callalbleScheduleRaw)
        callScheduleInt(idx,1) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,2) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,3) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
        callScheduleInt(idx,4) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
        
    end
    
    callScheduleInt = [20181012	20181012	20191014	20191014
20191009	20191014	20201012	20201012
20201007	20201012	20211012	20211012
20211007	20211012	20221012	20221012
20221007	20221012	20231012	20231012
20231009	20231012	20241014	20241014
20241009	20241014	20251013	20251013
20251008	20251013	20261012	20261012
20261007	20261012	20271012	20271012
20271007	20271012	20281012	20281012
20281009	20281012	20291012	20291012
20291008	20291012	20301014	20301014
20301009	20301014	20311013	20311013
20311008	20311013	20321012	20321012
20321007	20321012	20331012	20331012
];
    rangeAccrualInfo('callScheduleInt') = callScheduleInt;
    
    
    % fixings to do later
    
    rangeDays = zeros(size(couponScheduleInt,1),1);
    %rangeDays(1) = 1;
    rangeDays(1) = DateDiff(valueDate,rangeAccrualParams.startDate) + 1;
    
    rangeAccrualInfo('rangeDays') = rangeDays;
    
    rangeAccrualInfo('manualScheduleYN') = 1;
%     rangeAccrualInfo('callStrike') = rangeAccrualInfo('nominal');
    rangeAccrualInfo('callStrike') = 1.0;


%     rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenor','lower','upper','rangeDays','callYN', ...
%                                 'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
%                         {nominal,coupon,couponFreq,tenor,lower,upper,rangeDays,callYN,...
%                        couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;              

    rangeAccrual = CDRangeAccrual(rangeAccrualParams);

    globalDict('security') = rangeAccrual;
    

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % weekly step calculator
    % pricer = Pricer(rangeAccrual,USDLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,USDLGMRisky,numMethodAMC,true);
    assetModelMap = globalDict('assetModelMap');
    
    pricer = DailyWeeklyRiskyPricer_CDRA(globalDict('security'),assetModelMap(globalDict('refModelName')),numMethodAMC,false);
    pricingOutMC = pricer.computePriceMCCallable(valueDate);
    pricingOutClosed = pricer.computePriceClosedSimple(valueDate);
    
    if idxSimul==1
        SimulInfo.nonCallNPVs{1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    else
        SimulInfo.nonCallNPVs{end+1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{end+1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{end+1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    end
    
    disp(['nonCallNPV;  ',num2str(pricingOutMC.nonCall.npv)])
    disp(['callableNPV;  ',num2str(pricingOutMC.callable.npv)])
    disp(['nonCallClosedNPV;  ',num2str(pricingOutClosed.pricerInfo.nonCall.npv)])
end

end
aaa = 1.0;




In [ ]:
%hss_Octagon_CallableCDRA_test.m

clc
clear

%CDRA instrument
% product = containers.Map({'insId','context',},{'F181015-00005','MtM'});

% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F160517-34710','MtM'});
product = containers.Map({'insId','context',},{'F180802-78418','MtM'});


% 
% %CDCDKTBDualRA
% product = containers.Map({'insId','context',},{'F170216-45342','MtM'});
% 

valueDate = H_Date(20181015);
valueDateRaw = H_Date(20181015);
SimulInfo = struct;
SimulInfo.valueDates = {};
SimulInfo.nonCallNPVs = {};
SimulInfo.nonCallClosedNPVs = {};
SimulInfo.callableNPVs = {};

% global dictionary init
globalDict = containers.Map();

%% static data
% default calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;

% static Setting for globalDict
%yldCurveSetting

yiledCurveSettings = containers.Map();

%KRW_IRS Setting

yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;

%modelSettings
modelSettings = containers.Map();

%CK1F
ck1FModelParams = containers.Map();
vol_r = struct;
vol_r.quote = [0.01];
vol_r.tenor = [1.0];
ck1FModelParams('vol_r') = vol_r;
ck1FModelParams('meanR_r') = 0.03;
ck1FModelParams('joint_swap_tenor') = 1.0;
ck1FModelParams('modelName') = 'CK1F';
ck1FModelParams('targetSize') = 15;
ck1FModelParams('freq') = 4;
ck1FModelParams('matu') = 15.0;

modelSettings('CK1F') = ck1FModelParams;
%LGM2F

%BS
globalDict('modelSettings') = modelSettings;

%modelNameMap

modelNameMap =  containers.Map();
modelNameMap('KRW') = 'CK1F';


globalDict('modelNameMap') = modelNameMap;

%assetModelMap
assetModelMap =  containers.Map();
assetModelMap('KRW') = struct;
globalDict('assetModelMap')=assetModelMap;



for idxSimul=1:10
    
valueDate = jointCalendar.Advance(valueDateRaw,'Following',idxSimul-1,'day',false);
% valueDate = valueDateRaw.AddDate(idxSimul-1,'day');

if idxSimul==1
    SimulInfo.valueDates{1,1} = FngDate(valueDate);
else
    SimulInfo.valueDates{end+1,1} = FngDate(valueDate);
end


% get Instrument info

% instrumentQueryStr = 'http://45.249.2.10:50100/data/instruments/';
instrumentQueryStr = 'http://45.12.71.111:50100/data/instruments/';


queryStr = strcat(instrumentQueryStr,product('insId'));
instrumentJson = webread(queryStr);

instrumentPayCcy = instrumentJson.CcyCd.Cd;

globalDict('payCcy') = instrumentPayCcy;

components = instrumentJson.Components;

valGroup = strcat(components.ProdTpNm,'_',components.TmpltNm);

globalDict('valGroup') = valGroup;

%
if strcmp(valGroup,'InterestRateDerivatives_SimpleRA')
   %% SimpleRA Info 
    %security Info : startDate,endDate,nominal,payCcy from instrument.Components
    % construct rangeAccrualParams
    
    rangeAccrualParams.payCcy = globalDict('payCcy');
    rangeAccrualParams.startDate = H_Date(components.EffDt);
    rangeAccrualParams.endDate = H_Date(components.EndDt);
    
    rangeAccrualInfo = containers.Map();

    rangeAccrualInfo('nominal') = components.UnitAmt;
    
    %% payoff info start from instrument.Components.Payoffs
    payoffs = components.Payoffs;
 
    rangeAccrualInfo('coupon') = payoffs.FixedIr * 100.0;  % input mistake 0.03%
    
    rangeAccrualInfo('upper') = 0.0;
    if isfield(payoffs,'Cap1')
        rangeAccrualInfo('upper') = payoffs.Cap1;
    end

    rangeAccrualInfo('lower') = 0.0;
    if isfield(payoffs,'Floor1')
        rangeAccrualInfo('lower') = payoffs.Floor1;
    end

    
%% payoff info end from instrument.Components.Payoffs
    
   %% Underlying Info from instrumentJson.Components.Underlyings
    % getUnderlyings
    underlyings = components.Underlyings;
    underlyingsInfo = struct;
    for i=1:length(underlyings)
        underlyingsInfo(i).name = underlyings(i).UnderlyingId;
        underlyingsInfo(i).type = underlyings(i).Instrument.InsTpCd.Cd;
        underlyingsInfo(i).ccy  = underlyings(i).Instrument.CcyCd.Cd; 
    end
    
    % underlyingsInfo : underlying tenor Info
    if strcmp(underlyingsInfo(1).name,'CD91D')
        rangeAccrualInfo('tenor') = 0.25;
    end
    
    % GetUnderlyingObject
    if strcmp(underlyingsInfo(1).type,'RATE_INDEX')
            %get mapped fwd yld\
            asset = struct;
            asset.name = underlyingsInfo(1).ccy;
            asset.ccy = asset.name;
            asset.type = 'IR';
            
            %marketData init
            
            asset.allMarketData = containers.Map();
            
            % if referenceModel (discounting) then add mappedDiscountYLDCurve
            if strcmp(asset.ccy,globalDict('payCcy'))
                %discountYLDCurve
                discountYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                    asset.ccy,'&mapTrgtTpCd=DISCOUNT');
                mappedYldJson = webread(discountYldQueryStr);
                yldCurveName = mappedYldJson.Items.MapTrgtNm;
                            bootStrapFlag = true;
%             bootStrapFlag = false;
                ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);

                asset.allMarketData('discountYLD') = ycData;
                asset.allMarketData('discountCurve') = ycData.zcCurve.zeroCurve;
                
                globalDict('refModelName') = asset.ccy;
            end
            
%             modelSettings = globalDict('modelSettings'); 
%             modelParams = modelSettings(pricingModelName);
%             construct = str2func(pricingModelName);
%             pricingModel = construct(modelParams);
%             KRWLGM2F = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));
%             if strcmp(pricingModel,'CK1F')
%                 aaa = 1.0;
%             end
            
            %fwdYLD
            fwdYldQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                underlyingsInfo(1).name,'&mapTrgtTpCd=RISKFREE');
            mappedYldJson = webread(fwdYldQueryStr);
            yldCurveName = mappedYldJson.Items.MapTrgtNm;
            bootStrapFlag = true;
%             bootStrapFlag = false;
            ycData = GetYieldCurveFromOctagon(valueDate,yldCurveName,globalDict,bootStrapFlag);
            
            asset.allMarketData('fwdYLD') = ycData;
            asset.allMarketData('zeroCurve') = ycData.zcCurve.zeroCurve;
            %discountYLD
            
            %get mapped swaption volatility
            swaptionVolQueryStr = strcat('http://45.249.2.36:50100/data/contexts/MtM?paramId=', ...
                                underlyingsInfo(1).ccy,'&mapTrgtTpCd=SWAPTVOL');
            mappedVolJson = webread(swaptionVolQueryStr);
            
%             swaptionVolName = mappedVolJson.Items.MapTrgtId;
            swaptionVolName = mappedVolJson.Items.MapTrgtNm;

            swaptionVolData = GetSwaptionVolSurfaceFromOctagon(valueDate,swaptionVolName);
            rawDataSwaption = swaptionVolData.rawData;
            
            asset.allMarketData('swaptionVolRaw') = rawDataSwaption;
            
            mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate, underlyingsInfo(1).ccy,strcat(underlyingsInfo(1).ccy,'SwaptionVol')});
            swaptionVolMktData = MktData(mktData);
            asset.allMarketData('swaptionVol') = swaptionVolMktData;
            
            %modelInit
            modelNameMap = globalDict('modelNameMap');
            asset.modelName = modelNameMap(asset.name);
            
            if strcmp(asset.modelName,'CK1F')
                % construct ck1F model
                modelSettings = globalDict('modelSettings');
                ck1FModelParams = modelSettings('CK1F');
                
                % added modelParams
                
                ck1FModelParams('instrumentStartDate') = rangeAccrualParams.startDate;
                ck1FModelParams('instrumentEndDate') = rangeAccrualParams.endDate;
                ck1FModelParams('calibrationTargetTenor') =  1:1:14;
                
                ck1F = CK1F(IRModel(Model(asset.allMarketData,ck1FModelParams)));
                
                % calibration
                % construct black model 
                blackModelParams= containers.Map({'modelName'},{'Black'});
                black = IRBlack(IRModel(Model(asset.allMarketData,blackModelParams))); 
                
                
                calibOut = ck1F.CalibrateToATMDiagSwaptionForward(black,'global');
                
                vol_r_Calib = ck1F.vol_r;
                
                modelParamsCalib= ck1FModelParams;
                modelParamsCalib('vol_r') = vol_r_Calib;

                ck1FRisky = CK1FRisky(IRModelRisky(Model(asset.allMarketData,modelParamsCalib)));
                
                assetModelMap = globalDict('assetModelMap');
                assetModelMap(asset.name) = ck1FRisky;
                
                % map is a handle class so you don't need this step
%                 globalDict('assetModelMap') = assetModelMap;
                
            end
            
    end
    
    % instrument info : rnageAccrualLeg, (structured leg , callable leg
    % schedule
    items = instrumentJson.Components.Items;
    % find structured leg
    structuredLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemSubTpNm,'structured')
            structuredLeg = items{idx};
            break;
        end
    end
    
    rangeAccrualInfo('couponFreq') = structuredLeg.ExerFreqCd.Cd;
    
    % generated Schedule
    structuredSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
                                    rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
                                    'ModifiedFollowing', 'Backward',false);
    % manual Schedule
    structuredScheduleRaw = structuredLeg.Cashflows;
    %reset start end pay FngDate format array
    couponScheduleInt = zeros(length(structuredLeg.Cashflows),4);
    for idx =1 :length(structuredScheduleRaw)
        couponScheduleInt(idx,1) = FngDate(H_Date(structuredScheduleRaw(idx).ResetDt));
        couponScheduleInt(idx,2) = FngDate(H_Date(structuredScheduleRaw(idx).BegDt));
        couponScheduleInt(idx,3) = FngDate(H_Date(structuredScheduleRaw(idx).EndDt));
        couponScheduleInt(idx,4) = FngDate(H_Date(structuredScheduleRaw(idx).PayDt));
        
    end
    
    rangeAccrualInfo('couponScheduleInt') = couponScheduleInt;
    
    % find callable leg
    callalbleLeg = struct;
    for idx=1:length(items)
        if strcmp(items{idx}.CompoItemTpNm,'callable')
            callalbleLeg = items{idx};
            break;
        end
    end
    
    % generated Schedule
%     callableSchedule0 = Schedule(rangeAccrualParams.startDate,rangeAccrualParams.endDate,...
%                                     rangeAccrualInfo('couponFreq') ,globalDict('jointCalendar'),...
%                                     'ModifiedFollowing', 'Backward',false);

    % should be fixed to use callable schedule directly !
    nonCallTermYear = callalbleLeg.NonCallTermYear;
    
    callYN = ones(size(couponScheduleInt,1),1);
    %only yearly schedule
    for i=1:length(callYN)
        if i <= nonCallTermYear
            callYN(i) = 0;
        end
    end
    
    rangeAccrualInfo('callYN') = callYN;
    
    % manual Schedule
    callalbleScheduleRaw = callalbleLeg.ExerciseEvents;
    %reset start end pay FngDate format array
    callScheduleInt  = zeros(length(callalbleLeg.ExerciseEvents),4);
    for idx = 1:length(callalbleScheduleRaw)
        callScheduleInt(idx,1) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,2) = FngDate(H_Date(callalbleScheduleRaw(idx).ExerDt));
        callScheduleInt(idx,3) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
        callScheduleInt(idx,4) = FngDate(H_Date(callalbleScheduleRaw(idx).PayDt));
        
    end
    
    callScheduleInt = [20181012	20181012	20191014	20191014
20191009	20191014	20201012	20201012
20201007	20201012	20211012	20211012
20211007	20211012	20221012	20221012
20221007	20221012	20231012	20231012
20231009	20231012	20241014	20241014
20241009	20241014	20251013	20251013
20251008	20251013	20261012	20261012
20261007	20261012	20271012	20271012
20271007	20271012	20281012	20281012
20281009	20281012	20291012	20291012
20291008	20291012	20301014	20301014
20301009	20301014	20311013	20311013
20311008	20311013	20321012	20321012
20321007	20321012	20331012	20331012
];
    rangeAccrualInfo('callScheduleInt') = callScheduleInt;
    
    
    % fixings to do later
    
    rangeDays = zeros(size(couponScheduleInt,1),1);
    %rangeDays(1) = 1;
    rangeDays(1) = DateDiff(valueDate,rangeAccrualParams.startDate) + 1;
    
    rangeAccrualInfo('rangeDays') = rangeDays;
    
    rangeAccrualInfo('manualScheduleYN') = 1;
%     rangeAccrualInfo('callStrike') = rangeAccrualInfo('nominal');
    rangeAccrualInfo('callStrike') = 1.0;


%     rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenor','lower','upper','rangeDays','callYN', ...
%                                 'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
%                         {nominal,coupon,couponFreq,tenor,lower,upper,rangeDays,callYN,...
%                        couponScheduleInt,callScheduleInt,manualScheduleYN});

    rangeAccrualParams.params = rangeAccrualInfo;              

    rangeAccrual = CDRangeAccrual(rangeAccrualParams);

    globalDict('security') = rangeAccrual;
    

    numOfRandom = 40000;    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    % weekly step calculator
    % pricer = Pricer(rangeAccrual,USDLGM,numMethodAMC);
    % pricingOut = pricer.computePrice(valueDate);

    % daily weekly step calculator
    % pricer = DailyWeeklyRiskyPricer(rangeAccrual,USDLGMRisky,numMethodAMC,true);
    assetModelMap = globalDict('assetModelMap');
    
    pricer = DailyWeeklyRiskyPricer_CDRA(globalDict('security'),assetModelMap(globalDict('refModelName')),numMethodAMC,false);
    pricingOutMC = pricer.computePriceMCCallable(valueDate);
    pricingOutClosed = pricer.computePriceClosedSimple(valueDate);
    
    if idxSimul==1
        SimulInfo.nonCallNPVs{1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    else
        SimulInfo.nonCallNPVs{end+1,1} = pricingOutMC.nonCall.npv;
        SimulInfo.nonCallClosedNPVs{end+1,1} = pricingOutMC.callable.npv;
        SimulInfo.callableNPVs{end+1,1} = pricingOutClosed.pricerInfo.nonCall.npv;
    end
    
    disp(['nonCallNPV;  ',num2str(pricingOutMC.nonCall.npv)])
    disp(['callableNPV;  ',num2str(pricingOutMC.callable.npv)])
    disp(['nonCallClosedNPV;  ',num2str(pricingOutClosed.pricerInfo.nonCall.npv)])
end

end
aaa = 1.0;




In [ ]:
% hss_api_YieldCurveExe_inputTest_KTB.m

clc
clear

nowDate = '2018-05-10';
%% get calendars 
nowDateH = H_Date(nowDate);
fromDateStr = int2str(FngDate(nowDateH));
endDateStr = int2str(FngDate(AddDate(nowDateH,30,'y')));

calendarName1 = 'KRbk';
calendarName2 = 'KRnt';

calendar1 = getCalendar(calendarName1,fromDateStr,endDateStr);
calendar2 = getCalendar(calendarName2,fromDateStr,endDateStr);

aaa = 1.0;

DataNm = 'KRW_KTB';
curveType = 'market';
yieldCurveMarketQueryStr1 = 'http://45.249.2.10:50100/data/market/yieldcurves/';

queryStr = strcat(yieldCurveMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);
dataJson = webread(queryStr);

instruments = cell(length(dataJson.Yields),3);
for i=1:length(dataJson.Yields)
    sizeStr = int2str(cast(dataJson.Yields(i).Tenor.Value,'int32'));
    periodTypeStr = dataJson.Yields(i).Tenor.TenorCd.Cd;
    instruments{i,1} = strcat(sizeStr,periodTypeStr);
    instruments{i,2} = dataJson.Yields(i).YcInsTpNm;
    instruments{i,3} = dataJson.Yields(i).Rate;
    
end

% yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves?name=KRW_IRS';
yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves';

queryStr = strcat(yieldCurveMasterStr,'?name=',DataNm);
dataJson = webread(queryStr);

aaa =1.0;
function out = getCalendar(calendarName,fromDateStr,endDateStr)
    calendarQueryStr1 = 'http://45.249.2.10:50100/data/common/calendars/';
    calendarQueryStr2 = '?fromDt=';
    calendarQueryStr3 = '&toDt=';
    queryStr = strcat(calendarQueryStr1,calendarName,calendarQueryStr2,fromDateStr,calendarQueryStr3,endDateStr);
    dataJson = webread(queryStr);

    holidaysSize = length(dataJson.HoliDts);
    holidays = cell(holidaysSize,1);
    for i=1:holidaysSize
        holidays{i} = StrDate(H_Date(str2num(dataJson.HoliDts{i})),'str');
    end
    out.calendarName = calendarName;
    out.holidays = holidays;
    
end




In [ ]:
% hss_api_YieldCurveExe_inputTest.m

clc
clear

nowDate = '2017-10-01';
toDate = '2018-10-01';
% %% get calendars 
% nowDateH = H_Date(nowDate);
% fromDateStr = int2str(FngDate(nowDateH));
% toDateStr = int2str(FngDate(H_Date(toDate)));
% endDateStr = int2str(FngDate(AddDate(nowDateH,30,'y')));
% 
% fromDateQueryStr2 = '?fromDt=';
% endDateQueryStr3 = '&toDt=';
%     
% calendarName1 = 'KRbk';
% calendarName2 = 'KRnt';
% calendarName2 = 'KREX';
% 
% 
% calendar1 = getCalendar(calendarName1,fromDateStr,endDateStr);
% calendar2 = getCalendar(calendarName2,fromDateStr,endDateStr);
% 
% aaa = 1.0;
% 
% DataNm = 'KRW_IRS';
% % DataNm = 'KRW_KTB';
% DataNm = '032115';
% DataNm = '032111';
% 
% curveType = 'market';
% yieldCurveMarketQueryStr1 = 'http://45.249.2.10:50100/data/market/yieldcurves/';
% 
% 
% % queryStr = strcat(yieldCurveMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);
% % queryStr = strcat(yieldCurveMarketQueryStr1,DataNm,'/',fromDateStr,'-',toDateStr,'?type=',curveType);
% queryStr = strcat(yieldCurveMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);

% queryStr = 'http://45.249.2.10:50100/data/instruments/yieldcurves/';
queryStr = 'http://45.12.71.111:50100/data/market/yieldcurves/KRW_IRS/20201218';


options = weboptions('TimeOut',30);
dataJson = webread(queryStr,options);

instruments = cell(length(dataJson.Yields),3);
for i=1:length(dataJson.Yields)
    sizeStr = int2str(cast(dataJson.Yields(i).Tenor.Value,'int32'));
    periodTypeStr = dataJson.Yields(i).Tenor.TenorCd.Cd;
    instruments{i,1} = strcat(sizeStr,periodTypeStr);
    instruments{i,2} = dataJson.Yields(i).YcInsTpNm;
    instruments{i,3} = dataJson.Yields(i).Rate;
    
end

% yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves?name=KRW_IRS';
yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves';

queryStr = strcat(yieldCurveMasterStr,'?name=',DataNm);
dataJson = webread(queryStr);

aaa =1.0;
function out = getCalendar(calendarName,fromDateStr,endDateStr)
    calendarQueryStr1 = 'http://45.249.2.10:50100/data/common/calendars/';
    calendarQueryStr2 = '?fromDt=';
    calendarQueryStr3 = '&toDt=';
    queryStr = strcat(calendarQueryStr1,calendarName,calendarQueryStr2,fromDateStr,calendarQueryStr3,endDateStr);
    dataJson = webread(queryStr);

    holidaysSize = length(dataJson.HoliDts);
    holidays = cell(holidaysSize,1);
    for i=1:holidaysSize
        holidays{i} = StrDate(H_Date(str2num(dataJson.HoliDts{i})),'str');
    end
    out.calendarName = calendarName;
    out.holidays = holidays;
    
end




In [ ]:
% hss_api_SwaptionVolSurface_inputTest.m

clc
clear

nowDate = '2018-09-21';
%% get calendars 
nowDateH = H_Date(nowDate);
fromDateStr = int2str(FngDate(nowDateH));
endDateStr = int2str(FngDate(AddDate(nowDateH,30,'y')));

calendarName1 = 'KRbk';
calendarName2 = 'KRnt';

calendar1 = getCalendar(calendarName1,fromDateStr,endDateStr);
calendar2 = getCalendar(calendarName2,fromDateStr,endDateStr);

aaa = 1.0;

% DataNm = 'KRW_IRS';
DataNm = '032111_SWAPT';
% DataNm = 'KRW_KTB';

curveType = 'market';
% yieldCurveMarketQueryStr1 = 'http://45.249.2.10:50100/data/market/yieldcurves/';
% 'http://45.249.2.10:50100/data/market/vols/032111_SWAPT/20180910'
volSurfaceMarketQueryStr1 = 'http://45.249.2.10:50100/data/market/vols/';
% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/KRS000045062';


queryStr = strcat(volSurfaceMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);
dataJson = webread(queryStr);

instruments = cell(length(dataJson.Yields),3);
for i=1:length(dataJson.Yields)
    sizeStr = int2str(cast(dataJson.Yields(i).Tenor.Value,'int32'));
    periodTypeStr = dataJson.Yields(i).Tenor.TenorCd.Cd;
    instruments{i,1} = strcat(sizeStr,periodTypeStr);
    instruments{i,2} = dataJson.Yields(i).YcInsTpNm;
    instruments{i,3} = dataJson.Yields(i).Rate;
    
end

% yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves?name=KRW_IRS';
yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves';

queryStr = strcat(yieldCurveMasterStr,'?name=',DataNm);
dataJson = webread(queryStr);

aaa =1.0;
function out = getCalendar(calendarName,fromDateStr,endDateStr)
    calendarQueryStr1 = 'http://45.249.2.10:50100/data/common/calendars/';
    calendarQueryStr2 = '?fromDt=';
    calendarQueryStr3 = '&toDt=';
    queryStr = strcat(calendarQueryStr1,calendarName,calendarQueryStr2,fromDateStr,calendarQueryStr3,endDateStr);
    dataJson = webread(queryStr);

    holidaysSize = length(dataJson.HoliDts);
    holidays = cell(holidaysSize,1);
    for i=1:holidaysSize
        holidays{i} = StrDate(H_Date(str2num(dataJson.HoliDts{i})),'str');
    end
    out.calendarName = calendarName;
    out.holidays = holidays;
    
end




In [ ]:
% hss_api_instrument_inputTest.m

clc
clear

nowDate = '2018-09-21';
%% get calendars 
nowDateH = H_Date(nowDate);
fromDateStr = int2str(FngDate(nowDateH));
endDateStr = int2str(FngDate(AddDate(nowDateH,30,'y')));

calendarName1 = 'KRbk';
calendarName2 = 'KRnt';

calendar1 = getCalendar(calendarName1,fromDateStr,endDateStr);
calendar2 = getCalendar(calendarName2,fromDateStr,endDateStr);

aaa = 1.0;

% DataNm = 'KRW_IRS';
DataNm = '032111_SWAPT';
% DataNm = 'KRW_KTB';

curveType = 'market';
% yieldCurveMarketQueryStr1 = 'http://45.249.2.10:50100/data/market/yieldcurves/';
% 'http://45.249.2.10:50100/data/market/vols/032111_SWAPT/20180910'
% volSurfaceMarketQueryStr1 = 'http://45.249.2.10:50100/data/market/vols/';
% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/KRS000045062';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/KRS000067175';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/contexts/MtM';

% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/T181015-00001';

% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F160707-36623';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/G181012-00064';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F181015-00002';


instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/G181010-00582';

% my CDRA
instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F181015-00005';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/contexts/MtM';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F181025-00006';
instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F170208-45062';

instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments//F181029-00006';

% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/contexts/MtM';
dataJson1 = webread(instrumentQueryStr1);

instrumentQueryStr1 = 'http://45.249.2.36:50100/data/market/prices/USDIRS10Y/20181001-20181122';
dataJson1 = webread(instrumentQueryStr1);
instrumentQueryStr2 = 'http://45.249.2.36:50100/data/market/prices/KRWIRS10Y/20181001-20181122';
dataJson2 = webread(instrumentQueryStr2);
instrumentQueryStr3 = 'http://45.249.2.36:50100/data/market/prices/KRWIRS02Y/20181001-20181122';

instrumentQueryStr3 = 'http://45.249.2.36:50100/data/market/vols/0321L3_OCT_COMD/20190731';

dataJson3 = webread(instrumentQueryStr3);
% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/market/vols/FXKRWUSD_TV/20181022';

% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F181025-00001';
% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F181024-00013';
%Quanto CMS & CMS Spread Dual RA

% instrumentQueryStr1 = 'http://45.249.2.10:50100/data/instruments/F181016-00001';

% queryStr = strcat(volSurfaceMarketQueryStr1,DataNm,'/',fromDateStr,'?type=',curveType);

% dataJson = webread(queryStr);
dataJson = webread(instrumentQueryStr1);

instruments = cell(length(dataJson.Yields),3);
for i=1:length(dataJson.Yields)
    sizeStr = int2str(cast(dataJson.Yields(i).Tenor.Value,'int32'));
    periodTypeStr = dataJson.Yields(i).Tenor.TenorCd.Cd;
    instruments{i,1} = strcat(sizeStr,periodTypeStr);
    instruments{i,2} = dataJson.Yields(i).YcInsTpNm;
    instruments{i,3} = dataJson.Yields(i).Rate;
    
end

% yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves?name=KRW_IRS';
yieldCurveMasterStr = 'http://45.249.2.10:50100/data/market/yieldcurves';

queryStr = strcat(yieldCurveMasterStr,'?name=',DataNm);
dataJson = webread(queryStr);

aaa =1.0;
function out = getCalendar(calendarName,fromDateStr,endDateStr)
    calendarQueryStr1 = 'http://45.249.2.10:50100/data/common/calendars/';
    calendarQueryStr2 = '?fromDt=';
    calendarQueryStr3 = '&toDt=';
    queryStr = strcat(calendarQueryStr1,calendarName,calendarQueryStr2,fromDateStr,calendarQueryStr3,endDateStr);
    dataJson = webread(queryStr);

    holidaysSize = length(dataJson.HoliDts);
    holidays = cell(holidaysSize,1);
    for i=1:holidaysSize
        holidays{i} = StrDate(H_Date(str2num(dataJson.HoliDts{i})),'str');
    end
    out.calendarName = calendarName;
    out.holidays = holidays;
    
end




In [ ]:
% Downloadable MATLAB code, copyright Amritanshu Palaria
% H2 ground state from a non-relativistic Hartree Fock treatment using STO-4G basis set and Born Oppenheimer approximation  
% Program written by Amritanshu Palaria @ NCN, Apr 12, 2006
% For theoretical reference, please see "Computational Physics" by Thijssen
clear all;
% initialization
Z=1,S=1,A=0,F=0,V=0,D=0,g=0,Vecp=0,Vec=0,Val=0,EVal=0;
for y=1:41
    clear T,S,A,F,V,D,g,Vecp,Vec,Val,EVal;
    % y = 11;
    disp('Step '); disp(y);
    r = 0.45+0.05*y; % distance between nucleii in a.u.(units of a0, ie. Bohr radius)... since the problem is essentially 1-D, this is all that is needed
    disp('H-H bondlength'); disp(r);
    NNr=1/r;
    % alphas for the 4 Gaussian basis functions for each H atom
    alpha1 = [13.00773 1.962079 0.444529 0.1219492]; 
    alpha = [alpha1 alpha1];
    asize=size(alpha);
    % one H nucleus at origin, the other at r
    ra=[0 0 0 0 r r r r]; % the location of the atom with the ith basisfunction
    % first 4 basis fns come from the atom at origin
    % the next 4 come from the one at r
    % the S (overlap), T (K.E.) and A (Coulomb) matrices
    for i=1:8 % run over all basis functions
        for j=1:i % do not run over all basis functions to exploit symmetry
            % intermediate variables
            as = alpha(i)+alpha(j);
            ap = alpha(i)*alpha(j);
            rat=ap/as;
            % location of the gaussian resulting from product of
            % the 2 gaussian basis functions
            rp=(alpha(i)*ra(i)+alpha(j)*ra(j))/as;
            S(i,j)=(pi/as)^1.5*exp(-rat*(ra(i)-ra(j))^2);
            S(j,i)=S(i,j); % using symmetry
            T(i,j)=0.5*rat*(6-4*rat*(ra(i)-ra(j))^2)*S(i,j);
            T(j,i)=T(i,j); % using symmetry
            if (rp==0)
                F0=1+sqrt(pi)/2*erf(sqrt(as*(rp-r)^2))/sqrt(as*(rp-r)^2);
            elseif (rp==r)
                F0=sqrt(pi)/2*erf(sqrt(as*(rp-0)^2))/sqrt(as*(rp-0)^2)+1;
            else
                F0=sqrt(pi)/2*(erf(sqrt(as*(rp-0)^2))/sqrt(as*(rp-0)^2)+erf(sqrt(as*(rp-r)^2))/sqrt(as*(rp-r)^2));
            end
            A(i,j)=-2*pi*Z/as*exp(-rat*(ra(i)-ra(j))^2)*F0;
            A(j,i)=A(i,j); % using symmetry
        end
    end
    S
    T
    A
    [V,D]=eig(S); % gives eigenvectors in columns of V, this diagnolizes S
    for i=1:asize(2)
        V(:,i)=V(:,i)/(D(i,i)^0.5);
    end
    % the two-electron integrals P taking care of all symmetries
    % it is noted that the gaussian functions evaluate to REAL values
    % and the g operator is real and symmetric w.r.t. 1 and 2
    for i=1:8
        for j=1:i % do not run over all basis functions to exploit symmetry
            as1=alpha(i)+alpha(j);
            ap1=alpha(i)*alpha(j);
            rp=(alpha(i)*ra(i)+alpha(j)*ra(j))/as1;
            for k=1:i-1 % do not run over all basis functions to exploit symmetry
                for l=1:k % do not run over all basis functions to exploit symmetry
                    as2=alpha(k)+alpha(l);
                    ap2=alpha(k)*alpha(l);
                    rq=(alpha(k)*ra(k)+alpha(l)*ra(l))/as2;
                    if ((rp-rq)==0)
                        F0=1;
                    else
                        F0=sqrt(pi)/2*erf(sqrt(as1*as2*(rp-rq)^2/(as1+as2)))/sqrt(as1*as2*(rp-rq)^2/(as1+as2));
                    end
                    g(i,j,k,l)=2*pi^2.5/as1/as2/(as1+as2)^0.5*exp(-ap1*(ra(i)-ra(j))^2/as1-ap2*(ra(k)-ra(l))^2/as2)*F0;
                    g(k,l,i,j)=g(i,j,k,l); % using symmetry
                    g(j,i,k,l)=g(i,j,k,l); % using symmetry
                    g(i,j,l,k)=g(i,j,k,l); % using symmetry
                    g(j,i,l,k)=g(i,j,k,l); % using symmetry
                    g(k,l,j,i)=g(i,j,k,l); % using symmetry
                    g(l,k,i,j)=g(i,j,k,l); % using symmetry
                    g(l,k,j,i)=g(i,j,k,l); % using symmetry
                end
                k=i;
                for l=1:j
                    as2=alpha(k)+alpha(l);
                    ap2=alpha(k)*alpha(l);
                    rq=(alpha(k)*ra(k)+alpha(l)*ra(l))/as2;
                    if ((rp-rq)==0)
                        F0=1;
                    else
                        F0=sqrt(pi)/2*erf(sqrt(as1*as2*(rp-rq)^2/(as1+as2)))/sqrt(as1*as2*(rp-rq)^2/(as1+as2));
                    end
                    g(i,j,k,l)=2*pi^2.5/as1/as2/(as1+as2)^0.5*exp(-ap1*(ra(i)-ra(j))^2/as1-ap2*(ra(k)-ra(l))^2/as2)*F0;
                    g(k,l,i,j)=g(i,j,k,l); % using symmetry
                    g(j,i,k,l)=g(i,j,k,l); % using symmetry
                    g(i,j,l,k)=g(i,j,k,l); % using symmetry
                    g(j,i,l,k)=g(i,j,k,l); % using symmetry
                    g(k,l,j,i)=g(i,j,k,l); % using symmetry
                    g(l,k,i,j)=g(i,j,k,l); % using symmetry
                    g(l,k,j,i)=g(i,j,k,l); % using symmetry
                end
            end
        end
    end
    % Self-consistent loop
    C=ones(1,asize(2)); %initial guess
    nor=C*S*C'
    disp('Initial Guess');
    C=C/nor
    Elast=1;
    EVal=0;
    P=C'*C;
    count=0;
    rplot=-2:0.01:3; % range in atomic units (bohr) for plotting the probability density
    % figure();
    % Generating Fock matrix
    while((abs(EVal(1)-Elast))>0.000001)
        count=count+1;
        disp('Step #');
        disp(count);
        Elast=EVal(1)
        for i=1:8
            for j=1:8
                J=0;
                for k=1:8
                    for l=1:8
                        J=J+P(k,l)*g(i,j,k,l);
                    end
                end
                F(i,j)=T(i,j)+A(i,j)+J;
            end
        end
        % solve the generalized eigenvalue problem FC=ESC
        % inverse of V is its conjugate transpose because V is unitary
        Fp=V'*F*V; % modified Fock matrix
        [Vecp,Val]=eig(Fp);
        Vec=V*Vecp;
        EigVal=diag(Val,0)
        [EVal,index]=sort(EigVal);
        disp('Ground State eigenvalue from this step is (in a.u., i.e. Hartree):');
        Erec(count)=EVal(1) % lowest eigenvalue
        GrCoeff=Vec(:,index(1)); % eigenvector corresponding to ground state
        disp('C matrix from this step is:');
        C=GrCoeff.' %new C matrix, this is normalized w.r.t. S ie. C*S*C'=1
        disp('Input density matrix for next step is:');
        P=0.8*P+0.2*C'*C %new input density matrix
        GndWvFn=C(1:4)*exp(-(alpha(1:4)).'*(rplot.*rplot))+C(5:8)*exp(-(alpha(5:8)).'*((rplot-r).*(rplot-r))); % ground state wavefunction from STO-4G approximation
        %     Uncomment the next 4 lines if you need a plot of probability with step of convergence
        %     hold on;
        %     plot(rplot,(abs(GndWvFn)).^2,'r--','LineWidth',1.5);
        %     xlabel('r position');
        %     ylabel('Probability density');
        %     title('Plot of the probability density vs. r position with step of convergence');
    end
    Qs=0;
    for i=1:8
        for j=1:8
            for k=1:8
                for l=1:8
                    Qs=Qs+g(i,j,k,l)*C(i)*C(j)*C(k)*C(l);
                end
            end
        end
    end
    disp('The final total energy (including nuclear-nuclear repulsion) after convergence for this H-H bond is ');
    Eg(y)=2*C*(T+A)*C'+Qs+NNr; % the energy from h for each e, the e-e interaction and the nucleus-nucleus interaction added
    disp(Eg(y));
    if Eg(y)==min(Eg)
        rmin = r;
        Erecmin = Erec;
        WvFnmin = GndWvFn;
    end
end

figure();
plot([0.5:0.05:2.5],Eg,'bo','LineWidth',2);
title('Plot showing total energy from Hartree Fock treatment of H-H using Born Oppenheimer approximation');
xlabel('H-H bondlength');
ylabel('Total energy (in a.u.)');

disp('The least energy bond length (A) for H-H is '); disp(rmin*0.529177);
disp('The energy (Hartree) of this bond is '); disp(min(Eg));

figure();
plot(Erecmin,'ro','LineWidth',2);
axis([1 count min(Erecmin) max(Erecmin)+0.1]);
title('Plot showing convergence of Fock level with every step for the least energy H-H bondlength ');
xlabel('Step of convergence');
ylabel('Fock energy level (in a.u.)');

%plot of final probabiliy density
figure();
plot(rplot,(abs(WvFnmin)).^2,'LineWidth',2.5);
xlabel('r position');
ylabel('Probability density');
title('Plot of the probability density of electrons vs. r position for the least energy H-H bond');
axis tight;

In [ ]:
% Heston Prices Lewis 2001 Equation 3 11 20220414.m
% Heston (1993) prices using the characteristic function described in Alan
% Lewis' paper "A Simple Option Formula for General Jump-Diffusion and other
% Exponential Levy Processes"
% Finds the call price in Equation (3.11) of his paper

clc; clear;

% Spot price, strike price, risk free rate, dividend yield, and maturity
S   = 100;
K   = 100;
r   = 0.02;
q   = 0.0;
T   = 1.0;

% Heston model parameters.
kappa  = 2;       % Volatility reversion speed
theta  = 0.04;    % Volatility reversion level
sigma  = 0.05;     % Volatility of variance
rho    = -0.5;    % Correlation
v0     = 0.04;    % Initial variance
lambda = 0;       % Vol risk parameter
trap   = 1;       % Little trap formulation
param = [kappa theta sigma v0 rho];

% % Absicissas and weights
% [x w] = GenerateGaussLaguerre(32);
% trap = 1;
% K = [95:105];
% 
% % Find the Lewis (2001) prices from his Equation (3.11) and the orignal
% % and the original Heston (1993) prices
% fprintf('Lewis (2001) and Heston (1993) call prices\n')
% fprintf('---------------------------------------------\n')
% fprintf('Strike   LewisPrice   HestonPrice     Error\n')
% fprintf('---------------------------------------------\n')
% for k=1:length(K)
% 	% Lewis (2001) price using his Equation 3.11
% 	LewisPrice(k) = LewisPrice311(S,K(k),r,q,T,theta,kappa,sigma,rho,v0,trap,x,w);
% 	% Find the Heston price using Simpsons' 3/8 rule and the "Little Heston Trap" formulation
%     HestonPrice(k) = HestonPriceGaussLaguerre('C',S,K(k),T,r,q,param,trap,x,w);
% 	% Find the error
% 	e(k) = (HestonPrice(k)-LewisPrice(k))/HestonPrice(k)*100;
% 	% Display the results
%     fprintf(' %3.0f %12.4f %12.4f  %12.4f\n', K(k),LewisPrice(k),HestonPrice(k),e(k));
% end
% fprintf('---------------------------------------------\n')
% 
% Absicissas and weights
% [x w] = GenerateGaussLaguerre(64);
[x w] = GenerateGaussLaguerre(32);

trap = 1;
K = [100:100];

% Find the Lewis (2001) prices from his Equation (3.11) and the orignal
% and the original Heston (1993) prices
fprintf('Lewis (2001) and Heston (1993) call prices\n')
fprintf('---------------------------------------------\n')
fprintf('Strike   LewisPrice   HestonPrice     Error\n')
fprintf('---------------------------------------------\n')
for k=1:length(K)
	% Lewis (2001) price using his Equation 3.11
	LewisPrice(k) = LewisPrice311(S,K(k),r,q,T,theta,kappa,sigma,rho,v0,trap,x,w);
	% Find the Heston price using Simpsons' 3/8 rule and the "Little Heston Trap" formulation
    HestonPrice(k) = HestonPriceGaussLaguerre('C',S,K(k),T,r,q,param,trap,x,w);
	% Find the error
	e(k) = (HestonPrice(k)-LewisPrice(k))/HestonPrice(k)*100;
	% Display the results
    fprintf(' %3.0f %12.4f %12.4f  %12.4f\n', K(k),LewisPrice(k),HestonPrice(k),e(k));
end
fprintf('---------------------------------------------\n')

In [ ]:
function value = heston_like(theta, param)
% Heston-like parameterization
lambda = param(1);
value = 1./(lambda*theta) .* (1 - (1 - exp(-lambda*theta))./(lambda*theta));
end

In [ ]:
function [output] = HazardRateCurveBootStrap(name,dateH,dataIn,paramsIn)
%     KRNT.name = 'KRNT';
%     KRNT.holidaysList = {'2015-01-01'
% '2015-02-18'
% '2015-02-19'
% '2015-02-20'
% '2015-05-01'
% '2015-05-05'
% '2015-05-25'
% '2015-06-06'
% '2015-08-15'
% '2015-09-26'
% '2015-09-27'
% '2015-09-28'
% '2015-09-29'
% '2015-10-03'
% '2015-10-09'
% '2015-12-25'
% '2016-01-01'
% '2016-02-08'
% '2016-02-09'
% '2016-02-10'
% '2016-03-01'
% '2016-04-13'
% '2016-05-01'
% '2016-05-05'
% '2016-06-06'
% '2016-08-15'
% '2016-09-14'
% '2016-09-15'
% '2016-09-16'
% '2016-10-03'
% '2016-10-09'
% '2017-01-27'
% '2017-01-28'
% '2017-01-29'
% '2017-01-30'
% '2017-03-01'
% '2017-05-01'
% '2017-05-03'
% '2017-05-05'
% '2017-05-09'
% '2017-06-06'
% '2017-08-15'
% '2017-10-02'
% '2017-10-03'
% '2017-10-04'
% '2017-10-05'
% '2017-10-06'
% '2017-10-09'
% '2017-12-25'
% '2018-01-01'
% '2018-02-15'
% '2018-02-16'
% '2018-03-01'
% '2018-05-01'
% '2018-05-05'
% '2018-05-07'
% '2018-05-22'
% '2018-06-06'
% '2018-06-13'
% '2018-08-15'
% '2018-09-24'
% '2018-09-25'
% '2018-09-26'
% '2018-10-03'
% '2018-10-09'
% '2018-12-25'
% '2019-01-01'
% '2019-02-04'
% '2019-02-05'
% '2019-02-06'
% '2019-03-01'
% '2019-05-01'
% '2019-05-05'
% '2019-05-06'
% '2019-05-12'
% '2019-06-06'
% '2019-08-15'
% '2019-09-12'
% '2019-09-13'
% '2019-09-14'
% '2019-10-03'
% '2019-10-09'
% '2019-12-25'
% '2020-01-01'
% '2020-01-24'
% '2020-01-27'
% '2020-04-15'
% '2020-04-30'
% '2020-05-01'
% '2020-05-05'
% '2020-09-30'
% '2020-10-01'
% '2020-10-02'
% '2020-10-09'
% '2020-12-25'
% '2021-01-01'
% '2021-02-11'
% '2021-02-12'
% '2021-03-01'
% '2021-05-05'
% '2021-05-19'
% '2021-09-20'
% '2021-09-21'
% '2021-09-22'
% '2021-10-03'
% '2021-10-09'
% '2022-01-31'
% '2022-02-01'
% '2022-02-02'
% '2022-03-01'
% '2022-05-05'
% '2022-06-01'
% '2022-06-06'
% '2022-08-15'
% '2022-09-09'
% '2022-09-12'
% '2022-10-03'
% '2022-10-09'
% '2023-01-23'
% '2023-01-24'
% '2023-03-01'
% '2023-05-01'
% '2023-05-05'
% '2023-06-06'
% '2023-08-15'
% '2023-09-28'
% '2023-09-29'
% '2023-10-03'
% '2023-10-09'
% '2023-12-25'
% '2024-01-01'
% '2024-02-09'
% '2024-02-12'
% '2024-03-01'
% '2024-05-01'
% '2024-05-06'
% '2024-05-15'
% '2024-06-06'
% '2024-08-15'
% '2024-09-16'
% '2024-09-17'
% '2024-09-18'
% '2024-10-03'
% '2024-10-09'
% '2024-12-25'
% '2025-01-01'
% '2025-01-28'
% '2025-01-29'
% '2025-01-30'
% '2025-03-01'
% '2025-05-01'
% '2025-05-05'
% '2025-05-06'
% '2025-06-06'
% '2025-08-15'
% '2025-10-03'
% '2025-10-06'
% '2025-10-07'
% '2025-10-09'
% '2025-12-25'
% '2026-01-01'
% '2026-02-16'
% '2026-02-17'
% '2026-02-18'
% '2026-03-01'
% '2026-05-01'
% '2026-05-05'
% '2026-09-24'
% '2026-09-25'
% '2026-10-03'
% '2026-10-09'
% '2026-12-25'
% '2027-01-01'
% '2027-02-08'
% '2027-02-09'
% '2027-03-01'
% '2027-05-05'
% '2027-05-13'
% '2027-09-14'
% '2027-09-15'
% '2027-09-16'
% '2027-10-03'
% '2027-10-09'
% '2028-01-01'
% '2028-01-26'
% '2028-01-27'
% '2028-01-28'
% '2028-03-01'
% '2028-05-01'
% '2028-05-02'
% '2028-05-05'
% '2028-06-06'
% '2028-08-15'
% '2028-10-02'
% '2028-10-03'
% '2028-10-04'
% '2028-10-09'
% '2028-12-25'
% '2029-01-01'
% '2029-02-12'
% '2029-02-13'
% '2029-02-14'
% '2029-03-01'
% '2029-05-01'
% '2029-05-05'
% '2029-05-07'
% '2029-06-06'
% '2029-08-15'
% '2029-09-21'
% '2029-09-24'
% '2029-10-03'
% '2029-10-09'
% '2029-12-25'
% '2030-01-01'
% '2030-02-04'
% '2030-02-05'
% '2030-03-01'
% '2030-05-01'
% '2030-05-06'
% '2030-05-09'
% '2030-06-06'
% '2030-08-15'
% '2030-09-11'
% '2030-09-12'
% '2030-09-13'
% '2030-10-03'
% '2030-10-09'
% '2030-12-25'
% '2031-01-01'
% '2031-01-22'
% '2031-01-23'
% '2031-01-24'
% '2031-05-01'
% '2031-05-05'
% '2031-05-28'
% '2031-06-06'
% '2031-08-15'
% '2031-09-30'
% '2031-10-01'
% '2031-10-02'
% '2031-10-03'
% '2031-10-09'
% '2031-12-25'
% '2032-01-01'
% '2032-02-10'
% '2032-02-11'
% '2032-02-12'
% '2032-03-01'
% '2032-05-05'
% '2032-09-20'
% '2032-09-21'
% '2033-01-31'
% '2033-02-01'
% '2033-03-01'
% '2033-05-05'
% '2033-05-06'
% '2033-06-06'
% '2033-08-15'
% '2033-09-07'
% '2033-09-08'
% '2033-09-09'
% '2033-10-03'
% '2034-02-20'
% '2034-02-21'
% '2034-03-01'
% '2034-05-01'
% '2034-05-05'
% '2034-05-25'
% '2034-06-06'
% '2034-08-15'
% '2034-09-26'
% '2034-09-27'
% '2034-09-28'
% '2034-10-03'
% '2034-10-09'
% '2034-12-25'
% '2035-01-01'
% '2035-02-07'
% '2035-02-08'
% '2035-02-09'
% '2035-03-01'
% '2035-05-15'
% '2035-06-06'
% '2035-08-15'
% '2035-09-18'
% '2035-10-03'
% '2035-10-09'
% '2035-12-25'
% '2036-01-01'
% '2036-01-28'
% '2036-01-29'
% '2036-05-05'
% '2036-06-06'
% '2036-08-15'
% '2036-10-03'
% '2036-10-06'
% '2036-10-09'
% '2036-12-25'
% '2037-01-01'
% '2037-02-16'
% '2037-02-17'
% '2037-05-05'
% '2037-05-22'
% '2037-09-23'
% '2037-09-24'
% '2037-09-25'
% '2037-10-09'
% '2037-12-25'
% '2037-12-25'
% '2038-01-01'
% '2038-02-03'
% '2038-02-04'
% '2038-02-05'
% '2038-03-01'
% '2038-05-05'
% '2038-05-11'
% '2038-09-13'
% '2038-09-14'
% '2038-09-15'
%         };
% 
%     KRNTCalendar = Calendar();
%     KRNTCalendar.name = KRNT.name;
%     for i=1:length(KRNT.holidaysList)
%         KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
%     end
% 
%     KRBK.name = 'KRBK';
%     KRBK.holidaysList = {'2015-01-01'
% '2015-02-18'
% '2015-02-19'
% '2015-02-20'
% '2015-05-01'
% '2015-05-05'
% '2015-05-25'
% '2015-06-06'
% '2015-08-15'
% '2015-09-26'
% '2015-09-27'
% '2015-09-28'
% '2015-09-29'
% '2015-10-03'
% '2015-10-09'
% '2015-12-25'
% '2016-01-01'
% '2016-02-08'
% '2016-02-09'
% '2016-02-10'
% '2016-03-01'
% '2016-04-13'
% '2016-05-01'
% '2016-05-05'
% '2016-06-06'
% '2016-08-15'
% '2016-09-14'
% '2016-09-15'
% '2016-09-16'
% '2016-10-03'
% '2016-10-09'
% '2017-01-27'
% '2017-01-28'
% '2017-01-29'
% '2017-01-30'
% '2017-03-01'
% '2017-05-01'
% '2017-05-03'
% '2017-05-05'
% '2017-05-09'
% '2017-06-06'
% '2017-08-15'
% '2017-10-02'
% '2017-10-03'
% '2017-10-04'
% '2017-10-05'
% '2017-10-06'
% '2017-10-09'
% '2017-12-25'
% '2018-01-01'
% '2018-02-15'
% '2018-02-16'
% '2018-03-01'
% '2018-05-01'
% '2018-05-05'
% '2018-05-07'
% '2018-05-22'
% '2018-06-06'
% '2018-06-13'
% '2018-08-15'
% '2018-09-24'
% '2018-09-25'
% '2018-09-26'
% '2018-10-03'
% '2018-10-09'
% '2018-12-25'
% '2019-01-01'
% '2019-02-04'
% '2019-02-05'
% '2019-02-06'
% '2019-03-01'
% '2019-05-01'
% '2019-05-05'
% '2019-05-06'
% '2019-05-12'
% '2019-06-06'
% '2019-08-15'
% '2019-09-12'
% '2019-09-13'
% '2019-09-14'
% '2019-10-03'
% '2019-10-09'
% '2019-12-25'
% '2020-01-01'
% '2020-01-24'
% '2020-01-27'
% '2020-04-15'
% '2020-04-30'
% '2020-05-01'
% '2020-05-05'
% '2020-09-30'
% '2020-10-01'
% '2020-10-02'
% '2020-10-09'
% '2020-12-25'
% '2021-01-01'
% '2021-02-11'
% '2021-02-12'
% '2021-03-01'
% '2021-05-05'
% '2021-05-19'
% '2021-09-20'
% '2021-09-21'
% '2021-09-22'
% '2021-10-03'
% '2021-10-09'
% '2022-01-31'
% '2022-02-01'
% '2022-02-02'
% '2022-03-01'
% '2022-05-05'
% '2022-06-01'
% '2022-06-06'
% '2022-08-15'
% '2022-09-09'
% '2022-09-12'
% '2022-10-03'
% '2022-10-09'
% '2023-01-23'
% '2023-01-24'
% '2023-03-01'
% '2023-05-01'
% '2023-05-05'
% '2023-06-06'
% '2023-08-15'
% '2023-09-28'
% '2023-09-29'
% '2023-10-03'
% '2023-10-09'
% '2023-12-25'
% '2024-01-01'
% '2024-02-09'
% '2024-02-12'
% '2024-03-01'
% '2024-05-01'
% '2024-05-06'
% '2024-05-15'
% '2024-06-06'
% '2024-08-15'
% '2024-09-16'
% '2024-09-17'
% '2024-09-18'
% '2024-10-03'
% '2024-10-09'
% '2024-12-25'
% '2025-01-01'
% '2025-01-28'
% '2025-01-29'
% '2025-01-30'
% '2025-03-01'
% '2025-05-01'
% '2025-05-05'
% '2025-05-06'
% '2025-06-06'
% '2025-08-15'
% '2025-10-03'
% '2025-10-06'
% '2025-10-07'
% '2025-10-09'
% '2025-12-25'
% '2026-01-01'
% '2026-02-16'
% '2026-02-17'
% '2026-02-18'
% '2026-03-01'
% '2026-05-01'
% '2026-05-05'
% '2026-09-24'
% '2026-09-25'
% '2026-10-03'
% '2026-10-09'
% '2026-12-25'
% '2027-01-01'
% '2027-02-08'
% '2027-02-09'
% '2027-03-01'
% '2027-05-05'
% '2027-05-13'
% '2027-09-14'
% '2027-09-15'
% '2027-09-16'
% '2027-10-03'
% '2027-10-09'
% '2028-01-01'
% '2028-01-26'
% '2028-01-27'
% '2028-01-28'
% '2028-03-01'
% '2028-05-01'
% '2028-05-02'
% '2028-05-05'
% '2028-06-06'
% '2028-08-15'
% '2028-10-02'
% '2028-10-03'
% '2028-10-04'
% '2028-10-09'
% '2028-12-25'
% '2029-01-01'
% '2029-02-12'
% '2029-02-13'
% '2029-02-14'
% '2029-03-01'
% '2029-05-01'
% '2029-05-05'
% '2029-05-07'
% '2029-06-06'
% '2029-08-15'
% '2029-09-21'
% '2029-09-24'
% '2029-10-03'
% '2029-10-09'
% '2029-12-25'
% '2030-01-01'
% '2030-02-04'
% '2030-02-05'
% '2030-03-01'
% '2030-05-01'
% '2030-05-06'
% '2030-05-09'
% '2030-06-06'
% '2030-08-15'
% '2030-09-11'
% '2030-09-12'
% '2030-09-13'
% '2030-10-03'
% '2030-10-09'
% '2030-12-25'
% '2031-01-01'
% '2031-01-22'
% '2031-01-23'
% '2031-01-24'
% '2031-05-01'
% '2031-05-05'
% '2031-05-28'
% '2031-06-06'
% '2031-08-15'
% '2031-09-30'
% '2031-10-01'
% '2031-10-02'
% '2031-10-03'
% '2031-10-09'
% '2031-12-25'
% '2032-01-01'
% '2032-02-10'
% '2032-02-11'
% '2032-02-12'
% '2032-03-01'
% '2032-05-05'
% '2032-09-20'
% '2032-09-21'
% '2033-01-31'
% '2033-02-01'
% '2033-03-01'
% '2033-05-05'
% '2033-05-06'
% '2033-06-06'
% '2033-08-15'
% '2033-09-07'
% '2033-09-08'
% '2033-09-09'
% '2033-10-03'
% '2034-02-20'
% '2034-02-21'
% '2034-03-01'
% '2034-05-01'
% '2034-05-05'
% '2034-05-25'
% '2034-06-06'
% '2034-08-15'
% '2034-09-26'
% '2034-09-27'
% '2034-09-28'
% '2034-10-03'
% '2034-10-09'
% '2034-12-25'
% '2035-01-01'
% '2035-02-07'
% '2035-02-08'
% '2035-02-09'
% '2035-03-01'
% '2035-05-15'
% '2035-06-06'
% '2035-08-15'
% '2035-09-18'
% '2035-10-03'
% '2035-10-09'
% '2035-12-25'
% '2036-01-01'
% '2036-01-28'
% '2036-01-29'
% '2036-05-05'
% '2036-06-06'
% '2036-08-15'
% '2036-10-03'
% '2036-10-06'
% '2036-10-09'
% '2036-12-25'
% '2037-01-01'
% '2037-02-16'
% '2037-02-17'
% '2037-05-05'
% '2037-05-22'
% '2037-09-23'
% '2037-09-24'
% '2037-09-25'
% '2037-10-09'
% '2037-12-25'
% '2037-12-25'
% '2038-01-01'
% '2038-02-03'
% '2038-02-04'
% '2038-02-05'
% '2038-03-01'
% '2038-05-05'
% '2038-05-11'
% '2038-09-13'
% '2038-09-14'
% '2038-09-15'
%         };
% 
%     KRBKCalendar = Calendar();
%     KRBKCalendar.name = KRBK.name;
% 
%     for i=1:length(KRBK.holidaysList)
%         KRBKCalendar.AddHoliday(KRBK.holidaysList{i})
%     end
% 
%     jointCalendar = JointCalendar({KRNTCalendar,KRBKCalendar});
    
    jointCalendar = MakeCalendar(dateH,{},'30Y');
    valueDateH = dateH;
    %% import and construct an HazardRateCurve
    dataType = dataIn.dataType;
    dataTenor = dataIn.dataTenor;
    dataRate = dataIn.dataRate;
    
    
    % construct iborIndex
    
    
    
    rateHelpers = {};

    for i=1:length(dataRate)
        if strcmp(dataType{i},'SpreadCDS')
            spreadCDSSettlementDays = paramsIn.spreadCDSSettlementDays;
            spreadCDSBasis = paramsIn.spreadCDSBasis;
            spreadCDSConvention = paramsIn.spreadCDSConvention;
            spreadCDSPeriod = paramsIn.spreadCDSPeriod;
            spreadCDSDateGenerationRule = paramsIn.spreadCDSDateGenerationRule;
            settlesAccrual = paramsIn.settlesAccrual;
            paysAtDefaultTime = paramsIn.paysAtDefaultTime;
            engineType = paramsIn.engineType;
            
            recoveryRate = paramsIn.recoveryRate;
            discountCurve = paramsIn.discountCurve;
            
            spreadCDSHelperParams = containers.Map({'spreadCDSSettlementDays','fixingCalendar','spreadCDSBasis',...
                                                    'spreadCDSConvention','spreadCDSPeriod','spreadCDSDateGenerationRule',...
                                                    'settlesAccrual','paysAtDefaultTime','engineType',...
                                                    'recoveryRate','discountCurve'}, ...
                                    {spreadCDSSettlementDays,jointCalendar,spreadCDSBasis,...
                                     spreadCDSConvention,spreadCDSPeriod,spreadCDSDateGenerationRule,...
                                     settlesAccrual,paysAtDefaultTime,engineType,...
                                     recoveryRate,discountCurve});


            spreadCDSHelperParams('rate') = dataRate{i};
            spreadCDSHelperParams('rateTenor') = dataTenor{i};
            spreadCDSHelper = SpreadCDSHelper(spreadCDSHelperParams);
            rateHelpers{end+1} = spreadCDSHelper;
        elseif strcmp(dataType{i},'RiskyFixedBond')
            riskyFixedBondSettlementDays = paramsIn.riskyFixedBondSettlementDays;
            riskyFixedBondBasis = paramsIn.riskyFixedBondBasis;
            riskyFixedBondConvention = paramsIn.riskyFixedBondConvention;
            riskyFixedBondPeriod = paramsIn.riskyFixedBondPeriod;
            riskyFixedBondDateGenerationRule = paramsIn.riskyFixedBondDateGenerationRule;
            
            settlesAccrual = paramsIn.settlesAccrual;
            paysAtDefaultTime = paramsIn.paysAtDefaultTime;
            
            engineType = paramsIn.engineType;
            
            recoveryRate = paramsIn.recoveryRate;
            discountCurve = paramsIn.discountCurve;
            
            riskyFixedBondHelperParams = containers.Map({'riskyFixedBondSettlementDays','fixingCalendar','riskyFixedBondBasis',...
                                                    'riskyFixedBondConvention','riskyFixedBondPeriod','riskyFixedBondDateGenerationRule',...
                                                    'settlesAccrual','paysAtDefaultTime','engineType',...
                                                    'recoveryRate','discountCurve'}, ...
                                    {riskyFixedBondSettlementDays,jointCalendar,riskyFixedBondBasis,...
                                     riskyFixedBondConvention,riskyFixedBondPeriod,riskyFixedBondDateGenerationRule,...
                                     settlesAccrual,paysAtDefaultTime,engineType,...
                                     recoveryRate,discountCurve});


            riskyFixedBondHelperParams('rate') = dataRate{i};
            riskyFixedBondHelperParams('rateTenor') = dataTenor{i};
            riskyFixedBondHelper = RiskyFixedBondHelper(riskyFixedBondHelperParams);
            rateHelpers{end+1} = riskyFixedBondHelper;
            
        end
    end

    % forward & discount zeroCurve
    dummyCurve = SurvProbCurve(MktData(containers.Map('asOfDate',valueDateH)));
    
    defaultCurveParams = containers.Map({'rateHelpers','survProbCurve'},{rateHelpers,dummyCurve});

    tic
    defaultCurve = DefaultCurve(defaultCurveParams);
    out = defaultCurve.BootStrap();
    toc

    printDebugInfo = false;
    if isfield(paramsIn,'printDebugInfo')
        if paramsIn.printDebugInfo
            printDebugInfo = true;
        end
    end
    
    outInfo = struct;
    outinfo.out = out;
    outInfo.impliedRate = zeros(length(defaultCurve.rateHelpers),1);

    outInfo.diff = zeros(length(defaultCurve.rateHelpers),1);
    outInfo.NPV = zeros(length(defaultCurve.rateHelpers),1);

    for i=1:length(defaultCurve.rateHelpers)
        if strcmp(defaultCurve.rateHelpers{i}.rateType,'SpreadCDS')
            outInfo.impliedRate(i) = defaultCurve.rateHelpers{i}.ImpliedQuote();
            outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
        elseif strcmp(defaultCurve.rateHelpers{i}.rateType,'RiskyFixedBond')
            outInfo.impliedRate(i) = defaultCurve.rateHelpers{i}.ImpliedQuote();
            outInfo.diff(i) = 1.0 - outInfo.impliedRate(i);
        elseif strcmp(defaultCurve.rateHelpers{i}.rateType,'SpreadCDS')
            for j=1:length(defaultCurve.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates)
                defaultCurve.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates{j}.fixedRate = dataRate{i};
            end

            swapOut = defaultCurve.rateHelpers{i}.creditDefaultSwap.Calculate(defaultCurve.rateHelpers{i}.recoveryRate,...
                defaultCurve.rateHelpers{i}.survProbCurve,defaultCurve.rateHelpers{i}.discountCurve);
            outInfo.NPV(i) = swapOut.NPV;
            outInfo.impliedRate(i) = swapOut.fairSpread;
            outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
        end
    end
    
    if printDebugInfo
        outInfo = struct;
        outInfo.impliedRate = zeros(length(defaultCurve.rateHelpers),1);

        outInfo.diff = zeros(length(defaultCurve.rateHelpers),1);
        outInfo.NPV = zeros(length(defaultCurve.rateHelpers),1);

        for i=1:length(defaultCurve.rateHelpers)
            if strcmp(defaultCurve.rateHelpers{i}.rateType,'SpreadCDS')
                outInfo.impliedRate(i) = defaultCurve.rateHelpers{i}.ImpliedQuote();
                outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
            elseif strcmp(defaultCurve.rateHelpers{i}.rateType,'RiskyFixedBond')
                outInfo.impliedRate(i) = defaultCurve.rateHelpers{i}.ImpliedQuote();
                outInfo.diff(i) = 1.0 - outInfo.impliedRate(i);
            elseif strcmp(defaultCurve.rateHelpers{i}.rateType,'SpreadCDS')
                for j=1:length(defaultCurve.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates)
                    defaultCurve.rateHelpers{i}.vanillaSwap.fixedLeg.couponRates{j}.fixedRate = dataRate{i};
                end

                swapOut = defaultCurve.rateHelpers{i}.creditDefaultSwap.Calculate(defaultCurve.rateHelpers{i}.recoveryRate,...
                    defaultCurve.rateHelpers{i}.survProbCurve,defaultCurve.rateHelpers{i}.discountCurve);
                outInfo.NPV(i) = swapOut.NPV;
                outInfo.impliedRate(i) = swapOut.fairSpread;
                outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
            end
        end

        % CrosCheck auxilary CDS par spread?
        auxRateHelpers = {};
        impliedParCDSSPread = zeros(length(defaultCurve.rateHelpers),1);
        impliedYTM = zeros(length(defaultCurve.rateHelpers),1);

        if strcmp(dataType{1},'RiskyFixedBond')


            for i=1:length(dataRate)
                spreadCDSSettlementDays = paramsIn.spreadCDSSettlementDays;
                spreadCDSBasis = paramsIn.spreadCDSBasis;
                spreadCDSConvention = paramsIn.spreadCDSConvention;
                spreadCDSPeriod = paramsIn.spreadCDSPeriod;
                spreadCDSDateGenerationRule = paramsIn.spreadCDSDateGenerationRule;
                settlesAccrual = paramsIn.settlesAccrual;
                paysAtDefaultTime = paramsIn.paysAtDefaultTime;

                engineType = paramsIn.engineType;

                recoveryRate = paramsIn.recoveryRate;
                discountCurve = paramsIn.discountCurve;

                spreadCDSHelperParams = containers.Map({'spreadCDSSettlementDays','fixingCalendar','spreadCDSBasis',...
                                                            'spreadCDSConvention','spreadCDSPeriod','spreadCDSDateGenerationRule',...
                                                            'settlesAccrual','paysAtDefaultTime','engineType',...
                                                            'recoveryRate','discountCurve'}, ...
                                            {spreadCDSSettlementDays,jointCalendar,spreadCDSBasis,...
                                             spreadCDSConvention,spreadCDSPeriod,spreadCDSDateGenerationRule,...
                                             settlesAccrual,paysAtDefaultTime,engineType,...
                                             recoveryRate,discountCurve});


                spreadCDSHelperParams('rate') = dataRate{i};
                spreadCDSHelperParams('rateTenor') = dataTenor{i};
                spreadCDSHelper = SpreadCDSHelper(spreadCDSHelperParams);
                auxOut = spreadCDSHelper.creditDefaultSwap.Calculate(recoveryRate,out.survProbCurve,discountCurve);

                impliedParCDSSPread(i) = auxOut.fairSpread;
                auxRateHelpers{end+1} = riskyFixedBondHelper;
            end
        end

        % CrosCheck auxilary RiskyFixedBond fairRate(YTM)?
        if strcmp(dataType{1},'SpreadCDS')


            for i=1:length(dataRate)

                riskyFixedBondSettlementDays = paramsIn.riskyFixedBondSettlementDays;
                riskyFixedBondBasis = paramsIn.riskyFixedBondBasis;
                riskyFixedBondConvention = paramsIn.riskyFixedBondConvention;
                riskyFixedBondPeriod = paramsIn.riskyFixedBondPeriod;
                riskyFixedBondDateGenerationRule = paramsIn.riskyFixedBondDateGenerationRule;

                engineType = paramsIn.engineType;

                recoveryRate = paramsIn.recoveryRate;
                discountCurve = paramsIn.discountCurve;

                riskyFixedBondHelperParams = containers.Map({'riskyFixedBondSettlementDays','fixingCalendar','riskyFixedBondBasis',...
                                                        'riskyFixedBondConvention','riskyFixedBondPeriod','riskyFixedBondDateGenerationRule',...
                                                        'engineType',...
                                                        'recoveryRate','discountCurve'}, ...
                                        {riskyFixedBondSettlementDays,jointCalendar,riskyFixedBondBasis,...
                                         riskyFixedBondConvention,riskyFixedBondPeriod,riskyFixedBondDateGenerationRule,...
                                         engineType,...
                                         recoveryRate,discountCurve});


                riskyFixedBondHelperParams('rate') = dataRate{i};
                riskyFixedBondHelperParams('rateTenor') = dataTenor{i};
                riskyFixedBondHelper = RiskyFixedBondHelper(riskyFixedBondHelperParams);
                auxOut = riskyFixedBondHelper.riskyFixedBond.Calculate(recoveryRate,out.survProbCurve,discountCurve);
    %             auxOut = spreadCDSHelper.creditDefaultSwap.Calculate(recoveryRate,out.survProbCurve,discountCurve);
                impliedYTM(i) = auxOut.fairRate;
                auxRateHelpers{end+1} = spreadCDSHelper;
            end
        end

        aaa =1;

    %     targetTimeBucket = [0.0027397
    %     0.25
    %     0.5
    %     0.75
    %     1
    %     2
    %     3
    %     4
    %     5
    %     7
    %     10
    %     12
    %     15
    %     20
    %     25
    %     30
    %         ];


        output.impliedParCDSSPread = impliedParCDSSPread;
        output.impliedYTM = impliedYTM;
    end
    
    targetTimeBucket = paramsIn.targetTimeBucket;
    targetSurvProb = zeros(length(targetTimeBucket),1);
    for i=1:length(targetSurvProb)
        targetSurvProb(i) = defaultCurve.survProbCurve.SurvProb(targetTimeBucket(i));
    end

    targetOut = zeros(length(targetSurvProb),2);
    targetOut(:,1) = targetTimeBucket(:);
    targetOut(:,2) = targetSurvProb(:);

    output.targetOut = targetOut;
    output.times = out.times;
    output.survProb = out.sols;
    output.survProbCurve = out.survProbCurve;
end



In [ ]:

classdef H_RootObject < handle
    %RootObject of H_Library
    
    properties
    end
    
    methods
    end
    
end



In [ ]:
function out = H_nicdf(x)
    out = icdf('Normal',x,0.0,1.0);
end



In [ ]:
function out = H_ndf(x)
    out = 1.0/sqrt(2*pi)*exp(-0.5*x*x);
end



In [ ]:
function out = H_ncdf(x)
   if ~isreal(x)
        error('exception handling needed')
    end
    out = (1.0+erf(x/sqrt(2.0)))/2.0;
    
end



In [ ]:
function v = H_interpolation(x,y,u,interpolOrder)
    
    if nargin == 4
        interpolation_order = interpolOrder;
    else
        interpolation_order = 0;
    end
    
    n = length(x);
    k = 1;
    
    switch interpolation_order
        case 0 % piecewixe constant interpolation
            if u <= x(1)
                v = y(1);  % constant extrapolation
            elseif u > x(n)
                v= y(n); %constant extrapolation
            else    
                for j = 2:n
                    if u <= x(j)
                        k=j;
                        break;
                    end
                end
                v = y(k);
            end
        case 1 % piecewise linear interpolation
            if u <= x(1)
                v = y(1);  % constant extrapolation
            elseif u > x(n)
                v= y(n); %constant extrapolation
            else
                delta = diff(y)./diff(x);
                for j = 2:n
                     if u <= x(j)
                        k=j;
                        break;
                    end
                end
                % Evaluate interpolant
                s = u - x(k-1);
                v = y(k-1) + s.*delta(k-1);
            end
        case 3 % monotonic cubic spline interpolation
            if u <= x(1)
                v = y(1);  % constant extrapolation
            elseif u > x(n)
                v= y(n); %constant extrapolation
            else
                % Evaluate monotonic cubic interpolant
                
                v = pchip(x,y,u);
                
            end
        otherwise
            disp('only works for piecewise constant and linear  cases!!')
    end
    
            
end



In [ ]:
classdef H_Date < H_RootObject

    %Model is the basic object that represents a pricing model

 

    

    properties(SetAccess = public)

        day
        month
        year
        dateNum

    end

    

    methods

        function h_Date = H_Date(inDate)

            if ischar(inDate) && length(inDate) == 10

                [pyear,pmonth,pday] = datevec(inDate,'yyyy-mm-dd');

                h_Date.day = pday;
                h_Date.month = pmonth;
                h_Date.year =  pyear;
                h_Date.dateNum = datenum(pyear,pmonth,pday);
                
            elseif ischar(inDate) && length(inDate) == 8

                [pyear,pmonth,pday] = datevec(inDate,'yyyymmdd');

                h_Date.day = pday;
                h_Date.month = pmonth;
                h_Date.year =  pyear;
                h_Date.dateNum = datenum(pyear,pmonth,pday);

            elseif isnumeric(inDate)

                if inDate > 20000000  % somewhat adhoc but seems to work to differentialte between 47662 & 20170217

                    h_Date.day = mod(inDate,100);
                    h_Date.month = mod((inDate - h_Date.day)/100,100);
                    h_Date.year = (inDate - (h_Date.day + h_Date.month*100))/10000;
                    h_Date.dateNum = datenum(h_Date.year,h_Date.month,h_Date.day);

                else

                    [pyear,pmonth,pday] = datevec(inDate); 
                    h_Date.day = pday;
                    h_Date.month = pmonth;
                    h_Date.year =  pyear;
                    h_Date.dateNum = datenum(pyear,pmonth,pday);

                    

                end

            elseif isa(inDate,'H_Date')
                h_Date.day = inDate.day;
                h_Date.month = inDate.month;
                h_Date.year =  inDate.year;
                h_Date.dateNum = inDate.dateNum;

            end

            

        end

        

        function out = StrDate(h_Date,type)

            if strcmp(type,'str')
                strDate =  datestr(h_Date.dateNum,'yyyy-mm-dd');
                out = strDate;
            elseif strcmp(type,'fng')
                fngDate =  datestr(h_Date.dateNum,'yyyymmdd');
                out = fngDate;  
                
%                 fngDate =  h_Date.year*10000 + h_Date.month*100 + h_Date.day;
%                 out = fngDate;  
            end
        end

        

        

        function out = FngDate(h_Date)

            fngDate =  h_Date.year*10000 + h_Date.month*100 + h_Date.day;
            out = fngDate;    

        end

        function out = AddTenor(h_Date,tenor)
           if strcmp(tenor(end),'D')
                out = AddDate(h_Date,str2double(tenor(1:end-1)),'day');
           elseif strcmp(tenor(end),'W')
                numOfWeeks = str2double(tenor(1:end-1));
                numOfDays = 7 * numOfWeeks;
                out = AddDate(h_Date, numOfDays,'day');
           elseif strcmp(tenor(end),'M')
                out = AddDate(h_Date,str2double(tenor(1:end-1)),'month');
           elseif strcmp(tenor(end),'Y')
               out = AddDate(h_Date,str2double(tenor(1:end-1)),'year');
           else
               error('unimplemented')
           end
           
        end

        function out =AddDate(h_Date,num,dtype)

            if strcmp(dtype,'day')
                addDate =  addtodate(h_Date.dateNum,num,dtype);
                %addDate =  h_Date.dateNum + num;
            else
                if strcmp(dtype,'year') && num < 1
                    numMonth = round(num*12);
                    addDate =  addtodate(h_Date.dateNum,numMonth,'month');
                else
                    addDate =  addtodate(h_Date.dateNum,num,dtype);
                end
            end
            out = H_Date(addDate);    
        end

        function out = Next20th(h_Date)
            result = H_Date(num2str(h_Date.year*10000+h_Date.month*100 +20));
            if DateDiff(h_Date,result)
                result = result.AddDate(1,'month');
            end
            
            if rem(result.month,3) ~= 0
                skip = 3 - rem(result.month,3);
                result = result.AddDate(skip,'month');
            end
            
            out =result;
        end

        function out = DateDiff(h_Date1,h_Date2)
            % return h_Date1 - h_Date2 in days
            out = h_Date1.dateNum - h_Date2.dateNum;

        end
        
        function out = DayCount(h_Date1,h_Date2,dayCountConvention)
        
            if strcmp(dayCountConvention,'Act365')
                out = DateDiff(h_Date2, h_Date1);
            elseif strcmp(dayCountConvention,'Act360')
                out = DateDiff(h_Date2, h_Date1);
            elseif strcmp(dayCountConvention,'30360')
                yy2 = h_Date2.year;
                yy1 = h_Date1.year;
                mm2 = h_Date2.month;
                mm1 = h_Date1.month;
                dd2 = h_Date2.day;
                dd1 = h_Date1.day;
                
                if dd2==31 && dd1 < 30
                    dd2 = 1;
                    mm2 = mm2 +1;
                end
                
                out = 360*(yy2-yy1) + 30*(mm2-mm1-1) +max(0,30-dd1) +min(30,dd2);
                out = out;
            end
        end

        function out = DayCountFraction(h_Date1,h_Date2,dayCountConvention)
        
            if strcmp(dayCountConvention,'Act365')
                out = DateDiff(h_Date2, h_Date1)/365.0;
            elseif strcmp(dayCountConvention,'Act360')
                out = DateDiff(h_Date2, h_Date1)/360.0;
            elseif strcmp(dayCountConvention,'30360')
                yy2 = h_Date2.year;
                yy1 = h_Date1.year;
                mm2 = h_Date2.month;
                mm1 = h_Date1.month;
                dd2 = h_Date2.day;
                dd1 = h_Date1.day;
                
                if dd2==31 && dd1 < 30
                    dd2 = 1;
                    mm2 = mm2 +1;
                end
                
                out = 360*(yy2-yy1) + 30*(mm2-mm1-1) +max(0,30-dd1) +min(30,dd2);
                out = out/360.0;
            end
        end

        function out = IsEndOfMonth(h_Date)

            if h_Date.day == eomday(h_Date.year,h_Date.month)

                out = true;

            else

                out = false;

            end

            

        end

        

        function out = EndOfMonth(h_Date)

            arg = eomday(h_Date.year,h_Date.month) + ...
                h_Date.month*100 +h_Date.year*10000;
            out = H_Date(arg);

            

        end

        

        function out = WeekDay(h_Date)

            out = weekday(StrDate(h_Date,'str'));

        end

        

        function out = IsWeekend(h_Date)

            if WeekDay(h_Date) == 1 || WeekDay(h_Date) == 7

                out = true;

            else

                out = false;

            end

        end

        

        function out = eq(h_Date1,h_Date2)

            if h_Date1.dateNum == h_Date2.dateNum

                out =true;

            elseif h_Date1.day == h_Date2.day && ...
                   h_Date1.month == h_Date2.month && ...
                   h_Date1.year == h_Date2.year

               out = true;

            else

                out = false;

            end

        end
        
        function out = ne(h_Date1,h_Date2)

            if h_Date1.dateNum ~= h_Date2.dateNum

                out =true;

            elseif h_Date1.day ~= h_Date2.day || ...
                   h_Date1.month ~= h_Date2.month || ...
                   h_Date1.year ~= h_Date2.year

               out = true;

            else

                out = false;

            end

        end

        

        function out = ScheduleGenerate(initialDate,maturityDate,freq)

            daycount = 365.0;

%             if ccy == 'USD'

%                 daycount = 360.0;

%             end

            

            endDate = initialDate.AddDate(12/freq,'month');



%             startDates=[];

%             endDates = [];

%             resetDates = [];

%             payDates = [];

            numOfPeriod = 0;

            i = 1;

            while 1

                startDates(i) = initialDate.AddDate((i-1)*12/freq,'month');

                resetDates(i) = startDates(i);

                endDates(i)   = initialDate.AddDate(i*12/freq,'month');

                payDates(i)   = endDates(i);

                dayCount(i) = endDates(i).DateDiff(startDates(i));

                dayCountFraction(i) = endDates(i).DateDiff(startDates(i))/daycount;

                numOfPeriod = numOfPeriod + 1;

                if endDates(i).DateDiff(maturityDate) >= 0 break; end

                i=i+1;

            end

            out.startDates= startDates;

            out.resetDates = resetDates;

            out.endDates = endDates;

            out.payDates = payDates;

            out.dayCount = dayCount;

            out.dayCountFraction = dayCountFraction;

            out.numOfPeriod = numOfPeriod;

        end

        function out = ScheduleGenerateQL(initialDate,maturityDate,freq,calendar,dateGenRule,bdc,endOfMonth)
            
            dates={};
            
            if strcmp(dateGenRule,'Backward')
                temp_dates={};
                numOfPeriod = 0;
                i = 1;
                seed = maturityDate;
                while 1

                    temp_dates{end+1} = AddDate(seed,-(i-1)*12/freq,'month');
                    numOfPeriod = numOfPeriod + 1;
                    if temp_dates{i}.DateDiff(initialDate) <= 0 break; end
                    i=i+1;
                end
                
                dates= cell(numOfPeriod,1);
                
                for i=1:numOfPeriod
                    dates{i} = temp_dates{numOfPeriod-i+1};
                end
                
%                 out.dates= dates;
%                 out.numOfPeriod = numOfPeriod;
                
            elseif strcmp(dateGenRule,'Forward')
                numOfPeriod = 0;
                i = 1;
                seed = initialDate;
                while 1

                    dates{end+1} = AddDate(seed,(i-1)*12/freq,'month');
                    numOfPeriod = numOfPeriod + 1;
                    if dates{i}.DateDiff(maturityDate) >= 0 break; end
                    i=i+1;
                end

                
%                 out.dates= dates;
%                 out.numOfPeriod = numOfPeriod;
            end
            
            % adjust eom
            if endOfMonth && IsEndOfMonth(seed)
                for i=1:numOfPeriod
                    dates{i} =calendar.Adjust(EndOfMonth(dates{i}),'Preceding');
                end
            else
                for i=1:numOfPeriod
                    dates{i} =calendar.Adjust(dates{i},bdc);
                end
            end
            
            out.dates = dates;
            out.numOfPeriod = numOfPeriod;
        end

        function out = ScheduleGenerateBDC(initialDate,maturityDate,freq,calendar,dateGenRule,bdc,dc,endOfMonth)
            
            startDates={};
            endDates = {};
            resetDates = {};
            payDates = {};
            dayCount = [];
            dayCountFraction = [];
            
            if strcmp(dateGenRule,'Backward')
                temp_startDates={};
                temp_endDates = {};
                temp_resetDates = {};
                temp_payDates = {};
                numOfPeriod = 0;
                i = 1;
                while 1

                    temp_endDates{end+1} = calendar.Advance(maturityDate,bdc,-(i-1)*12/freq,'month',endOfMonth);
                    temp_payDates{end+1}   = temp_endDates{i};
                    temp_startDates{end+1}   = calendar.Advance(maturityDate,bdc,-i*12/freq,'month',endOfMonth);
                    temp_resetDates{end+1} = temp_startDates{i};
                    
                    numOfPeriod = numOfPeriod + 1;
                    if temp_startDates{i}.DateDiff(initialDate) <= 0 break; end
                    i=i+1;
                end
                
                startDates= cell(numOfPeriod,1);
                endDates = cell(numOfPeriod,1);
                resetDates = cell(numOfPeriod,1);
                payDates = cell(numOfPeriod,1);
                dayCount  = zeros(numOfPeriod,1);
                dayCountFraction = zeros(numOfPeriod,1);
                
                for i=1:numOfPeriod
                    startDates{i} = temp_startDates{numOfPeriod-i+1};
                    resetDates{i} = temp_resetDates{numOfPeriod-i+1};
                    endDates{i} = temp_endDates{numOfPeriod-i+1};
                    payDates{i} = temp_payDates{numOfPeriod-i+1};
                    dayCount(i) = DayCount(startDates{i},endDates{i},dc);
                    dayCountFraction(i) = DayCountFraction(startDates{i},endDates{i},dc);
                end
                
                out.startDates= startDates;
                out.resetDates = resetDates;
                out.endDates = endDates;
                out.payDates = payDates;
                out.dayCount = dayCount;
                out.dayCountFraction = dayCountFraction;
                out.numOfPeriod = numOfPeriod;
                
            elseif strcmp(dateGenRule,'Forward')
                numOfPeriod = 0;
                i = 1;
                while 1

                    startDates{end+1} = calendar.Advance(initialDate,bdc,(i-1)*12/freq,'month',endOfMonth);
                    resetDates{end+1} = startDates{i};
                    endDates{end+1}   = calendar.Advance(initialDate,bdc,i*12/freq,'month',endOfMonth);
                    payDates{end+1}   = endDates{i};
                    dayCount = [dayCount;DayCount(startDates{i},endDates{i},dc)];
                    dayCountFraction = [dayCountFraction;DayCountFraction(startDates{i},endDates{i},dc)];
                    numOfPeriod = numOfPeriod + 1;
                    if endDates{i}.DateDiff(maturityDate) >= 0 break; end
                    i=i+1;
                end

                
                out.startDates= startDates;
                out.resetDates = resetDates;
                out.endDates = endDates;
                out.payDates = payDates;
                out.dayCount = dayCount;
                out.dayCountFraction = dayCountFraction;
                out.numOfPeriod = numOfPeriod;
            end
            
        end

        

  

    end

    

end





In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
